# Assignment 1: Data Parsing, Cleansing and Integration
## Task 1 and 2
#### Student Name: Toh Kah Hie
#### Student ID: s3936897

Date: 12/08/2022

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used:
* pandas
* re
* numpy
* sklearn
* itertools
* matplotlib
* datetime
* xml
* difflib
* Counter
* england (External .py created)

## Introduction
This assessment provides students with hands on experience with parsing and cleaning a dataset. First, given an xml file, the internal structure is carefully studied and all necessary information are extracted and gathered into a DataFrame. After that, data cleaning and formatting is performed to remove any duplicates, errors and fill in some null values. The cleaned dataset would then be saved into a csv file.

## Importing libraries 

In [1]:
import xml.etree.ElementTree as etree
from itertools import chain
from collections import Counter
import pandas as pd
import re
import datetime
import england
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import difflib
from matplotlib.cbook import boxplot_stats

## Task 1. Parsing Data

### 1.1. Examining and loading data

![Heading of the xml file](img/xml_header.jpg)
From a basic glance at the xml file in Notepad++, it can be seen the data is contained in \<JobAd\> and is nested under 2 tags \<Records\> \> \<Src\>

![Body of the xml file](img/xml_body.jpg)
Looking at the \<JobAd\>s it can be seen that (for now) every tag has attributes **OpenDate** and **CloseDate**, and their children tags include:
<ul>
<li>&lt;Id&gt;</li>
<li>&lt;Title&gt;</li>
<li>&lt;Location&gt;</li>
<li>&lt;Company&gt;</li>
<li>&lt;ContractTime&gt;</li>
<li>&lt;Category&gt;</li>
<li>&lt;Salary&gt;</li>
</ul>

Additionally, all children tags contain **Character Data** (\<![CDATA[]]\>), by which these data would not be parsed by the xml parser.

In [2]:
# Parse the xml file using ElementTree
tree = etree.parse("./s3936897_dataset1.xml")

In [3]:
# Get the root of the tree, which should be <Records>
root = tree.getroot()
print("Root tag: ",root.tag)

# Under the root tag, surprisingly there is 107 children and they are all under the tag 'Src'
print("Number of children of root: ",len(root))
print("Child tags contain: ",set([child.tag for child in root]))
print('Number of nested tag under all <Src>s: ',Counter(chain.from_iterable([[children.tag for children in c] for c in root])))

Root tag:  Records
Number of children of root:  107
Child tags contain:  {'Src'}
Number of nested tag under all <Src>s:  Counter({'JobAd': 50753, 'SourceName': 107})


In [4]:
# It is then hypothesized that there are several <JobAd>s under each <Src>
# Each children under each child of root is counted
print('Number of elements under root: ',len(root))
jb_ad_count = 0
print("\nFirst few <JobAD>s: ")

# For <Src> in root
for child in root:
    # Print first few <JobAd>
    if jb_ad_count <1000:
        print('Number of elements under ',child.tag,': ',len(child))
    
    # Counter to count number of nested tag for <Src>
    counter = Counter([nested.tag for nested in child])
    for key in counter.keys():
        # Print first few nested Tags
        if jb_ad_count <1000:
            print("Number of occurence of tag ",key,": ",counter[key])
        
        # Count the total number of <JobAd>
        if key == 'JobAd':
            jb_ad_count += counter[key]
            
    # Print first few <JobAd>        
    if jb_ad_count <1000:
        print("\n")
        
# Total number of <JobAd> present
print('\nNumber of <JobAd>s: ',jb_ad_count)

Number of elements under root:  107

First few <JobAD>s: 
Number of elements under  Src :  659
Number of occurence of tag  SourceName :  1
Number of occurence of tag  JobAd :  658


Number of elements under  Src :  7161
Number of occurence of tag  SourceName :  1
Number of occurence of tag  JobAd :  7160

Number of <JobAd>s:  50753


From the output above, it can be seen that there are a total of 50,573 \<JobAd\> tags and under each child \<Src\> there is 1 \<SourceName\> and several \<JobAd\>s. Now we will look at the child's attributes

In [5]:
#Get child tags present in each children tag
print("Child tags present in nested tags (tag,nested_tag): \n",set(chain.from_iterable([[(child_elements.tag,children.tag) for children in child_elements] for child_elements in child for child in root])))

print("\nAttributes for each tag (tag,attributes): ",set(chain.from_iterable([[(ch.tag,frozenset(ch.attrib.keys())) for ch in c] for c in root])))

Child tags present in nested tags (tag,nested_tag): 
 {('JobAd', 'Salary'), ('JobAd', 'ContractType'), ('JobAd', 'ContractTime'), ('JobAd', 'Category'), ('JobAd', 'Id'), ('JobAd', 'Title'), ('JobAd', 'Location'), ('JobAd', 'Company')}

Attributes for each tag (tag,attributes):  {('SourceName', frozenset()), ('JobAd', frozenset({'OpenDate', 'CloseDate'}))}


From here it can be seen that nested tags of \<JobAd\> includes 8 tags and 2 attributes (shown above) while \<SourceName\> contains no nested tags or attributes

In [6]:
limit = 5
i = 0
for children in root:
    # Print number of nested tags under <Src>
    counter = Counter(nested.tag for nested in children)
    
    # Print number of nested tags under child of <Src>
    ncounter = Counter(chain.from_iterable([[b.tag for b in a] for a in children]))
    
    val = list(ncounter.values())
    # If not all counts are the same means that there are values missing, print the first 5 with missing data
    if i < limit and not all(element == val[0] for element in val):
        print(counter)
        print(ncounter)
        print('\n')
        i += 1

Counter({'JobAd': 658, 'SourceName': 1})
Counter({'Id': 658, 'Title': 658, 'Location': 658, 'Company': 658, 'Category': 658, 'Salary': 625, 'ContractTime': 441})


Counter({'JobAd': 970, 'SourceName': 1})
Counter({'Id': 970, 'Title': 970, 'Location': 970, 'Category': 970, 'ContractTime': 966, 'Salary': 924, 'ContractType': 909, 'Company': 850})


Counter({'JobAd': 9224, 'SourceName': 1})
Counter({'Id': 9224, 'Title': 9224, 'Location': 9224, 'Company': 9224, 'Category': 9224, 'Salary': 9224, 'ContractTime': 9085, 'ContractType': 139})


Counter({'JobAd': 434, 'SourceName': 1})
Counter({'Id': 434, 'Title': 434, 'Location': 434, 'ContractTime': 434, 'Category': 434, 'Salary': 417})


Counter({'JobAd': 2536, 'SourceName': 1})
Counter({'Id': 2536, 'Title': 2536, 'Location': 2536, 'Category': 2536, 'Salary': 2536, 'Company': 2527, 'ContractType': 2368, 'ContractTime': 168})




From a number of counts above, it can be see that under \<JobAd\>, not all contains all tags listed previously

### 1.2 Parsing data into the required format

In [7]:
# Initialize a dictionary to store all data collected from the xml
dataDict = {}
dataDict['Id'] = []
dataDict['Title'] = []
dataDict['Location'] = []
dataDict['Company'] = []
dataDict['ContractType'] = []
dataDict['ContractTime'] = []
dataDict['Category'] = []
dataDict['Salary'] = []
dataDict['OpenDate'] = []
dataDict['CloseDate'] = []
dataDict['SourceName'] = []

for elem in tree.findall('Src'):
    for child in elem.findall('JobAd'):
        # Attributes
        dataDict['OpenDate'].append(child.attrib['OpenDate'])
        dataDict['CloseDate'].append(child.attrib['CloseDate'])
        # Nested Tags
        dataDict['Id'].append(child.findall('Id')[0].text)
        dataDict['Title'].append(child.findall('Title')[0].text)
        dataDict['Location'].append(child.findall('Location')[0].text)
        dataDict['ContractTime'].append(child.findall('ContractTime')[0].text if child.findall('ContractTime') else None)
        dataDict['Category'].append(child.findall('Category')[0].text if child.findall('Category') else None)
        dataDict['Salary'].append(child.findall('Salary')[0].text if child.findall('Salary') else None)
        dataDict['Company'].append(child.findall('Company')[0].text if child.findall('Company') else None)
        dataDict['ContractType'].append(child.findall('ContractType')[0].text if child.findall('ContractType') else None)
        
        # Child of the parent of each <JobAd>
        dataDict['SourceName'].append(elem.findall('SourceName')[0].text)

df = pd.DataFrame(dataDict)
# Save into a csv file to read afterwards
df.to_csv('job_ads.csv',index=False)
df.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70077187,PTS Electrician,Plymouth,Resourcing Solutions,None,contract,Engineering Jobs,14400,20121015T000000,20121214T000000,rengineeringjobs.com
1,67945005,Technical Support Engineer French Speaking,Southampton,Matchtech Group plc.,None,None,Engineering Jobs,24500,20120624T000000,20120724T000000,rengineeringjobs.com
2,71196024,Training Administrator Rotherham,Rotherham,Randstad,None,permanent,Engineering Jobs,17250,20120510T000000,20120808T000000,rengineeringjobs.com
3,72459197,Engineering Services Manager West London,West London,Randstad,None,permanent,Engineering Jobs,46500,20120705T120000,20121003T120000,rengineeringjobs.com
4,71684849,"Electrical Estimator Dartford, Kent",Dartford,Randstad,None,permanent,Engineering Jobs,45000,20120320T150000,20120519T150000,rengineeringjobs.com


## Task 2. Auditing and cleansing the loaded data

The newly saved data is loaded to check for errors ranging from duplicated values, null values, incorrect or incorrect format of values.

### Task 2.1. Loading and Examaning data

In [8]:
df = pd.read_csv('job_ads.csv')

In [9]:
errorDict = {}
errorDict['indexOfdf'] = []
errorDict['Id'] = []
errorDict['ColumnName'] = []
errorDict['Original'] = []
errorDict['Modified'] = []
errorDict['Modified'] = []
errorDict['ErrorType'] = []
errorDict['Fixing'] = []
# Initialize a dictionary that records data problems and actions made to fix it

In [10]:
# Check initial datatypes
df.dtypes

Id               int64
Title           object
Location        object
Company         object
ContractType    object
ContractTime    object
Category        object
Salary          object
OpenDate        object
CloseDate       object
SourceName      object
dtype: object

From the output above it can be seen that all data types are of **Object** data type

According to the specifications, columns and data types of the dataset should be as follows:
<ul>
    <li>Id : Integer (8 digit integer)</li>
    <li>TItle: Strings (Null values as 'non-specified')</li>
    <li>Location: String (Null values as 'non-specified')</li>
    <li>Company: String (Null values as 'non-specified')</li>
    <li>ContractType: String ['full_time','part_time','non-specified']</li>
    <li>ContractTime: String  ['permanent','contract','non-specified']</li>
    <li>Category: String ['IT Jobs','Healthcare & Nursing Jobs','Engineering Jobs','Accounting & FInance Jobs','Sales Jobs','Hospitality & Catering Jobs','Teaching Jobs','PR,Advertising & Marketing Jobs']</li>
    <li>Salary: Float (No null values allowed, 2 decimal places)</li>
    <li>OpenDate: Datetime (YYYY-MM-DD HH:MM:SS)</li>
    <li>CloseDate: Datetime (YYYY-MM-DD HH:MM:SS)</li>
    <li>SourceName: String (Null values should be 'non-specified')</li>
</ul>

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50753 entries, 0 to 50752
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            50753 non-null  int64 
 1   Title         50753 non-null  object
 2   Location      50753 non-null  object
 3   Company       48177 non-null  object
 4   ContractType  27320 non-null  object
 5   ContractTime  42009 non-null  object
 6   Category      50753 non-null  object
 7   Salary        50116 non-null  object
 8   OpenDate      50753 non-null  object
 9   CloseDate     50753 non-null  object
 10  SourceName    50753 non-null  object
dtypes: int64(1), object(10)
memory usage: 4.3+ MB


From the output above it can be see that columns 'Company','ContractType','ContractTime' and 'Salary' contain null values. Those values would be cleaned accordingly

In [12]:
def addToErrorDict(errorDict,index,Id,cname,ori,mod,etype,fixing):
#     num_invalid = len(df[filtr])
    errorDict['indexOfdf'].append(index)
    errorDict['Id'].append(Id)
    errorDict['ColumnName'].append(cname)
    errorDict['Original'].append(ori)
    errorDict['Modified'].append(mod)
    errorDict['ErrorType'].append(etype)
    errorDict['Fixing'].append(fixing)
    return errorDict

The above function adds necessary error info into the Error Dictionary

### Task 2.2 Examining and Cleaning Id Column

In [13]:
# Get duplicated Ids
df[df['Id'].duplicated()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


It seems that every single Id in the dataset is unique!

In [14]:
# Match the Id using regex to see if the Id has 8 digits
df[~df['Id'].astype('str').str.match(r'\d{8}',na=False)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


It can also be seen that every Id in the dataset has 8 digits, which matches the required format, no cleaning required!

### Task 2.3 Examining and Cleaning ContractType Column

In [15]:
# Get number of unique contract types
df['ContractType'].value_counts()

             11273
full_time    11244
-             3363
part_time     1440
Name: ContractType, dtype: int64

From the output above it can be seen that there are no typos to the values, but null values came in form of empty values or just a dash (-). All these values would be replaced with 'non-specified'

In [16]:
# Get contract types that is not full time or part time
valid_contracts = df['ContractType'].isin(['full_time','part_time'])
df[~valid_contracts]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70077187,PTS Electrician,Plymouth,Resourcing Solutions,NaN,contract,Engineering Jobs,14400,20121015T000000,20121214T000000,rengineeringjobs.com
1,67945005,Technical Support Engineer French Speaking,Southampton,Matchtech Group plc.,NaN,NaN,Engineering Jobs,24500,20120624T000000,20120724T000000,rengineeringjobs.com
2,71196024,Training Administrator Rotherham,Rotherham,Randstad,NaN,permanent,Engineering Jobs,17250,20120510T000000,20120808T000000,rengineeringjobs.com
3,72459197,Engineering Services Manager West London,West London,Randstad,NaN,permanent,Engineering Jobs,46500,20120705T120000,20121003T120000,rengineeringjobs.com
4,71684849,"Electrical Estimator Dartford, Kent",Dartford,Randstad,NaN,permanent,Engineering Jobs,45000,20120320T150000,20120519T150000,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50747,68704692,Graduate SoftwareWeb Developer,London,UNKNOWN auto added by vacany import,NaN,NaN,IT Jobs,25000,20120112T120000,20120312T120000,grb.uk.com
50748,72653749,GCHQ Graduate Technologists various roles,Cheltenham,GCHQ,NaN,NaN,Engineering Jobs,25400,20120412T000000,20120611T000000,grb.uk.com
50749,72420369,Graduate Operations Technical Analyst,Hungerford,UNKNOWN auto added by vacany import,NaN,NaN,IT Jobs,NaN,20130913T150000,20131112T150000,grb.uk.com
50750,72653751,Barclays Future Leaders Development Programmes,Hackney,Barclays,NaN,NaN,IT Jobs,36000,20130721T150000,20130919T150000,grb.uk.com


In [17]:
# Record error to errorDict
for ind in df[~valid_contracts].index:
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'ContractType','None','Non-specified','Missing values','Set missing values to non-specified')

In [18]:
# Replace all invalid values with non-specified
df.loc[~valid_contracts,'ContractType'] = 'non-specified'
df[~valid_contracts].head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70077187,PTS Electrician,Plymouth,Resourcing Solutions,non-specified,contract,Engineering Jobs,14400,20121015T000000,20121214T000000,rengineeringjobs.com
1,67945005,Technical Support Engineer French Speaking,Southampton,Matchtech Group plc.,non-specified,NaN,Engineering Jobs,24500,20120624T000000,20120724T000000,rengineeringjobs.com
2,71196024,Training Administrator Rotherham,Rotherham,Randstad,non-specified,permanent,Engineering Jobs,17250,20120510T000000,20120808T000000,rengineeringjobs.com
3,72459197,Engineering Services Manager West London,West London,Randstad,non-specified,permanent,Engineering Jobs,46500,20120705T120000,20121003T120000,rengineeringjobs.com
4,71684849,"Electrical Estimator Dartford, Kent",Dartford,Randstad,non-specified,permanent,Engineering Jobs,45000,20120320T150000,20120519T150000,rengineeringjobs.com


### Task 2.4 Examining and Cleaning ContractTime Column

In [19]:
df['ContractTime'].value_counts()

permanent    31010
contract      5610
-             3201
              2188
Name: ContractTime, dtype: int64

There are also missing values present in the column 'ContractTime'. They will all be set to non-specified as well

In [20]:
# Get contract times that is not permanent or contract
valid_contracts = df['ContractTime'].isin(['permanent','contract'])
# Record every error into errorDict
for ind in df[~valid_contracts].index:
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'ContractTime','None','Non-specified','Missing values','Set missing values to non-specified')

# Replace those invalid values as non-specified
df.loc[~valid_contracts,'ContractTime'] = 'non-specified'
df[~valid_contracts]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1,67945005,Technical Support Engineer French Speaking,Southampton,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,24500,20120624T000000,20120724T000000,rengineeringjobs.com
5,67944712,Theatre Lead/Deputy Manager,UK,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,NaN,20120424T000000,20120524T000000,rengineeringjobs.com
6,67946379,Technical Director Strategic Modelling,Woking,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,61500,20121214T000000,20130113T000000,rengineeringjobs.com
10,66160421,Structural CAD Technician,South Croydon,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,24500,20130221T120000,20130307T120000,rengineeringjobs.com
11,67946033,Offshore Project Manager of HVDC package,London,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,51000,20120617T000000,20120701T000000,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50741,72531462,EI/POS net developer,UK,JobG8,non-specified,non-specified,IT Jobs,60000,20130408T120000,20130607T120000,cardandpaymentjobs.com
50747,68704692,Graduate SoftwareWeb Developer,London,UNKNOWN auto added by vacany import,non-specified,non-specified,IT Jobs,25000,20120112T120000,20120312T120000,grb.uk.com
50748,72653749,GCHQ Graduate Technologists various roles,Cheltenham,GCHQ,non-specified,non-specified,Engineering Jobs,25400,20120412T000000,20120611T000000,grb.uk.com
50749,72420369,Graduate Operations Technical Analyst,Hungerford,UNKNOWN auto added by vacany import,non-specified,non-specified,IT Jobs,NaN,20130913T150000,20131112T150000,grb.uk.com


Here ends the cleaning for the column ContractTime

### Task 2.5 Examining and Loading Category Column

In [21]:
df['Category'].value_counts()

IT Jobs                             13170
Healthcare & Nursing Jobs            8067
Engineering Jobs                     7517
Accounting & Finance Jobs            6772
Sales Jobs                           4869
Hospitality & Catering Jobs          4376
Teaching Jobs                        3455
PR, Advertising & Marketing Jobs     2527
Name: Category, dtype: int64

From the value counts above, it can be seen that there are no typos on values for the Category column. Thus, this column is kept as is.

### Task 2.6 Examining and Loading Date Columns

In [22]:
df[['OpenDate','CloseDate']].head()

,OpenDate,CloseDate
0,20121015T000000,20121214T000000
1,20120624T000000,20120724T000000
2,20120510T000000,20120808T000000
3,20120705T120000,20121003T120000
4,20120320T150000,20120519T150000


It can be seen that the OpenDate and CloseDate column does not have the correct data type (Datetime). Thus, they will all be converted into the correct format

In [23]:
# Record error to errorDict
errorDict = addToErrorDict(errorDict,'ALL','ALL','OpenDate','String','DateTime','Incorrect Data format','Convert from string to datetime')

In [24]:
# Function to format date string into dates
def FormatDate(date_str,date_pattern,ordered):
    date_time =re.match(date_pattern,date_str)
    # If regex find no match, print out the date and do not change the vale
    if date_time == None:
        print("Invalid Date found")
        print(date_str)
        return date_str
    else:
        date_time = date_time.groups()
        # If date is YYYYMMDD
        if ordered:
            date = datetime.datetime.strptime(date_time[0],'%Y%m%d').date()
        else:
            date = datetime.datetime.strptime(date_time[0],'%Y%d%m').date()
        # Convert time string to time
        time = datetime.datetime.strptime(date_time[1],'%H%M%S').time()
        
        # Return converted datetime
        return datetime.datetime.combine(date,time)   

In [25]:
date_pattern = r'(20[1-2]\d(?:0[1-9]|1[0-2])(?:0[1-9]|[1-2][0-9]|3[0-1]))T(\d{6})'
df['OpenDate'] = df['OpenDate'].apply(lambda x: FormatDate(x,date_pattern,True))

Invalid Date found
20122405T120000


From here it shows that there is one date that is incorrect in format. it seems that the month and date is reversed. This incident will be recorded in the ErrorDict and corrected

In [26]:
rvr_date_pattern = r'(20[1-2]\d(?:0[1-9]|[1-2][0-9]|3[0-1])(?:0[1-9]|1[0-2]))T(\d{6})'
incorrect_date = df['OpenDate'].str.fullmatch(pat = rvr_date_pattern, na=False)
df[incorrect_date]

errorDict = addToErrorDict(errorDict,df.index[incorrect_date],df[incorrect_date]['Id'],'OpenDate','String','DateTime','Incorrect Data format (Reversed)','Convert from string to datetime')

df.loc[incorrect_date,'OpenDate'] = df.loc[incorrect_date,'OpenDate'].apply(lambda x: FormatDate(x,rvr_date_pattern,False))
df[incorrect_date]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8588,71335675,Technical Digital Consultant,Craigiehall,Hudson IT,non-specified,permanent,IT Jobs,26000,2012-05-24 12:00:00,20120623T120000,jobserve.com


Now that the column OpenDate is formatted, it is time to format CloseDate column

In [27]:
errorDict = addToErrorDict(errorDict,'ALL','ALL','CloseDate','String','DateTime','Incorrect Data format','Convert from string to datetime')

In [28]:
date_pattern = r'(20[1-2]\d(?:0[1-9]|1[0-2])(?:0[1-9]|[1-2][0-9]|3[0-1]))T(\d{6})'
df['CloseDate'] = df['CloseDate'].apply(lambda x: FormatDate(x,date_pattern,True))

It seems that there is no error within the CloseDate column and everything is converted to datetime safely. Now is the time to check if there are any violation in the dates (if OpenDate > CloseDate)

In [29]:
valid_duration = df['CloseDate'] > df['OpenDate']
df[~valid_duration]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
3931,66600505,AREA SALES EXECUTIVE,Leeds,Candidate Source,non-specified,permanent,Sales Jobs,30000,2014-01-13 12:00:00,2013-11-14 12:00:00,cv-library.co.uk
14972,68699721,Product Specialist Sports Therapy,West Midlands,Delta Consultants.,non-specified,permanent,Sales Jobs,25000,2013-10-06 12:00:00,2013-09-06 12:00:00,totaljobs.com
39133,71659260,Reservations Manager 6 Month Contract,London,Stafffinders,non-specified,non-specified,Hospitality & Catering Jobs,21500,2012-03-11 00:00:00,2012-01-11 00:00:00,jobs.catererandhotelkeeper.com
42944,71224150,SENIOR M&E/ELECTRICAL ENGINEERWATERBRISTOLCONT...,Bristol,Progressive Recruitment,non-specified,non-specified,Engineering Jobs,57600,2012-03-14 00:00:00,2012-02-13 00:00:00,gojobsearch.co.uk
44223,68849704,Telesales and Information Operatives,Wellingborough,Fox Resourcing,non-specified,non-specified,Sales Jobs,10742,2013-06-21 00:00:00,2013-06-07 00:00:00,jobstoday.co.uk


From the table above, there are several data with invalid duration, there is a possiblity that the user filled in the dates in the wrong order. Thus, these 2 values are switched over to deal with this error

In [30]:
df[['OpenDate','CloseDate']] = df[['OpenDate','CloseDate']].astype("string")
# Convert to string for the swapping
df.dtypes

Id               int64
Title           object
Location        object
Company         object
ContractType    object
ContractTime    object
Category        object
Salary          object
OpenDate        string
CloseDate       string
SourceName      object
dtype: object

In [31]:
# Record every error to errorDict
for ind in df.index[~valid_duration]:
    open_date = str(df['OpenDate'][ind])
    close_date = str(df['CloseDate'][ind])
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],"OpenDate,CloseDate"," ".join([open_date,close_date])," ".join([close_date,open_date]),'Invalid Duration','Switch values from the columns')

In [32]:
# Swap values
df.loc[~valid_duration,['OpenDate','CloseDate']] = df.loc[~valid_duration,['CloseDate','OpenDate']].values
df[~valid_duration]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
3931,66600505,AREA SALES EXECUTIVE,Leeds,Candidate Source,non-specified,permanent,Sales Jobs,30000,2013-11-14 12:00:00,2014-01-13 12:00:00,cv-library.co.uk
14972,68699721,Product Specialist Sports Therapy,West Midlands,Delta Consultants.,non-specified,permanent,Sales Jobs,25000,2013-09-06 12:00:00,2013-10-06 12:00:00,totaljobs.com
39133,71659260,Reservations Manager 6 Month Contract,London,Stafffinders,non-specified,non-specified,Hospitality & Catering Jobs,21500,2012-01-11 00:00:00,2012-03-11 00:00:00,jobs.catererandhotelkeeper.com
42944,71224150,SENIOR M&E/ELECTRICAL ENGINEERWATERBRISTOLCONT...,Bristol,Progressive Recruitment,non-specified,non-specified,Engineering Jobs,57600,2012-02-13 00:00:00,2012-03-14 00:00:00,gojobsearch.co.uk
44223,68849704,Telesales and Information Operatives,Wellingborough,Fox Resourcing,non-specified,non-specified,Sales Jobs,10742,2013-06-07 00:00:00,2013-06-21 00:00:00,jobstoday.co.uk


Now the all the duration is valid, all data of these 2 columns will be converted to DateTime

In [33]:
df['OpenDate'] = pd.to_datetime(df['OpenDate'])
df['CloseDate'] = pd.to_datetime(df['CloseDate'])
df.dtypes

Id                       int64
Title                   object
Location                object
Company                 object
ContractType            object
ContractTime            object
Category                object
Salary                  object
OpenDate        datetime64[ns]
CloseDate       datetime64[ns]
SourceName              object
dtype: object

Here we have done converting and fixing the columns OpenDate and CloseDate

### Task 2.7 Examining and Cleaning SourceName Column

In [34]:
# Get data with null SourceName
df[df['SourceName'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


It seems like every row of data has a SourceName!

In [35]:
# Get SourceName that contains something that is not work or space
df[df['SourceName'].str.contains('[^\s\w]+')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,70077187,PTS Electrician,Plymouth,Resourcing Solutions,non-specified,contract,Engineering Jobs,14400,2012-10-15 00:00:00,2012-12-14 00:00:00,rengineeringjobs.com
1,67945005,Technical Support Engineer French Speaking,Southampton,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,24500,2012-06-24 00:00:00,2012-07-24 00:00:00,rengineeringjobs.com
2,71196024,Training Administrator Rotherham,Rotherham,Randstad,non-specified,permanent,Engineering Jobs,17250,2012-05-10 00:00:00,2012-08-08 00:00:00,rengineeringjobs.com
3,72459197,Engineering Services Manager West London,West London,Randstad,non-specified,permanent,Engineering Jobs,46500,2012-07-05 12:00:00,2012-10-03 12:00:00,rengineeringjobs.com
4,71684849,"Electrical Estimator Dartford, Kent",Dartford,Randstad,non-specified,permanent,Engineering Jobs,45000,2012-03-20 15:00:00,2012-05-19 15:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50748,72653749,GCHQ Graduate Technologists various roles,Cheltenham,GCHQ,non-specified,non-specified,Engineering Jobs,25400,2012-04-12 00:00:00,2012-06-11 00:00:00,grb.uk.com
50749,72420369,Graduate Operations Technical Analyst,Hungerford,UNKNOWN auto added by vacany import,non-specified,non-specified,IT Jobs,NaN,2013-09-13 15:00:00,2013-11-12 15:00:00,grb.uk.com
50750,72653751,Barclays Future Leaders Development Programmes,Hackney,Barclays,non-specified,non-specified,IT Jobs,36000,2013-07-21 15:00:00,2013-09-19 15:00:00,grb.uk.com
50751,66372417,Graduate Engineer (Water industry),South Lanarkshire,Carrington West Limited,full_time,contract,Engineering Jobs,20000,2013-01-19 00:00:00,2013-04-19 00:00:00,thegraduate.co.uk


It seems like some of the source names are websites. They all look correct, thus they will be filtered out

In [36]:
df[df['SourceName'].str.contains('[^\s\w\.]+')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
658,67299034,Security Systems Sales Manager,East Midlands,SSR C T,non-specified,permanent,Sales Jobs,45000,2012-01-23 12:00:00,2012-02-22 12:00:00,cv-library.co.uk
659,66893534,Skills For Life Tutor (**** Hours),South West London,R3,non-specified,permanent,Teaching Jobs,22000,2013-11-15 15:00:00,2014-02-13 15:00:00,cv-library.co.uk
660,66925525,Trainee Web Developer,Byfleet,Clockwork Recruitment Ltd,non-specified,permanent,IT Jobs,22500,2012-06-23 00:00:00,2012-07-23 00:00:00,cv-library.co.uk
661,71556865,Licensed Animal Technician,Cambridge,S3 Science,non-specified,permanent,Healthcare & Nursing Jobs,19193,2012-09-17 12:00:00,2012-10-17 12:00:00,cv-library.co.uk
662,69079681,Assistant Project Manager Installation,Redhill,Zebra Technical Recruitment Ltd,non-specified,permanent,Engineering Jobs,35000,2012-02-19 00:00:00,2012-05-19 00:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50360,69928387,Head of Maths Faculty,Nottinghamshire,The Holgate School,non-specified,non-specified,Teaching Jobs,NaN,2012-10-27 15:00:00,2012-11-10 15:00:00,Teaching jobs - TES Connect
50361,67058705,Faculty Leader Science,Hertfordshire,Francis Combe Academy,non-specified,non-specified,Teaching Jobs,NaN,2013-10-07 15:00:00,2013-12-06 15:00:00,Teaching jobs - TES Connect
50362,71287706,Assistant Director MIS,Stoke-On-Trent,Stoke on Trent College Cauldon Campus,non-specified,non-specified,Teaching Jobs,39194.0,2012-02-08 00:00:00,2012-04-08 00:00:00,Teaching jobs - TES Connect
50363,71552520,Maths Subject Leader,Bournemouth,Harewood College,non-specified,non-specified,Teaching Jobs,5106.0,2013-02-09 12:00:00,2013-04-10 12:00:00,Teaching jobs - TES Connect


Some of them contains "-" as well, they seem to not be wrong thus we will continue

In [37]:
df[df['SourceName'].str.contains('[^\s\w\.\-]+')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


Here we end our examination for the column SourceName, it seems like all is well!

### Task 2.7 Examining and Cleaning Title Column

In [38]:
# Capitalize each word of title and record all into errorDict
df['Title'] = df['Title'].str.title()
errorDict = addToErrorDict(errorDict,'ALL','ALL','Title','word','Word','Incorrect capitalizations','Capitalize each word')
df['Title'].value_counts()

Paeds (Ot)                                                2
Assistant Brand Manager Fmcg                              2
Private Equity, Investment Associate                      2
Pts Electrician                                           1
Teacher Of Primary (**** Posts) Kinloss Primary School    1
                                                         ..
Microsoft Crm Dynamics  Bristol                           1
Bdm  Asset Finance / Leasing / Motor Finance              1
Account Executive  Agency Side                            1
Media Sales Executive With German                         1
Health & Social Care Tutor/Assessor  Bristol              1
Name: Title, Length: 50750, dtype: int64

From above it can be seen there are only 3 titles that is not unique

In [39]:
df[df['Title'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


It seems like there is no null values in the 'Title' column

In [40]:
# Get titles that has non alphanumeric characters
df[df['Title'].str.fullmatch('\W+')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
6782,50870368,****,UK,Rullion Engineering Ltd,non-specified,permanent,Engineering Jobs,34500,2013-09-03 12:00:00,2013-12-02 12:00:00,cv-library.co.uk


It seems like there is one data with just "\*"! The title of this data would then be set as 'Non-specified'.

In [41]:
invalid_title = df['Title'].str.fullmatch('\W+')
ind = df[invalid_title].index[0]

# Record error to Error Dict
errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Title',df['Title'][ind],'Non-specified','Missing values','Replace value as Non-specified')

# Set the incorrect title as non-specified
df.loc[invalid_title,'Title'] = 'non-specified'
df[invalid_title]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
6782,50870368,non-specified,UK,Rullion Engineering Ltd,non-specified,permanent,Engineering Jobs,34500,2013-09-03 12:00:00,2013-12-02 12:00:00,cv-library.co.uk


Now let's check for duplicated values

In [42]:
df[df[['Title']].duplicated(keep=False)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
44411,69120030,Assistant Brand Manager Fmcg,Gatwick,London4Jobs,full_time,permanent,"PR, Advertising & Marketing Jobs",29500,2012-09-06 12:00:00,2012-10-06 12:00:00,london4jobs.co.uk
44438,71295275,Assistant Brand Manager Fmcg,Gatwick,London4Jobs,full_time,permanent,"PR, Advertising & Marketing Jobs",29500,2012-09-06 12:00:00,2012-10-06 12:00:00,london4jobs.co.uk
44497,72132299,"Private Equity, Investment Associate",Birmingham,CH Associates,non-specified,permanent,Accounting & Finance Jobs,47000,2012-07-22 12:00:00,2012-08-21 12:00:00,accountancyagejobs.com
44568,71295288,"Private Equity, Investment Associate",Birmingham,CH Associates.,non-specified,permanent,Accounting & Finance Jobs,47000,2012-07-22 12:00:00,2012-08-21 12:00:00,accountancyagejobs.com
48044,70141144,Paeds (Ot),Cambridgeshire,Castlerock Recruitment Group,non-specified,permanent,Healthcare & Nursing Jobs,28000,2013-11-11 12:00:00,2013-12-11 12:00:00,tntjobs.co.uk
48126,71295314,Paeds (Ot),Cambridgeshire,Castlerock Recruitment Group,non-specified,permanent,Healthcare & Nursing Jobs,28000,2013-11-11 12:00:00,2013-12-11 12:00:00,tntjobs.co.uk


There are a total of 3 duplicated values in the dataset. The duplicated row would be removed.

In [43]:
duplicates = df['Title'].duplicated()
# Record to errorDict
for ind in df.index[duplicates]:
    errorDict = addToErrorDict(errorDict,'ALL',df['Id'][ind],'ALL','ALL','ALL','Duplicated values','Removed duplicated values')
    
# Drop duplicates
df.drop_duplicates(['Title'],inplace=True)
df[duplicates]

C:\Users\User\AppData\Local\Temp\ipykernel_2328\2512627482.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[duplicates]


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


It does seem like there are a lot of information residing in the titles, let's check if we could fill some null values of other columns through the titles

In [44]:
# Get titles that contains words like Time to find phrases like 'Part time'
df[df['Title'].str.contains(r'\b[tT]ime\b')].head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
181,68179073,Senior Administrator / Part Time 10Am To 300Pm,Hinckley,Fusion People,non-specified,contract,Engineering Jobs,15360,2013-09-01 00:00:00,2013-09-15 00:00:00,rengineeringjobs.com
740,71554929,Part Time Accounts Senior (Practice Background...,Sevenoaks,Prospect Recruitment Ltd,part_time,contract,Accounting & Finance Jobs,25000,2012-01-13 00:00:00,2012-01-27 00:00:00,cv-library.co.uk
884,69931029,Part Time Credit Controller Solihull,Solihull,Office Angels,non-specified,contract,Accounting & Finance Jobs,15360,2013-10-23 15:00:00,2013-11-06 15:00:00,cv-library.co.uk
918,68999863,Sales Service Advisor Part Time,UK,Osborne Appointments,non-specified,permanent,Sales Jobs,13843,2012-01-07 00:00:00,2012-03-07 00:00:00,cv-library.co.uk
994,71750572,Dermatology Specialist Nurse Part Time,Brighton,Equal Approach Limited,part_time,non-specified,Healthcare & Nursing Jobs,29000,2013-06-18 00:00:00,2013-08-17 00:00:00,cv-library.co.uk


Indeed, some of them has null ContractType but the title says 'Part time'!

In [45]:
# Get rows of data with Part time or full time phrases and is non-specified in ContractType
no_type = df['ContractType'] == 'non-specified'
has_type = df['Title'].str.contains(r'\b([pP]art|[fF]ull) [tT]ime\b')
indices = list(df[no_type&has_type].index)
for ind in indices:
    # Get matched group (part OR full)
    typ = re.search(r'\b([pP]art|[fF]ull) [tT]ime\b',df['Title'][ind]).groups()[0].lower()
    # join with _time
    new = typ+'_time'
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'ContractType','non-specified',new,'None but info in Title column','Fill none with found info')
    # Set 'ContractType' to a new value
    df.loc[ind,'ContractType'] = new
df[no_type&has_type].head()

C:\Users\User\AppData\Local\Temp\ipykernel_2328\2810980817.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_type = df['Title'].str.contains(r'\b([pP]art|[fF]ull) [tT]ime\b')


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
181,68179073,Senior Administrator / Part Time 10Am To 300Pm,Hinckley,Fusion People,part_time,contract,Engineering Jobs,15360,2013-09-01 00:00:00,2013-09-15 00:00:00,rengineeringjobs.com
884,69931029,Part Time Credit Controller Solihull,Solihull,Office Angels,part_time,contract,Accounting & Finance Jobs,15360,2013-10-23 15:00:00,2013-11-06 15:00:00,cv-library.co.uk
918,68999863,Sales Service Advisor Part Time,UK,Osborne Appointments,part_time,permanent,Sales Jobs,13843,2012-01-07 00:00:00,2012-03-07 00:00:00,cv-library.co.uk
1019,71557106,Staff Nurse Rgn/Rmn Nights Yeovil ******** Ph ...,Yeovil,SYK Recruitment,part_time,permanent,Healthcare & Nursing Jobs,24912,2012-03-15 00:00:00,2012-05-14 00:00:00,cv-library.co.uk
1883,70215539,Cad Designer East Sussex 36 Months Full Time,East Sussex,People4business Limited,full_time,contract,Engineering Jobs,38400,2013-07-21 15:00:00,2013-08-20 15:00:00,cv-library.co.uk


A total of 184 null values are filled! Now let's look at others!

In [46]:
no_time = df['ContractTime'] == 'non-specified'
has_time = df['Title'].str.contains(r'\b([cC]ontract|[pP]erm\w+)\b')
df[no_time&has_time]

C:\Users\User\AppData\Local\Temp\ipykernel_2328\3119724438.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_time = df['Title'].str.contains(r'\b([cC]ontract|[pP]erm\w+)\b')


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
3265,66902133,Part Time Marketing Coordinator (Contract),Taunton,Office Angels,part_time,non-specified,"PR, Advertising & Marketing Jobs",13000,2013-11-02 00:00:00,2013-12-02 00:00:00,cv-library.co.uk
19441,69600051,Iro Permanent Gwent,UK,Eden Brown Ltd,part_time,non-specified,Healthcare & Nursing Jobs,35877,2013-12-02 12:00:00,2014-01-01 12:00:00,totaljobs.com
22710,68834820,Catering Manager – Leading Contract Caterer – ...,UK,Rockford Recruitment,non-specified,non-specified,Hospitality & Catering Jobs,22000,2013-05-26 00:00:00,2013-07-25 00:00:00,caterer.com
22752,68832342,Chef Manager – London – Contract Catering – To...,South East London,Collins King & Associates Limited,non-specified,non-specified,Hospitality & Catering Jobs,35000,2012-05-26 15:00:00,2012-07-25 15:00:00,caterer.com
22864,70734291,Permanent Junior Sous Chef ****K **** Aa Ros...,Northamptonshire,Brigade Recruitment,non-specified,non-specified,Hospitality & Catering Jobs,22000,2013-12-11 15:00:00,2014-02-09 15:00:00,caterer.com
...,...,...,...,...,...,...,...,...,...,...,...
47519,69029679,Marketing Assistant Fleet (12 Month Contract),Fleet,NaN,non-specified,non-specified,"PR, Advertising & Marketing Jobs",22000,2012-11-03 00:00:00,2013-01-02 00:00:00,britishjobsonthe.net
47534,69026153,Mobile Electrical Engineer Commercial Contrac...,Slough,NaN,non-specified,non-specified,Engineering Jobs,30000,2013-03-03 00:00:00,2013-04-02 00:00:00,britishjobsonthe.net
47542,67211817,Accounting Lecturer ****Mth Fixed Term Contract,Southampton,NaN,non-specified,non-specified,Teaching Jobs,31000,2012-05-22 12:00:00,2012-06-05 12:00:00,britishjobsonthe.net
50207,72328226,Internal Auditor (3 Month Contract),Manchester,The Fragrance Shop,non-specified,non-specified,IT Jobs,,2012-01-09 12:00:00,2012-02-08 12:00:00,emptylemon.co.uk


Some titles also contains answers to ContractTime!

In [47]:
# Get rows that does not specify contract time but the title contains either contract or permanent
no_time = df['ContractTime'] == 'non-specified'
has_time = df['Title'].str.contains(r'\b([cC]ontract|[pP]ermanent)\b')
indices = list(df[no_time&has_time].index)
for ind in indices:
    # Get matched group (contract OR permanent)
    typ = re.search(r'\b([cC]ontract|[pP]ermanent)\b',df['Title'][ind]).groups()[0].lower()
    # Record to error dict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'ContractTime','non-specified',typ,'None but info in Title column','Fill none with found info')
    # Fill na with newly found information
    df.loc[ind,'ContractTime'] = typ
df[no_time&has_time].head()

C:\Users\User\AppData\Local\Temp\ipykernel_2328\924239738.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_time = df['Title'].str.contains(r'\b([cC]ontract|[pP]ermanent)\b')


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
3265,66902133,Part Time Marketing Coordinator (Contract),Taunton,Office Angels,part_time,contract,"PR, Advertising & Marketing Jobs",13000,2013-11-02 00:00:00,2013-12-02 00:00:00,cv-library.co.uk
19441,69600051,Iro Permanent Gwent,UK,Eden Brown Ltd,part_time,permanent,Healthcare & Nursing Jobs,35877,2013-12-02 12:00:00,2014-01-01 12:00:00,totaljobs.com
22710,68834820,Catering Manager – Leading Contract Caterer – ...,UK,Rockford Recruitment,non-specified,contract,Hospitality & Catering Jobs,22000,2013-05-26 00:00:00,2013-07-25 00:00:00,caterer.com
22752,68832342,Chef Manager – London – Contract Catering – To...,South East London,Collins King & Associates Limited,non-specified,contract,Hospitality & Catering Jobs,35000,2012-05-26 15:00:00,2012-07-25 15:00:00,caterer.com
22864,70734291,Permanent Junior Sous Chef ****K **** Aa Ros...,Northamptonshire,Brigade Recruitment,non-specified,permanent,Hospitality & Catering Jobs,22000,2013-12-11 15:00:00,2014-02-09 15:00:00,caterer.com


In [48]:
# Add all titles into as there are still possible erorrs in them
errorDict = addToErrorDict(errorDict,'ALL','ALL','Title','No','No','Possible incorrect values','Kept as is')

No further examinations are done on the Title column as there is too much values to consider, taking into account that there is only limited resources. Thus, here we complete our data cleaning for the column Title

### Task 2.8 Examining and Cleaning Location Column

In [49]:
# Format Location into words with first letter capitalized
errorDict = addToErrorDict(errorDict, 'ALL','ALL','Location','word','Word','Incorrect word format','Captalize each word')
df['Location'] = df['Location'].str.title()

print('Total number of unique locations: ',len(df['Location'].unique()))
df['Location'].unique()[:50]

Total number of unique locations:  488


array(['Plymouth', 'Southampton', 'Rotherham', 'West London', 'Dartford',
       'Uk', 'Woking', 'Essex', 'Ipswich', 'The City', 'South Croydon',
       'London', 'Cambridge', 'Manchester', 'Windsor', 'Redhill',
       'North East England', 'Chester', 'Fleet', 'Portsmouth',
       'Gateshead', 'Reigate', 'Maidenhead', 'North Yorkshire',
       'Blackpool', 'Telford', 'St. Albans', 'Newbury', 'Bournemouth',
       'Aberdeenshire', 'Peterborough', 'West Sussex', 'Bristol',
       'Salisbury', 'Midlothian', 'North London', 'Lewisham', 'Maidstone',
       'Tonbridge', 'Doncaster', 'South West England', 'Coventry', 'Kent',
       'South West London', 'Bath', 'Huntingdon', 'Crawley', 'Scotland',
       'Wales', 'Birmingham'], dtype=object)

From the output above it can be see that there are a total of 488 unique values for location. It looks like all these locations are states and cities of the United Kingdom. Thus, a list of states and cities of United Kingdom are collected and compared against the dataset get invalid locations.

In [50]:
# england.py contains england states and cities
england_states = [x.lower() for x in england.states]
england_cities = [x.lower() for x in england.cities]

# Check if location exists in the list above
valid_location = df['Location'].str.lower().isin(england_states+england_cities)

print('Number of locations after filtering: ',len(df[~valid_location]['Location'].value_counts()))

# Get 25 least repeated values within the dataset
df[~valid_location]['Location'].value_counts().nsmallest(25)

Number of locations after filtering:  110


Manchaster               1
Livepool                 1
Londn                    1
Hampshira                1
Cembridge                1
Oxfords                  2
Leads                    2
Warringtn                2
Croydan                  2
North Finchley           5
Sutton-In-Ashfield       7
St. Neots                8
Newcastle-Under-Lyme     9
Trent                    9
Canary Wharf             9
Lutterworth              9
Llandaff North          10
Broadfield              10
Cressington             10
Thorpe St. Andrew       10
Brent Cross             10
St. Ives                11
Gatwick                 11
King'S Lynn             11
Axbridge                11
Name: Location, dtype: int64

From the output above, we managed to cut down the locations needed to be evaluated to 110 values. Looking at the locations with least value counts, it can also be seen that when the count gets more than 5 or so, it is more likely to be a valid city. Thus, top 20 locations with the least value count is evaluated

In [51]:
# Function for finding closes match of a location
def find_closest_match(location,series,cut):
    data = list(series)
    return difflib.get_close_matches(location,set(data).difference([location]), n = 1,cutoff=cut)

In [52]:
# Find index the 25 least repeated "invalid" locations
least_25_locations = df[~valid_location]['Location'].value_counts().nsmallest(25).index

for location in least_25_locations:
    # Fint closest match
    print(location,find_closest_match(location,df['Location'],0.80))

Manchaster ['Manchester']
Livepool ['Liverpool']
Londn ['London']
Hampshira ['Hampshire']
Cembridge ['Cambridge']
Oxfords ['Oxford']
Leads ['Leeds']
Warringtn ['Warrington']
Croydan ['Croydon']
North Finchley []
Sutton-In-Ashfield []
St. Neots []
Newcastle-Under-Lyme []
Trent []
Canary Wharf []
Lutterworth []
Llandaff North []
Broadfield []
Cressington []
Thorpe St. Andrew []
Brent Cross []
St. Ives []
Gatwick []
King'S Lynn []
Axbridge ['Uxbridge']


From the output above it can be seen the first 9 values are most likely typos, while 'Axbridge' is a valid location in England. Thus, these values would be replaced by its closest match, which all seem to be accurate.

In [53]:
# Get 9 least repeated location names
top_9_locations = list(df[~valid_location]['Location'].value_counts().nsmallest(9).index)

df[df['Location'].isin(top_9_locations)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
3894,72635548,Remediation/Earthworks Site Manager Warrington,Warringtn,Penguin Recruitment,non-specified,permanent,Engineering Jobs,37500,2012-10-14 12:00:00,2012-12-13 12:00:00,cv-library.co.uk
3957,70607755,Design Engineer (Ce I),Warringtn,BPS,non-specified,permanent,Engineering Jobs,36000,2013-08-09 00:00:00,2013-11-07 00:00:00,cv-library.co.uk
27576,72187429,Branch Manager Manchester 1301573 Manchester,Manchaster,Prospect Health,full_time,non-specified,Healthcare & Nursing Jobs,32500,2012-08-17 00:00:00,2012-11-15 00:00:00,staffnurse.com
30794,69851673,Financial Services Sales Face To Face Liverpool,Livepool,Select Appointments,non-specified,permanent,Accounting & Finance Jobs,13000,2012-01-14 15:00:00,2012-03-14 15:00:00,jobsite.co.uk
33023,66434582,Citrix/System Engineer Oxford,Oxfords,NaN,non-specified,permanent,IT Jobs,37500,2012-11-17 15:00:00,2013-01-16 15:00:00,theitjobboard.co.uk
33815,68993982,Senior Credit Risk Modelling Analyst,Leads,MERJE,non-specified,permanent,Accounting & Finance Jobs,32000,2012-12-05 00:00:00,2013-01-04 00:00:00,jobsinrisk.com
35525,69041514,Senior Information Analyst,Leads,Venn Group Ltd,non-specified,non-specified,IT Jobs,48000,2013-01-12 15:00:00,2013-03-13 15:00:00,MyUkJobs
38125,68705032,Chef De Partie Seasonal Short Order Cooking R...,Londn,Chef Results,non-specified,non-specified,Hospitality & Catering Jobs,17000,2012-01-27 12:00:00,2012-02-10 12:00:00,jobs.catererandhotelkeeper.com
40477,66749067,Registered Mental Health Nurses Rmn,Croydan,NaN,part_time,non-specified,Healthcare & Nursing Jobs,38400,2013-02-01 00:00:00,2013-04-02 00:00:00,Jobcentre Plus
41703,69568721,Manufacturing Engineer Injection Moulding,Hampshira,Austin Fraser Ltd,full_time,permanent,Engineering Jobs,37500,2013-06-26 00:00:00,2013-09-24 00:00:00,energygenerationjobs.com


In [54]:
# Record to Error Dict before making changes
for ind in df.index[df['Location'].isin(top_9_locations)]:
    
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Location',df['Location'][ind],find_closest_match(df['Location'][ind],df['Location'].unique(),0.80)[0],'Mispelling','Replace mispelled with the correct word')

In [55]:
# Dictionary to replace the 9 faulty locations
replace_location = {'Manchaster': 'Manchester',
'Livepool': 'Liverpool',
'Londn': 'London',
'Hampshira': 'Hampshire',
'Cembridge': 'Cambridge',
'Oxfords': 'Oxford',
'Leads': 'Leeds',
'Warringtn': 'Warrington',
'Croydan': 'Croydon'}

df['Location'].replace(replace_location,inplace=True)
df['Location'].value_counts()

Uk                    7698
London                6564
South East London     2728
The City              1165
Central London         825
                      ... 
Bourne End               9
St. Neots                8
South Brent              7
Sutton-In-Ashfield       7
North Finchley           5
Name: Location, Length: 479, dtype: int64

Here, we are done formatting the 'Location' column

### Task 2.9 Examining and Cleaning Company Column

In [56]:
df[df['Company'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
9397,69215963,Solar Sales Consultant Inverness,Inverness,NaN,full_time,permanent,Sales Jobs,50000,2013-03-03 15:00:00,2013-06-01 15:00:00,fish4.co.uk
9405,71470861,Cnc Machinist / Cnc Machine Operator / Cnc Ope...,Keighley,NaN,full_time,permanent,Engineering Jobs,24960,2013-04-22 15:00:00,2013-06-21 15:00:00,fish4.co.uk
9416,70315006,P/T Cook,Uk,NaN,part_time,permanent,Hospitality & Catering Jobs,13612,2013-01-02 00:00:00,2013-04-02 00:00:00,fish4.co.uk
9442,71471052,Funds & Pricing Analyst,Uk,NaN,full_time,contract,Accounting & Finance Jobs,62000,2013-05-10 12:00:00,2013-06-09 12:00:00,fish4.co.uk
9471,69228189,"Canvasser, Field Marketing And Sales Promotor",Leeds,NaN,full_time,permanent,"PR, Advertising & Marketing Jobs",22800,2013-09-23 12:00:00,2013-11-22 12:00:00,fish4.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
49959,69931438,Revenue/Pricing Analyst,Lancashire,NaN,non-specified,permanent,"PR, Advertising & Marketing Jobs",27500.0,2012-07-12 15:00:00,2012-08-11 15:00:00,simplymarketingjobs.co.uk
50559,72121207,Assistant Support Workers X ****,Edinburgh,NaN,non-specified,contract,Healthcare & Nursing Jobs,16570,2012-03-03 12:00:00,2012-04-02 12:00:00,scotsman.com
50560,70769728,Magazine Advertising Sales Executive (Sports &...,Uk,NaN,non-specified,permanent,Sales Jobs,NaN,2013-10-29 12:00:00,2013-12-28 12:00:00,scotsman.com
50561,68223651,Community Care Workers,Edinburgh,NaN,non-specified,permanent,Healthcare & Nursing Jobs,17280,2013-06-23 00:00:00,2013-09-21 00:00:00,scotsman.com


There are several data with no companies. All these values would be set to non-specified

In [57]:
missing_company = df['Company'].isna()
indices = list(df[missing_company].index)

# Add to ErroDict
for ind in indices:
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company','None','Non-specified','Missing values','Replace value as Non-specified')
    
# Set null values as non-specified
df.loc[missing_company,'Company'] = 'non-specified'
df[missing_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
9397,69215963,Solar Sales Consultant Inverness,Inverness,non-specified,full_time,permanent,Sales Jobs,50000,2013-03-03 15:00:00,2013-06-01 15:00:00,fish4.co.uk
9405,71470861,Cnc Machinist / Cnc Machine Operator / Cnc Ope...,Keighley,non-specified,full_time,permanent,Engineering Jobs,24960,2013-04-22 15:00:00,2013-06-21 15:00:00,fish4.co.uk
9416,70315006,P/T Cook,Uk,non-specified,part_time,permanent,Hospitality & Catering Jobs,13612,2013-01-02 00:00:00,2013-04-02 00:00:00,fish4.co.uk
9442,71471052,Funds & Pricing Analyst,Uk,non-specified,full_time,contract,Accounting & Finance Jobs,62000,2013-05-10 12:00:00,2013-06-09 12:00:00,fish4.co.uk
9471,69228189,"Canvasser, Field Marketing And Sales Promotor",Leeds,non-specified,full_time,permanent,"PR, Advertising & Marketing Jobs",22800,2013-09-23 12:00:00,2013-11-22 12:00:00,fish4.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
49959,69931438,Revenue/Pricing Analyst,Lancashire,non-specified,non-specified,permanent,"PR, Advertising & Marketing Jobs",27500.0,2012-07-12 15:00:00,2012-08-11 15:00:00,simplymarketingjobs.co.uk
50559,72121207,Assistant Support Workers X ****,Edinburgh,non-specified,non-specified,contract,Healthcare & Nursing Jobs,16570,2012-03-03 12:00:00,2012-04-02 12:00:00,scotsman.com
50560,70769728,Magazine Advertising Sales Executive (Sports &...,Uk,non-specified,non-specified,permanent,Sales Jobs,NaN,2013-10-29 12:00:00,2013-12-28 12:00:00,scotsman.com
50561,68223651,Community Care Workers,Edinburgh,non-specified,non-specified,permanent,Healthcare & Nursing Jobs,17280,2013-06-23 00:00:00,2013-09-21 00:00:00,scotsman.com


In [58]:
# Get companies that does not contain alphanuerical characters
invalid_company = df['Company'].str.fullmatch(r'\W+')
df[invalid_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
24798,66425908,Support Workeraberdeenshire,Aberdeenshire,-,non-specified,non-specified,Healthcare & Nursing Jobs,15360,2013-04-24 15:00:00,2013-06-23 15:00:00,careworx.co.uk
24799,46629065,Registered Manager East Sussex Fostering Services,East Sussex,-,full_time,non-specified,Healthcare & Nursing Jobs,-,2012-01-08 12:00:00,2012-02-07 12:00:00,careworx.co.uk
24800,46626555,Consultant Clinical Psychologist Job Norwich,Norwich,-,full_time,non-specified,Healthcare & Nursing Jobs,50000,2012-03-14 00:00:00,2012-04-13 00:00:00,careworx.co.uk
24801,56209802,Staff Nurses Winchester,Winchester,-,non-specified,non-specified,Healthcare & Nursing Jobs,24333,2013-10-17 15:00:00,2013-10-31 15:00:00,careworx.co.uk
24802,64573309,Rgn Nights Stokeontrent Nights Perm Role Excel...,Stoke-On-Trent,-,full_time,non-specified,Healthcare & Nursing Jobs,-,2013-09-02 15:00:00,2013-11-01 15:00:00,careworx.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
49590,67027226,Media Relations Manager,Cambridgeshire,-,non-specified,contract,"PR, Advertising & Marketing Jobs",35000,2012-10-11 12:00:00,2012-12-10 12:00:00,michaelpage.co.uk
49591,67027220,Digital Campaign Manager,Leicestershire,-,non-specified,permanent,"PR, Advertising & Marketing Jobs",38500,2013-07-19 00:00:00,2013-10-17 00:00:00,michaelpage.co.uk
49592,72425166,Principal/Managing Consultant,Bath,-,non-specified,permanent,Accounting & Finance Jobs,80000,2013-05-15 15:00:00,2013-08-13 15:00:00,michaelpage.co.uk
49593,71142128,Pricing Analyst Telematics,Guildford,-,non-specified,permanent,Accounting & Finance Jobs,39000,2012-09-30 15:00:00,2012-11-29 15:00:00,michaelpage.co.uk


From the output above it can be seem that a lot of company names are labeled as "-" and there could be others but they are all non alphanumerical values. Thus, they will all be replaced with non-specified

In [59]:
indices = df[invalid_company].index
for ind in indices:
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',df['Company'][ind],'non-specified','Dummy values','Set to non-specified')
# Set to non-specified
df.loc[invalid_company,'Company'] = 'non-specified'
df[invalid_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
24798,66425908,Support Workeraberdeenshire,Aberdeenshire,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,15360,2013-04-24 15:00:00,2013-06-23 15:00:00,careworx.co.uk
24799,46629065,Registered Manager East Sussex Fostering Services,East Sussex,non-specified,full_time,non-specified,Healthcare & Nursing Jobs,-,2012-01-08 12:00:00,2012-02-07 12:00:00,careworx.co.uk
24800,46626555,Consultant Clinical Psychologist Job Norwich,Norwich,non-specified,full_time,non-specified,Healthcare & Nursing Jobs,50000,2012-03-14 00:00:00,2012-04-13 00:00:00,careworx.co.uk
24801,56209802,Staff Nurses Winchester,Winchester,non-specified,non-specified,non-specified,Healthcare & Nursing Jobs,24333,2013-10-17 15:00:00,2013-10-31 15:00:00,careworx.co.uk
24802,64573309,Rgn Nights Stokeontrent Nights Perm Role Excel...,Stoke-On-Trent,non-specified,full_time,non-specified,Healthcare & Nursing Jobs,-,2013-09-02 15:00:00,2013-11-01 15:00:00,careworx.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
49590,67027226,Media Relations Manager,Cambridgeshire,non-specified,non-specified,contract,"PR, Advertising & Marketing Jobs",35000,2012-10-11 12:00:00,2012-12-10 12:00:00,michaelpage.co.uk
49591,67027220,Digital Campaign Manager,Leicestershire,non-specified,non-specified,permanent,"PR, Advertising & Marketing Jobs",38500,2013-07-19 00:00:00,2013-10-17 00:00:00,michaelpage.co.uk
49592,72425166,Principal/Managing Consultant,Bath,non-specified,non-specified,permanent,Accounting & Finance Jobs,80000,2013-05-15 15:00:00,2013-08-13 15:00:00,michaelpage.co.uk
49593,71142128,Pricing Analyst Telematics,Guildford,non-specified,non-specified,permanent,Accounting & Finance Jobs,39000,2012-09-30 15:00:00,2012-11-29 15:00:00,michaelpage.co.uk


Now that dummy values are being accounted for, it is time to check for invalid values

In [60]:
# Get companies that has non alphanumeric characters but not non-specified
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+')
non_specified = df['Company'] == 'non-specified'
df[~valid_company & ~non_specified]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1,67945005,Technical Support Engineer French Speaking,Southampton,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,24500,2012-06-24 00:00:00,2012-07-24 00:00:00,rengineeringjobs.com
5,67944712,Theatre Lead/Deputy Manager,Uk,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,NaN,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
6,67946379,Technical Director Strategic Modelling,Woking,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,61500,2012-12-14 00:00:00,2013-01-13 00:00:00,rengineeringjobs.com
10,66160421,Structural Cad Technician,South Croydon,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,24500,2013-02-21 12:00:00,2013-03-07 12:00:00,rengineeringjobs.com
11,67946033,Offshore Project Manager Of Hvdc Package,London,Matchtech Group plc.,non-specified,non-specified,Engineering Jobs,51000,2012-06-17 00:00:00,2012-07-01 00:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50709,70768357,Perl Developer Junior / Senior ****K ****K,Uk,Orleston Associates Ltd.,full_time,non-specified,IT Jobs,32500,2013-05-01 00:00:00,2013-06-30 00:00:00,jobs.perl.org
50714,72118590,Senior Perl Developer Global Dot Com ****K ****K,London,Orleston Associates Ltd.,full_time,non-specified,IT Jobs,57500,2013-08-24 00:00:00,2013-10-23 00:00:00,jobs.perl.org
50730,69815649,Futures Dma Desk Support,London,Agora Search & Selection,non-specified,permanent,Accounting & Finance Jobs,90000.0,2013-06-13 15:00:00,2013-08-12 15:00:00,frontofficebanking.com
50747,68704692,Graduate Softwareweb Developer,London,UNKNOWN auto added by vacany import,non-specified,non-specified,IT Jobs,25000,2012-01-12 12:00:00,2012-03-12 12:00:00,grb.uk.com


It seems like some companies ends with a ".". These symbols would be replaced.

In [61]:
# Get companies ending with a .
company_dot = df['Company'].str.fullmatch('.*\.$',na=False)
indices = df[company_dot].index
for ind in indices:
    # edd to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',df['Company'][ind],df['Company'][ind][:-1],'Punctuation at the end','Remove punctuation')
# Remove the ending .
df.loc[company_dot,'Company'] = df['Company'][ind][:-1]
df[company_dot]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1,67945005,Technical Support Engineer French Speaking,Southampton,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,24500,2012-06-24 00:00:00,2012-07-24 00:00:00,rengineeringjobs.com
5,67944712,Theatre Lead/Deputy Manager,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,NaN,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
6,67946379,Technical Director Strategic Modelling,Woking,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,61500,2012-12-14 00:00:00,2013-01-13 00:00:00,rengineeringjobs.com
10,66160421,Structural Cad Technician,South Croydon,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,24500,2013-02-21 12:00:00,2013-03-07 12:00:00,rengineeringjobs.com
11,67946033,Offshore Project Manager Of Hvdc Package,London,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,51000,2012-06-17 00:00:00,2012-07-01 00:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
49920,72609749,Paediatric Neuro Rehab Staff Nurse Central Lo...,Central London,Orleston Associates Ltd,non-specified,permanent,Healthcare & Nursing Jobs,33465,2012-09-17 15:00:00,2012-10-17 15:00:00,nursingtimesjobs.com
49930,72609221,Senior Odp Surrey,Surrey,Orleston Associates Ltd,non-specified,permanent,Healthcare & Nursing Jobs,34000,2012-11-23 00:00:00,2013-01-22 00:00:00,nursingtimesjobs.com
50674,56281394,Chinese Speaking Pa Finance,London,Orleston Associates Ltd,full_time,permanent,Accounting & Finance Jobs,37500,2013-06-21 00:00:00,2013-08-20 00:00:00,jobsfinancial.com
50709,70768357,Perl Developer Junior / Senior ****K ****K,Uk,Orleston Associates Ltd,full_time,non-specified,IT Jobs,32500,2013-05-01 00:00:00,2013-06-30 00:00:00,jobs.perl.org


After this, let's check for other errors

In [62]:
# Valid company should contain alphanumerical characters separated by spaces
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+',na=False)
df[~valid_company & ~non_specified]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
117,69669159,Mechanical Engineer / Fe Analyst,Knutsford,Intec Ltd,non-specified,permanent,Engineering Jobs,42500,2013-03-23 12:00:00,2013-05-22 12:00:00,rengineeringjobs.com
230,68544534,Structured Cabling Engineers M4 Corridor,Slough,Intec Ltd,non-specified,permanent,Engineering Jobs,25200,2013-05-05 00:00:00,2013-08-03 00:00:00,rengineeringjobs.com
794,69083414,Multiskilled Shift Engineer,South East London,Mobile Technical Staff Technical Engineering,non-specified,permanent,Engineering Jobs,27500,2013-08-27 12:00:00,2013-11-25 12:00:00,cv-library.co.uk
877,72540018,Nursery Nurse Baby Room Bristol,Bristol,Hays Specialist Recruitment Further Education,non-specified,permanent,Teaching Jobs,16000,2013-03-10 15:00:00,2013-06-08 15:00:00,cv-library.co.uk
892,71125509,Hvac Maintenance Engineer Central London ****K...,The City,"Randstad Construction, Property Engineering",non-specified,permanent,Engineering Jobs,29000,2012-03-18 15:00:00,2012-05-17 15:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50687,69035743,Lead Calculations Engineer,Wolverhampton,Intec Limited,non-specified,permanent,Engineering Jobs,39000,2012-04-15 00:00:00,2012-05-15 00:00:00,renewablescareers.com
50699,67383954,Telesales Consultant Telemarketing / Coldcalling,Uk,Eclipse Limited,non-specified,permanent,Sales Jobs,20000,2013-12-11 15:00:00,2014-03-11 15:00:00,legalprospects.com
50730,69815649,Futures Dma Desk Support,London,Agora Search & Selection,non-specified,permanent,Accounting & Finance Jobs,90000.0,2013-06-13 15:00:00,2013-08-12 15:00:00,frontofficebanking.com
50747,68704692,Graduate Softwareweb Developer,London,UNKNOWN auto added by vacany import,non-specified,non-specified,IT Jobs,25000,2012-01-12 12:00:00,2012-03-12 12:00:00,grb.uk.com


It is strange that lookin at the values above, it seems like they should pass the regex as they all have alphanumerical characters. Let's check these values individually

In [63]:
df[~valid_company & ~non_specified]['Company']

117                                           Intec  Ltd
230                                           Intec  Ltd
794       Mobile Technical Staff   Technical Engineering
877      Hays Specialist Recruitment   Further Education
892          Randstad Construction, Property Engineering
                              ...                       
50687                                     Intec  Limited
50699                                   Eclipse  Limited
50730                           Agora Search & Selection
50747              UNKNOWN   auto added by vacany import
50749              UNKNOWN   auto added by vacany import
Name: Company, Length: 2674, dtype: object

So some of them has more than just 1 white spaces with the word 'Ltd' or 'Limited' at the end. These values would be further evaluated

In [64]:
multi_space_ltd = df['Company'].str.contains(r'\s\s+(?:[lL][tT][dD]|[lL]imited)',na=False)
df[multi_space_ltd]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
117,69669159,Mechanical Engineer / Fe Analyst,Knutsford,Intec Ltd,non-specified,permanent,Engineering Jobs,42500,2013-03-23 12:00:00,2013-05-22 12:00:00,rengineeringjobs.com
230,68544534,Structured Cabling Engineers M4 Corridor,Slough,Intec Ltd,non-specified,permanent,Engineering Jobs,25200,2013-05-05 00:00:00,2013-08-03 00:00:00,rengineeringjobs.com
1354,69540430,External Sales Electrical Wholesale,Leicester,ERS Ltd,non-specified,permanent,Sales Jobs,23500,2012-06-13 15:00:00,2012-09-11 15:00:00,cv-library.co.uk
1394,69933882,Trade Counter Sales Electrical Wholesale,Leeds,ERS Ltd,non-specified,permanent,Sales Jobs,16500,2012-09-11 00:00:00,2012-11-10 00:00:00,cv-library.co.uk
1477,72539789,Intervention Progressions Manager,Birmingham,Edustaff Ltd,non-specified,permanent,Teaching Jobs,25294,2013-04-04 00:00:00,2013-05-04 00:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50103,71747316,Engineer Surveyor (Derby),Derbyshire,Forces Recruitment Services Ltd,full_time,permanent,Engineering Jobs,30500.0,2013-10-29 00:00:00,2013-11-28 00:00:00,thecareerengineer.com
50111,66697323,Continuous Improvement Manager,Buckinghamshire,MPI Aviation Ltd,full_time,permanent,Engineering Jobs,37500.0,2013-01-01 12:00:00,2013-04-01 12:00:00,thecareerengineer.com
50466,71070728,Electrician/Building Service Engineer,Lancashire,Forces Recruitment Services Ltd,full_time,permanent,Engineering Jobs,29000,2013-01-29 00:00:00,2013-02-28 00:00:00,JobSearch
50687,69035743,Lead Calculations Engineer,Wolverhampton,Intec Limited,non-specified,permanent,Engineering Jobs,39000,2012-04-15 00:00:00,2012-05-15 00:00:00,renewablescareers.com


Looking at the values, it seems like there are a lot of companies with the term 'Ltd' and 'Limited'. Since they both mean the same thing, Lets replace all of them into one universal term 'Limited' first

In [65]:
# Function to replace ltd, limited, Ltd with Ltd
def replace_ltd(company):
    ltds = re.match(r'.*([lL](?:[tT][dD]|imited)\.?).*',company).groups()
    for ltd in ltds:
#         print('Before Altering: ',company)
        company = company.replace(ltd,'Ltd')
#         print('After Altering: ',company)
    return company

In [66]:
# Get companies with words ltd, Ltd, limited, Limited
limited = df['Company'].str.contains(r'[lL](?:[tT][dD]|imited)\.?',na=False)
indices = df[limited].index
for ind in indices:
    company = df['Company'][ind]
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',company,replace_ltd(company),'Multiple variations of a term','Set to a global term')

# Replace those values with one global value Ltd
df.loc[limited,'Company'] = df.loc[limited,'Company'].apply(lambda x: replace_ltd(x))
df[limited]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1,67945005,Technical Support Engineer French Speaking,Southampton,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,24500,2012-06-24 00:00:00,2012-07-24 00:00:00,rengineeringjobs.com
5,67944712,Theatre Lead/Deputy Manager,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,NaN,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
6,67946379,Technical Director Strategic Modelling,Woking,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,61500,2012-12-14 00:00:00,2013-01-13 00:00:00,rengineeringjobs.com
10,66160421,Structural Cad Technician,South Croydon,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,24500,2013-02-21 12:00:00,2013-03-07 12:00:00,rengineeringjobs.com
11,67946033,Offshore Project Manager Of Hvdc Package,London,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,51000,2012-06-17 00:00:00,2012-07-01 00:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50724,68538231,Treasury Sales Urgent,London,City Wharf Financial Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,30000.0,2012-02-20 15:00:00,2012-04-20 15:00:00,frontofficebanking.com
50736,68538230,Junior Corporate Fx Sales (X****) Immediate S...,London,City Wharf Financial Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,20000.0,2013-04-20 12:00:00,2013-07-19 12:00:00,frontofficebanking.com
50742,67142856,Senior Business Analyst Data Mining,Uk,Global Payments Network Ltd,non-specified,permanent,IT Jobs,50000,2012-01-30 12:00:00,2012-02-29 12:00:00,cardandpaymentjobs.com
50751,66372417,Graduate Engineer (Water Industry),South Lanarkshire,Carrington West Ltd,full_time,contract,Engineering Jobs,20000,2013-01-19 00:00:00,2013-04-19 00:00:00,thegraduate.co.uk


Now that every variations of 'Limited' is formatted uniformly, it is time to go back to checking multiple white spaces

In [67]:
# Get companies with 2 spaces before the word Ltd
multi_space_ltd = df['Company'].str.contains(r'\s\s+(?:[lL][tT][dD]|[lL]imited)',na=False)
df[multi_space_ltd]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
117,69669159,Mechanical Engineer / Fe Analyst,Knutsford,Intec Ltd,non-specified,permanent,Engineering Jobs,42500,2013-03-23 12:00:00,2013-05-22 12:00:00,rengineeringjobs.com
230,68544534,Structured Cabling Engineers M4 Corridor,Slough,Intec Ltd,non-specified,permanent,Engineering Jobs,25200,2013-05-05 00:00:00,2013-08-03 00:00:00,rengineeringjobs.com
1354,69540430,External Sales Electrical Wholesale,Leicester,ERS Ltd,non-specified,permanent,Sales Jobs,23500,2012-06-13 15:00:00,2012-09-11 15:00:00,cv-library.co.uk
1394,69933882,Trade Counter Sales Electrical Wholesale,Leeds,ERS Ltd,non-specified,permanent,Sales Jobs,16500,2012-09-11 00:00:00,2012-11-10 00:00:00,cv-library.co.uk
1477,72539789,Intervention Progressions Manager,Birmingham,Edustaff Ltd,non-specified,permanent,Teaching Jobs,25294,2013-04-04 00:00:00,2013-05-04 00:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50103,71747316,Engineer Surveyor (Derby),Derbyshire,Forces Recruitment Services Ltd,full_time,permanent,Engineering Jobs,30500.0,2013-10-29 00:00:00,2013-11-28 00:00:00,thecareerengineer.com
50111,66697323,Continuous Improvement Manager,Buckinghamshire,MPI Aviation Ltd,full_time,permanent,Engineering Jobs,37500.0,2013-01-01 12:00:00,2013-04-01 12:00:00,thecareerengineer.com
50466,71070728,Electrician/Building Service Engineer,Lancashire,Forces Recruitment Services Ltd,full_time,permanent,Engineering Jobs,29000,2013-01-29 00:00:00,2013-02-28 00:00:00,JobSearch
50687,69035743,Lead Calculations Engineer,Wolverhampton,Intec Ltd,non-specified,permanent,Engineering Jobs,39000,2012-04-15 00:00:00,2012-05-15 00:00:00,renewablescareers.com


Those white spaces would all be removed

In [68]:
# Function to remove the extra whitespace
def remove_whitespace(company):
    ltds = re.match(r'.*(\s\s+)Ltd.*',company).groups()
    for ltd in ltds:
#         print('Before Altering: ',company)
        company = company.replace(ltd,' ')
#         print('After Altering: ',company)
    return company

In [69]:
# Get companies with 2 white spaces together before Ltd
multispace_ltd = df['Company'].str.contains(r'\s\s+Ltd',na=False)
indices = df[multispace_ltd].index
for ind in indices:
    company = df['Company'][ind]
    new_company = remove_whitespace(company)
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',company,new_company,'Multiple white spaces before a term','Set to only 1 whitespace')
    
    # Replace the multiple whitespaces as 1 whitespace
    df.loc[ind,'Company'] = new_company
    
df[multispace_ltd]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
117,69669159,Mechanical Engineer / Fe Analyst,Knutsford,Intec Ltd,non-specified,permanent,Engineering Jobs,42500,2013-03-23 12:00:00,2013-05-22 12:00:00,rengineeringjobs.com
230,68544534,Structured Cabling Engineers M4 Corridor,Slough,Intec Ltd,non-specified,permanent,Engineering Jobs,25200,2013-05-05 00:00:00,2013-08-03 00:00:00,rengineeringjobs.com
1354,69540430,External Sales Electrical Wholesale,Leicester,ERS Ltd,non-specified,permanent,Sales Jobs,23500,2012-06-13 15:00:00,2012-09-11 15:00:00,cv-library.co.uk
1394,69933882,Trade Counter Sales Electrical Wholesale,Leeds,ERS Ltd,non-specified,permanent,Sales Jobs,16500,2012-09-11 00:00:00,2012-11-10 00:00:00,cv-library.co.uk
1477,72539789,Intervention Progressions Manager,Birmingham,Edustaff Ltd,non-specified,permanent,Teaching Jobs,25294,2013-04-04 00:00:00,2013-05-04 00:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50103,71747316,Engineer Surveyor (Derby),Derbyshire,Forces Recruitment Services Ltd,full_time,permanent,Engineering Jobs,30500.0,2013-10-29 00:00:00,2013-11-28 00:00:00,thecareerengineer.com
50111,66697323,Continuous Improvement Manager,Buckinghamshire,MPI Aviation Ltd,full_time,permanent,Engineering Jobs,37500.0,2013-01-01 12:00:00,2013-04-01 12:00:00,thecareerengineer.com
50466,71070728,Electrician/Building Service Engineer,Lancashire,Forces Recruitment Services Ltd,full_time,permanent,Engineering Jobs,29000,2013-01-29 00:00:00,2013-02-28 00:00:00,JobSearch
50687,69035743,Lead Calculations Engineer,Wolverhampton,Intec Ltd,non-specified,permanent,Engineering Jobs,39000,2012-04-15 00:00:00,2012-05-15 00:00:00,renewablescareers.com


Now that extra white spaces are remove, time to check for other errors

In [70]:
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+',na=False)
df[~valid_company & ~non_specified]['Company']

794       Mobile Technical Staff   Technical Engineering
877      Hays Specialist Recruitment   Further Education
892          Randstad Construction, Property Engineering
1102                   HPR Consultancy Ltd t/a HPRtalent
1152                      Cooper Lomaz   Bury St Edmunds
                              ...                       
50658                              Net Recruit.co.uk Ltd
50684                     Atkins Ltd   Water Environment
50730                           Agora Search & Selection
50747              UNKNOWN   auto added by vacany import
50749              UNKNOWN   auto added by vacany import
Name: Company, Length: 2212, dtype: object

It can be seen that one of the values contain the term 't/a'. This term seems to refer to "trading as". For example, "Legal name" t/a "Business name". Let's check if every single company is named the same

In [71]:
# Get companies with the term t/a
ta_companies = df['Company'].str.contains(r'[tT]/[aA]',na=False)
df[ta_companies]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1102,66600547,Internal Sales Executive Ppe/Workwear,Manchester,HPR Consultancy Ltd t/a HPRtalent,non-specified,permanent,Sales Jobs,16000,2013-04-14 15:00:00,2013-05-14 15:00:00,cv-library.co.uk
2279,69537392,Financial Engineer,The City,Foremost Recruitment Ltd t/a Pathway Personnel,non-specified,permanent,Accounting & Finance Jobs,57500,2012-05-23 15:00:00,2012-07-22 15:00:00,cv-library.co.uk
3035,70410175,Export Development Manager Equine Nutrition,Uk,RIG Veterinary Ltd T/A RIG Animal Health Recruit,non-specified,permanent,Sales Jobs,57500,2012-12-31 15:00:00,2013-03-31 15:00:00,cv-library.co.uk
4440,66600480,Business Development Manager Pet Nutrition,Cardiff,RIG Veterinary Ltd T/A RIG Animal Health Recruit,non-specified,permanent,Sales Jobs,32000,2012-03-27 00:00:00,2012-06-25 00:00:00,cv-library.co.uk
5397,70764450,Regulatory Affairs Manager Animal Nutrition,South East London,RIG Veterinary Ltd T/A RIG Animal Health Recruit,non-specified,permanent,Healthcare & Nursing Jobs,57500,2013-02-28 12:00:00,2013-05-29 12:00:00,cv-library.co.uk
6523,66179102,Sales Team Leader,Uk,HPR Consultancy Ltd t/a HPRtalent,non-specified,permanent,Sales Jobs,22500,2012-06-21 00:00:00,2012-09-19 00:00:00,cv-library.co.uk
6819,66925730,Windows 7 Roll Out Migration Support,Berkshire,HPR Consultancy Ltd t/a HPRtalent,non-specified,contract,IT Jobs,27600,2013-01-31 12:00:00,2013-04-01 12:00:00,cv-library.co.uk
7340,66600517,Territory Manager Animal Insurance,Newbury,RIG Veterinary Ltd T/A RIG Animal Health Recruit,non-specified,permanent,Sales Jobs,,2013-09-22 15:00:00,2013-12-21 15:00:00,cv-library.co.uk
8909,68628287,Head Of It And Development,Devon,Summit Computer Recruitment Ltd t/a itecopeople,non-specified,permanent,IT Jobs,,2012-05-25 15:00:00,2012-08-23 15:00:00,jobserve.com
21226,69545552,Cnet Software Developer,Uk,PPR Solutions t/a ByBox Tech Resource,non-specified,permanent,IT Jobs,42500,2012-07-30 00:00:00,2012-10-28 00:00:00,cwjobs.co.uk


Let's try the company name "NFP Grp Limited T/A First Point IT"

In [72]:
it_company = df['Company'].str.contains(r'First Point IT',na=False)
df[it_company][:15]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8431,68628064,"Senior Developer Ruby, Rails Top Track **** ...",London,First Point IT,non-specified,permanent,IT Jobs,62500,2013-03-28 00:00:00,2013-06-26 00:00:00,jobserve.com
8531,68628114,Sql Datawarehouse Developer Sql ****/**** Gai...,London,First Point IT,non-specified,permanent,IT Jobs,40000,2012-07-01 00:00:00,2012-09-29 00:00:00,jobserve.com
9387,68628127,Senior Business Intelligence (Bi)/Dwh Dataware...,London,First Point IT,non-specified,permanent,IT Jobs,50000,2013-11-04 00:00:00,2013-12-04 00:00:00,jobserve.com
11843,68712327,"Ppc Manager, Uk No**** Phenom, London, ****K",South East London,First Point IT,non-specified,permanent,"PR, Advertising & Marketing Jobs",36500 To 38500,2013-02-17 00:00:00,2013-04-18 00:00:00,totaljobs.com
11946,68098589,Training Manager/Consultant,South East London,First Point IT,non-specified,permanent,IT Jobs,57500,2012-02-06 00:00:00,2012-05-06 00:00:00,totaljobs.com
12174,71184872,"Web Analyst, Europe No**** Ecom, London, ****K",South East London,First Point IT,non-specified,permanent,IT Jobs,45000,2012-01-12 12:00:00,2012-04-11 12:00:00,totaljobs.com
15675,68712409,"R**** Oracle Financial Consultant London, **...",South East London,First Point IT,non-specified,contract,IT Jobs,50000,2012-10-23 00:00:00,2012-11-06 00:00:00,totaljobs.com
15958,68715405,Technical Lead Caspnet Mvc Scrum Tdd Hospitali...,South East London,First Point IT,non-specified,permanent,IT Jobs,60000,2012-08-24 00:00:00,2012-10-23 00:00:00,totaljobs.com
16584,71184863,Database Architect Sql Server London ********K,South East London,First Point IT,non-specified,permanent,IT Jobs,67500,2013-10-02 00:00:00,2013-12-01 00:00:00,totaljobs.com
18422,68715391,C Winforms Developer Dev Express Fx/Trading G...,South East London,First Point IT,non-specified,permanent,IT Jobs,46000,2013-05-11 15:00:00,2013-07-10 15:00:00,totaljobs.com


It can be seen that some does not have the 't/a's! In this case, phrases before the term 't/a' would be removed

In [73]:
# Remove phrases before the term t/a
def remove_ta(company):
    ltds = re.match(r'.*\s[tT][\\/][aA]\s(.*)',company).groups()
    # If there are any errors
    if len(ltds)>1:
        print(company,ltds)
        return company
    else:
        for ltd in ltds:
#             print('Before Altering: ',company)
            company = ltd
#             print('After Altering: ',company)
        return company

In [74]:
# Get data with companies containing the term t/a
ta_company = df['Company'].str.contains(r'\s[tT][\\/][aA]\s',na=False)
indices = df[ta_company].index
for ind in indices:
    company = df['Company'][ind]
    new_company = remove_ta(company)
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',company,new_company,'Contains extra information t/a','Remove extra information')
    # Remove phrases before t/a
    df.loc[ind,'Company'] = new_company
    
df[ta_company].head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1102,66600547,Internal Sales Executive Ppe/Workwear,Manchester,HPRtalent,non-specified,permanent,Sales Jobs,16000,2013-04-14 15:00:00,2013-05-14 15:00:00,cv-library.co.uk
2279,69537392,Financial Engineer,The City,Pathway Personnel,non-specified,permanent,Accounting & Finance Jobs,57500,2012-05-23 15:00:00,2012-07-22 15:00:00,cv-library.co.uk
3035,70410175,Export Development Manager Equine Nutrition,Uk,RIG Animal Health Recruit,non-specified,permanent,Sales Jobs,57500,2012-12-31 15:00:00,2013-03-31 15:00:00,cv-library.co.uk
4440,66600480,Business Development Manager Pet Nutrition,Cardiff,RIG Animal Health Recruit,non-specified,permanent,Sales Jobs,32000,2012-03-27 00:00:00,2012-06-25 00:00:00,cv-library.co.uk
5397,70764450,Regulatory Affairs Manager Animal Nutrition,South East London,RIG Animal Health Recruit,non-specified,permanent,Healthcare & Nursing Jobs,57500,2013-02-28 12:00:00,2013-05-29 12:00:00,cv-library.co.uk


We continue check for other possible errors

In [75]:
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+',na=False)
df[~valid_company & ~non_specified]['Company']

794       Mobile Technical Staff   Technical Engineering
877      Hays Specialist Recruitment   Further Education
892          Randstad Construction, Property Engineering
1152                      Cooper Lomaz   Bury St Edmunds
1259      Mobile Technical Staff   Technical Engineering
                              ...                       
50658                              Net Recruit.co.uk Ltd
50684                     Atkins Ltd   Water Environment
50730                           Agora Search & Selection
50747              UNKNOWN   auto added by vacany import
50749              UNKNOWN   auto added by vacany import
Name: Company, Length: 2173, dtype: object

Back to the multispace problem. It seems like sentences after the whitespace or commas contains some extra information

In [76]:
# Get companies with multiple spaces together
multispace_company = df['Company'].str.contains(r'(?:\s\s+|,\s*)',na=False)
df[multispace_company]['Company']

794       Mobile Technical Staff   Technical Engineering
877      Hays Specialist Recruitment   Further Education
892          Randstad Construction, Property Engineering
1152                      Cooper Lomaz   Bury St Edmunds
1259      Mobile Technical Staff   Technical Engineering
                              ...                       
49812      Sad Business School Ltd, University of Oxford
50359           Seashell Trust   Royal School Manchester
50684                     Atkins Ltd   Water Environment
50747              UNKNOWN   auto added by vacany import
50749              UNKNOWN   auto added by vacany import
Name: Company, Length: 1073, dtype: object

From above, it does seem like they are all extra information. Thus, values after those would be removed

In [77]:
# Remove extra information after whitespace
def remove_aft_whitespace(company):
    ltds = re.match(r'(.*)(?:\s\s+|\,\s*).*',company).groups()
    if len(ltds)>1:
        print(company,ltds)
        return company
    else:
        for ltd in ltds:
#             print('Before Altering: ',company)
            company = ltd
#             print('After Altering: ',company)
        return company

In [78]:
# Get companies with extra white spaces
multispace = df['Company'].str.contains(r'(?:\s\s+|,\s*)',na=False)
indices = df[multispace].index
for ind in indices:
    company = df['Company'][ind]
    new_company = remove_aft_whitespace(company)
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',company,new_company,'Contains extra information','Remove extra information')
    # Remove phrases after the multiple whitespaces
    df.loc[ind,'Company'] = new_company
    
df[multispace].head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
794,69083414,Multiskilled Shift Engineer,South East London,Mobile Technical Staff,non-specified,permanent,Engineering Jobs,27500,2013-08-27 12:00:00,2013-11-25 12:00:00,cv-library.co.uk
877,72540018,Nursery Nurse Baby Room Bristol,Bristol,Hays Specialist Recruitment,non-specified,permanent,Teaching Jobs,16000,2013-03-10 15:00:00,2013-06-08 15:00:00,cv-library.co.uk
892,71125509,Hvac Maintenance Engineer Central London ****K...,The City,Randstad Construction,non-specified,permanent,Engineering Jobs,29000,2012-03-18 15:00:00,2012-05-17 15:00:00,cv-library.co.uk
1152,68357073,Helpdesk Analyst,Norwich,Cooper Lomaz,non-specified,permanent,IT Jobs,17000,2013-07-21 12:00:00,2013-09-19 12:00:00,cv-library.co.uk
1259,69560338,Leading Hand (Section Leader),Uk,Mobile Technical Staff,non-specified,permanent,Engineering Jobs,29760,2013-05-22 00:00:00,2013-06-21 00:00:00,cv-library.co.uk


Now that there are all removed, let's hope that the errors are minimized

In [79]:
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+',na=False)
df[~valid_company & ~non_specified]['Company']

1384                     in pact.co.uk
1567         RecruitmentRevolution.com
2056         Recruitmentconsultant.com
2178         RecruitmentRevolution.com
2572         RecruitmentRevolution.com
                     ...              
50548           EasyWebRecruitment.com
50592    Financialadviser careers .Com
50608    Financialadviser careers .Com
50658            Net Recruit.co.uk Ltd
50730         Agora Search & Selection
Name: Company, Length: 1270, dtype: object

Those companies with .com seems to be valid. Let's check if there are same company names without the .com

In [80]:
# Get companies with the name recruitment revolution
names = ['RecruitmentRevolution','Recruitmentconsultant','Recruitment Revolution','Recruitment consultant','Recruitment Consultant','RecruitmentConsultant']
xcom_company = df['Company'].isin(names)
df[xcom_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
35555,68609180,Systems Administrator / 1St/2Nd Line Technical...,Reading,Recruitment Revolution,non-specified,non-specified,IT Jobs,25000,2012-08-06 15:00:00,2012-10-05 15:00:00,MyUkJobs
35697,62004478,Business Development Executive Leading Web/Mo...,Woking,Recruitment Revolution,non-specified,non-specified,"PR, Advertising & Marketing Jobs",30000,2012-05-29 00:00:00,2012-07-28 00:00:00,MyUkJobs
35713,62004672,Online Sales Executive Uk'S Fastest Growing J...,Fleet,Recruitment Revolution,non-specified,non-specified,"PR, Advertising & Marketing Jobs",16500,2012-09-24 00:00:00,2012-10-08 00:00:00,MyUkJobs
35731,62004687,Lamp / Php Developer Work With Clients Like C...,West London,Recruitment Revolution,non-specified,non-specified,IT Jobs,30000,2013-07-04 12:00:00,2013-09-02 12:00:00,MyUkJobs
35846,68609136,Tax Compliance Accountant Leading Financial P...,Bradford,Recruitment Revolution,non-specified,non-specified,Accounting & Finance Jobs,33500,2012-03-09 00:00:00,2012-03-23 00:00:00,MyUkJobs
35851,62004822,Hosting Technical Support Manager Top 5 Globa...,Southampton,Recruitment Revolution,non-specified,non-specified,IT Jobs,42500,2012-07-22 12:00:00,2012-08-21 12:00:00,MyUkJobs
35864,62003694,Product Sales Executive Rapid Growing 1 Milli...,South East London,Recruitment Revolution,non-specified,non-specified,Accounting & Finance Jobs,24000,2012-07-15 00:00:00,2012-07-29 00:00:00,MyUkJobs
35867,62003562,Lead Generation / Telesales Executive Top 5 G...,Southampton,Recruitment Revolution,non-specified,non-specified,IT Jobs,20000,2012-01-31 00:00:00,2012-03-31 00:00:00,MyUkJobs
35895,62004106,Digital Graphic Designer Leading Investment H...,London,Recruitment Revolution,non-specified,non-specified,Engineering Jobs,31000,2012-06-18 15:00:00,2012-08-17 15:00:00,MyUkJobs
35947,62004644,Field Charity Fundraiser Leading Animal Welfa...,Darlington,Recruitment Revolution,non-specified,non-specified,Sales Jobs,30000,2013-05-03 00:00:00,2013-07-02 00:00:00,MyUkJobs


Well, it seems that some of them are.. Thus, those '.com's with company names without would be removed

In [81]:
# Function to remove .com and .nets
def remove_com(company):
    # Get the word .com OR .net
    domain = re.match(r'.*\s*(\.[cC][oO][mM]|\.[nN][eE][tT]).*',company).groups()
    for names in domain:
        if names != None:
            # Replace the values with ''
            name = company.replace(names,'')
            # Split the word into multiple words by capital letters
            namelist = re.findall('[A-Z][^A-Z]*\s?', name)
            
            # Join them with space to form a new string
            namelist = [x.strip() for x in namelist]
            company_name = " ".join(namelist)
            
            # If no words with capital letters
            if len(namelist) == 0:
                # Use name as the company name
                company_name = name
            
            # Find closest match of the newly split company
            closest_matches = find_closest_match(company_name,df['Company'].unique(),0.85)
            # If the company name exists
            if company_name in df['Company'].unique():
                return company_name
            # If a close match is found
            elif len(closest_matches) != 0 and closest_matches[0] != company:
                print("Found close match for ",company_name," as ",find_closest_match(company_name,closest_matches,0.85))
                return company_name
            else:
                body = name.strip()
                return body

In [82]:
# Get companies with .com and .net present
com_company = df['Company'].str.contains(r'(\.[cC][oO][mM]|\.[nN][eE][tT])',na=False)
indices = list(df[com_company].index)
for ind in indices:
    company = df['Company'][ind]
    # Get the newly formatted values
    new_company = remove_com(company)
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',company,new_company,'Name varient','Set to existing names')
    # Replace values with correctly formatted ones
    df.loc[ind,'Company'] = new_company

C:\Users\User\AppData\Local\Temp\ipykernel_2328\1132304127.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  com_company = df['Company'].str.contains(r'(\.[cC][oO][mM]|\.[nN][eE][tT])',na=False)


Found close match for  Easy Web Recruitment  as  ['Easy Web Recruitment ']
Found close match for  Recruitment Revolution Ltd  as  ['Recruitment Revolution.Com Ltd']
Found close match for  Recruitment Consultant  as  ['Recruitmentconsultant']
Found close match for  Gemploy Ltd  as  ['Remploy Ltd']
Found close match for  Hunt4 Staff  as  ['Hunt 4 Staff']
Found close match for  Recruitment Consulatant  as  ['Recruitment Consultant']
Found close match for  Rental Cars  as  ['Rentalcars']


In [83]:
df[com_company].head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1567,68181706,Software Developer / Software Engineer (Net) ...,Warwick,Recruitment Revolution,non-specified,permanent,IT Jobs,35000,2012-09-08 15:00:00,2012-11-07 15:00:00,cv-library.co.uk
2056,69004660,Contracts Sales Engineer,Northampton,Recruitmentconsultant,non-specified,permanent,Sales Jobs,27500,2013-09-16 00:00:00,2013-12-15 00:00:00,cv-library.co.uk
2178,69001494,New Business Sales Executive Sms Text Messagi...,Worthing,Recruitment Revolution,non-specified,permanent,Sales Jobs,30000,2013-05-14 00:00:00,2013-06-13 00:00:00,cv-library.co.uk
2572,69017271,"Net Developer Aspnet, Sql Media Solutions Pr...",Sheffield,Recruitment Revolution,non-specified,permanent,IT Jobs,,2012-09-28 00:00:00,2012-12-27 00:00:00,cv-library.co.uk
2795,69022557,Front End Developer Exciting Tech Startup Bac...,Covent Garden,Recruitment Revolution,non-specified,permanent,IT Jobs,35000,2012-04-24 15:00:00,2012-07-23 15:00:00,cv-library.co.uk


Now that those values ending with '.com' is being taken care of, we move on to the next values

In [84]:
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+')
df[~valid_company & ~non_specified]['Company']

1384                                 in pact.co.uk
2937                                 in pact.co.uk
3126                                 in pact.co.uk
3174                                 in pact.co.uk
3279                                 in pact.co.uk
                           ...                    
50199    Guy's and St Thomas' NHS Foundation Trust
50229                                          B&Q
50460                   Sotheby's Institute of Art
50658                        Net Recruit.co.uk Ltd
50730                     Agora Search & Selection
Name: Company, Length: 1099, dtype: object

It seems like there are a lot of companies with '.co.uk'

In [85]:
# Get company with .co.uk
co_uk = df['Company'].str.contains(r'[\.\s][cC][oO][\.\s][uU][kK]')
df[co_uk]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
1384,71554842,Head Of Resource Planning,East Sussex,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,60000,2012-08-30 12:00:00,2012-10-29 12:00:00,cv-library.co.uk
2937,68994859,Team Manager Pension Payments,Surrey,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,28000,2012-11-05 15:00:00,2013-01-04 15:00:00,cv-library.co.uk
3126,68994403,Team Leader Award Winning Financial Services Co,Epsom,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,26500,2012-07-21 00:00:00,2012-10-19 00:00:00,cv-library.co.uk
3174,68994880,Sales Quality Consultant,Uk,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,25000,2013-03-14 15:00:00,2013-04-13 15:00:00,cv-library.co.uk
3279,71554835,Occupational Pensions Administrator,Birmingham,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,20500,2012-11-01 15:00:00,2012-12-01 15:00:00,cv-library.co.uk
3607,71554834,Experienced / Senior Pensions Administrator,Berkshire,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,27000,2013-01-04 00:00:00,2013-04-04 00:00:00,cv-library.co.uk
4186,68994873,Dc Pensions Administrator,Berkshire,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,23000,2013-07-22 15:00:00,2013-10-20 15:00:00,cv-library.co.uk
4270,68994862,"Pensions Team Leader, Leading Consultancy",Surrey,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,29000,2012-05-09 15:00:00,2012-08-07 15:00:00,cv-library.co.uk
4628,68994870,Experienced Pensions Administrator Leading Co...,Uk,in pact.co.uk,non-specified,permanent,Accounting & Finance Jobs,25500,2012-06-20 12:00:00,2012-08-19 12:00:00,cv-library.co.uk
4943,71295494,Networking Engineer/ Cabling Manager,Slough,Easyvacancy.co.uk,non-specified,permanent,Engineering Jobs,25000,2013-01-19 00:00:00,2013-02-18 00:00:00,cv-library.co.uk


The term .co.uk seems to be a domain extension. Let's see if same companies without the .co.uk exists. Let's try out Net Recruit and Advanse

In [86]:
ntr = df['Company'].str.match(r'[nN]et [rR]ecruit')
df[ntr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
30404,68347173,Laser Cutting Sales Estimator,Uk,Net Recruit.co.uk Ltd,non-specified,permanent,Sales Jobs,30000,2012-03-26 12:00:00,2012-05-25 12:00:00,jobsite.co.uk
35177,66127036,Systems Analyst,Cheshire,Net Recruit.co.uk Ltd,full_time,permanent,IT Jobs,30000,2012-06-20 12:00:00,2012-07-20 12:00:00,planetrecruit.com
50658,69535123,"Collection Advisor, Credit Management",Gloucestershire,Net Recruit.co.uk Ltd,full_time,permanent,Accounting & Finance Jobs,15500,2012-07-10 12:00:00,2012-08-09 12:00:00,jobsfinancial.com


In [87]:
ntr = df['Company'].str.match(r'Advanse')
df[ntr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
20101,66609779,Joint Head Of Ict,Uk,Advanse.co.uk,non-specified,permanent,IT Jobs,86000,2012-09-10 12:00:00,2012-12-09 12:00:00,cwjobs.co.uk
44512,67774574,Uk Financial Controller,Bristol,Advanse,non-specified,permanent,Accounting & Finance Jobs,47500,2013-01-11 00:00:00,2013-02-10 00:00:00,accountancyagejobs.com


There are indeed some without the 'co.uk.'!

In [88]:
# Function similar to remove_com but removes .co.uk
def remove_couk(company):
    domain = re.match(r'.*\s*([\.\s][cC][oO][\.\s][uU][kK]).*',company).groups()[0]
    name = company.replace(domain,'')
    
    # SPlit namelist by capital letters and form a new string
    namelist = re.findall('[A-Z][^A-Z]*\s?', name)
    company_name = " ".join(namelist).strip()
    
    # If no capital letters, use the name without the .co.uk
    if len(namelist) == 0:
        company_name = name
        
    # Get closest match
    closest_match = find_closest_match(company_name,df['Company'].unique(),0.85)
    
    # Return the company name if it exists
    if company_name in df['Company'].unique():
        return company_name
    # If a close match is found
    elif len(closest_match) != 0 and closest_match[0] != company:
        print("Found close match for ",company_name," as ",closest_match)
        return closest_match[0]
    else:
        body = name.strip()
        return body

In [89]:
indices = list(df[co_uk].index)
for ind in indices:
    company = df['Company'][ind]
    new_company = remove_couk(company)
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',company,new_company,'Name varient','Set to existing names')
    # Replace values with newly formatted values
    df.loc[ind,'Company'] = new_company

Found close match for  Easyvacancy  as  ['Easy Vacancy']
Found close match for  Advanse  as  ['Advanse ']
Found close match for  Net  Recruit  Ltd  as  ['New Recruit Ltd']
Found close match for  N D K  Consulting  Ltd  as  ['NDK Consulting Ltd']
Found close match for  Net  Recruit  Ltd  as  ['New Recruit Ltd']
Found close match for  Motorecruit  as  ['Moto Recruit']
Found close match for  Motorecruit  as  ['Moto Recruit']
Found close match for  Easyvacancy  as  ['easyvacancy']
Found close match for  Blue Town Online  as  ['BlueTown Online']
Found close match for  Blue Town Online  as  ['BlueTown Online']
Found close match for  Blue Town Online  as  ['BlueTown Online']
Found close match for  Blue Town Online  as  ['BlueTown Online']
Found close match for  Blue Town Online  as  ['BlueTown Online']
Found close match for  Blue Town Online  as  ['BlueTown Online']
Found close match for  Blue Town Online  as  ['BlueTown Online']
Found close match for  Blue Town Online  as  ['BlueTown Online'

Other than removing .co.uk, some of them are also replaced as to its closest match

In [90]:
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+')
df[~valid_company & ~non_specified]['Company']

6112                                       CVL:LDN
7904                       C.O.A.L IT Services Ltd
7950            Lawrence Harvey Search & Selection
7977                       Canary Wharf & City Ltd
8025                       C.O.A.L IT Services Ltd
                           ...                    
50186    States of Jersey Health & Social Services
50199    Guy's and St Thomas' NHS Foundation Trust
50229                                          B&Q
50460                   Sotheby's Institute of Art
50730                     Agora Search & Selection
Name: Company, Length: 1052, dtype: object

There seems to be some value with the ":" symbol, let's check them

In [91]:
# Get companies with symbol :
colon_company = df['Company'].str.contains(r':')
df[colon_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
6112,69021870,Senior It Technician (3Rd Line) Excellent Uk ...,North West London,CVL:LDN,non-specified,permanent,IT Jobs,36000,2012-04-06 15:00:00,2012-07-05 15:00:00,cv-library.co.uk
10875,68696700,Senior Account Director Automotive,Warwick,id:recruitment,non-specified,permanent,"PR, Advertising & Marketing Jobs",45000,2012-05-06 12:00:00,2012-07-05 12:00:00,totaljobs.com
11755,68681772,Senior Pr Account Director,Uk,id:recruitment,non-specified,permanent,"PR, Advertising & Marketing Jobs",52500,2013-07-27 00:00:00,2013-10-25 00:00:00,totaljobs.com
13963,68694672,Studio Manager Creative,Leicester,id:recruitment,non-specified,permanent,"PR, Advertising & Marketing Jobs",32500,2012-08-26 15:00:00,2012-09-25 15:00:00,totaljobs.com
18636,68712256,Account Manager Retail,Rotherham,id:recruitment,non-specified,permanent,"PR, Advertising & Marketing Jobs",28500,2012-09-29 15:00:00,2012-10-29 15:00:00,totaljobs.com
30100,69687486,Digital Manager,Uk,sk:n Clinics,non-specified,permanent,IT Jobs,35000,2013-11-12 00:00:00,2014-01-11 00:00:00,jobsite.co.uk
41642,69054425,Mobile Planner / Buyer,London,Section:Media Ltd,full_time,permanent,Sales Jobs,26000,2012-06-01 12:00:00,2012-08-30 12:00:00,mediaweekjobs.co.uk
41671,69055842,Promotions Account Manager,Central London,Section:Media Ltd,full_time,permanent,Sales Jobs,28500,2012-10-25 12:00:00,2012-12-24 12:00:00,mediaweekjobs.co.uk


Searching up these values, it seems like 'sk:n Clinics' and 'Section:Media Ltd' are actual companies. Thus, it close matches would be searched, if there are no close matches then we will keep as is

In [92]:
indices = list(df[colon_company].index)
# Find close matches
for ind in indices:
    print(df['Company'][ind],find_closest_match(df['Company'][ind],df['Company'].unique(),0.85))

CVL:LDN []
id:recruitment []
id:recruitment []
id:recruitment []
id:recruitment []
sk:n Clinics []
Section:Media Ltd []
Section:Media Ltd []


Since they are no close matches, they will be kept as is but it will be recorded into the ErrorDict

In [93]:
indices = list(df[colon_company].index)
for ind in indices:
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',df['Company'][ind],'','Company contains : character','Kept as is')

In [94]:
colon_company = df['Company'].str.contains(r':')
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+')
df[~valid_company & ~non_specified & ~colon_company]['Company']

7904                       C.O.A.L IT Services Ltd
7950            Lawrence Harvey Search & Selection
7977                       Canary Wharf & City Ltd
8025                       C.O.A.L IT Services Ltd
8044                    Digital Guru's Recruitment
                           ...                    
50186    States of Jersey Health & Social Services
50199    Guy's and St Thomas' NHS Foundation Trust
50229                                          B&Q
50460                   Sotheby's Institute of Art
50730                     Agora Search & Selection
Name: Company, Length: 1044, dtype: object

A lot of the remaining data contains the '&' characters

In [95]:
# Get companies containing the & character
amp_company = df['Company'].str.contains(r'&')
df[amp_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
7950,68627844,Sap Bw Consultant Birmingham/Manchester ****K,West Midlands,Lawrence Harvey Search & Selection,non-specified,permanent,IT Jobs,45000,2013-03-13 00:00:00,2013-04-12 00:00:00,jobserve.com
7977,71774937,"Biztalk Specialist Global Corporation, Egham ...",Egham,Canary Wharf & City Ltd,non-specified,permanent,IT Jobs,,2013-12-09 15:00:00,2014-03-09 15:00:00,jobserve.com
8096,72372986,Qa Automated Tester,Central London,Hudson Banking & Finance,non-specified,permanent,IT Jobs,70000,2012-10-08 15:00:00,2012-11-07 15:00:00,jobserve.com
8324,71336370,"Test Analyst, System, Integration, Uat, Siebel...",London,Capita Assurance & Testing,non-specified,contract,IT Jobs,60000,2012-04-30 12:00:00,2012-05-30 12:00:00,jobserve.com
8559,68628231,Oracle Bi Obiee Consultant London ****K****K,London,Lawrence Harvey Search & Selection,non-specified,permanent,IT Jobs,75000,2013-10-12 15:00:00,2013-12-11 15:00:00,jobserve.com
...,...,...,...,...,...,...,...,...,...,...,...
48349,69133890,European Sales And Marketing Administrator – C...,Uk,Veenus Hotels & Experiences,non-specified,permanent,"PR, Advertising & Marketing Jobs",18750,2013-06-05 15:00:00,2013-09-03 15:00:00,Multilingualvacancies
49764,70578346,B**** Fire Safety Engineer,Uk,Health & Safety Laboratory,non-specified,non-specified,Teaching Jobs,34635,2012-01-23 12:00:00,2012-02-22 12:00:00,Jobs Ac
50186,71356729,Senior Staff Nurses,Uk,States of Jersey Health & Social Services,non-specified,non-specified,Healthcare & Nursing Jobs,35926,2013-08-06 15:00:00,2013-10-05 15:00:00,rcnbulletinjobs.co.uk
50229,72328268,B&Q Finance Business Partner,Hampshire,B&Q,non-specified,permanent,IT Jobs,42500,2013-05-13 00:00:00,2013-06-12 00:00:00,emptylemon.co.uk


Let's dive deeper into some of the values

In [96]:
# Get companies with the name Lawrence Harvey
df[df['Company'].str.contains('Lawrence Harvey')]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
7950,68627844,Sap Bw Consultant Birmingham/Manchester ****K,West Midlands,Lawrence Harvey Search & Selection,non-specified,permanent,IT Jobs,45000,2013-03-13 00:00:00,2013-04-12 00:00:00,jobserve.com
8559,68628231,Oracle Bi Obiee Consultant London ****K****K,London,Lawrence Harvey Search & Selection,non-specified,permanent,IT Jobs,75000,2013-10-12 15:00:00,2013-12-11 15:00:00,jobserve.com
8574,72160555,Principal/Senior Bi Consultant London ****K*...,London,Lawrence Harvey Search & Selection,non-specified,permanent,IT Jobs,90000,2012-03-27 12:00:00,2012-06-25 12:00:00,jobserve.com
17940,71680779,French Corporate Sales Consultant London City,London,Lawrence Harvey Search & Selection,non-specified,permanent,Sales Jobs,26500,2013-02-26 15:00:00,2013-05-27 15:00:00,totaljobs.com
19832,72549672,Sas Consultant London **** / ****K,South East London,Lawrence Harvey,non-specified,permanent,IT Jobs,55000,2012-02-06 15:00:00,2012-03-07 15:00:00,cwjobs.co.uk
20196,71565739,Vdi Desktop Citrix Architect/Presales (Ccia/Cc...,Uk,Lawrence Harvey,non-specified,permanent,IT Jobs,70000,2012-06-26 15:00:00,2012-08-25 15:00:00,cwjobs.co.uk
20467,70781767,Consultant Sap (Sd / Mm / Hum / Wm) Conseil ...,Uk,Lawrence Harvey,non-specified,permanent,IT Jobs,40055,2013-10-07 00:00:00,2014-01-05 00:00:00,cwjobs.co.uk
20833,69038417,Oracle Dba Leeds ****K ****K,Leeds,Lawrence Harvey,non-specified,permanent,IT Jobs,55000,2012-07-31 15:00:00,2012-09-29 15:00:00,cwjobs.co.uk
21016,71564994,Sap Data Migration Consultant Bodsmanchesteren...,Uk,Lawrence Harvey,non-specified,permanent,IT Jobs,47500,2012-11-18 00:00:00,2012-12-18 00:00:00,cwjobs.co.uk
21037,71566372,Principal Microsoft Sccm Desktop Consultant *...,Uk,Lawrence Harvey,non-specified,permanent,IT Jobs,65000,2013-10-02 15:00:00,2013-11-01 15:00:00,cwjobs.co.uk


It looks like companies may or may not contain the 'Search & Selection' term! All these value would be looked up and removed

In [97]:
# Get companies with the phrase search and selection
ss = df['Company'].str.contains('[sS]earch\s(&|and)?\s?[sS]election.*')
df[ss]

C:\Users\User\AppData\Local\Temp\ipykernel_2328\44381664.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ss = df['Company'].str.contains('[sS]earch\s(&|and)?\s?[sS]election.*')


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
2724,70770700,Cisco Network Analyst,Central London,Bishop Search Selection,non-specified,contract,IT Jobs,90000,2013-03-02 15:00:00,2013-03-16 15:00:00,cv-library.co.uk
2889,69081556,Semi Skilled Building Maintenance Engineer,Dundee,PDA Search Selection,non-specified,permanent,Engineering Jobs,22000,2013-09-20 12:00:00,2013-11-19 12:00:00,cv-library.co.uk
3180,70219038,Bms Support Engineer,Scotland,PDA Search Selection,non-specified,permanent,Engineering Jobs,,2013-11-02 00:00:00,2014-01-31 00:00:00,cv-library.co.uk
4255,69012180,Scientific Director,London,ID Search Selection,non-specified,permanent,Healthcare & Nursing Jobs,55000,2012-09-02 12:00:00,2012-12-01 12:00:00,cv-library.co.uk
4394,67903619,"Agile Java Developer, Online Gaming",Central London,Bishop Search Selection,non-specified,permanent,IT Jobs,53000,2013-05-21 00:00:00,2013-07-20 00:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
49462,71375525,Qualified Accountant Immediate Start,Belfast,Bond Search Selection Ltd,full_time,contract,Accounting & Finance Jobs,32500,2012-09-30 15:00:00,2012-11-29 15:00:00,nijobs.com
49498,67637107,Finance Business Analyst,Croydon,Greybridge Search and Selection Ltd,non-specified,permanent,Accounting & Finance Jobs,55000,2013-09-03 15:00:00,2013-10-03 15:00:00,onlineinsurancejobs.co.uk
49694,69133532,Senior Manager It Project & Program Risk Assu...,Central London,CGS Search Selection,full_time,permanent,Accounting & Finance Jobs,90000,2013-05-16 15:00:00,2013-08-14 15:00:00,careersinaudit.com
50014,70574295,Hr Project Manager (6 Months Temporary With St...,North West London,PDA Search and Selection Ltd,full_time,contract,Engineering Jobs,35000.0,2013-06-17 00:00:00,2013-09-15 00:00:00,thecareerengineer.com


In [98]:
indices = list(df[ss].index)
for ind in indices:
    # Get the search and selection word
    match = re.search(r'(\s[sS]earch\s(?:&|and)?\s?[sS]election)',df['Company'][ind]).groups()[0]
    # Remove the value
    df.loc[ind,'Company'] = df.loc[ind,'Company'].replace(match,'')
df[ss]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
2724,70770700,Cisco Network Analyst,Central London,Bishop,non-specified,contract,IT Jobs,90000,2013-03-02 15:00:00,2013-03-16 15:00:00,cv-library.co.uk
2889,69081556,Semi Skilled Building Maintenance Engineer,Dundee,PDA,non-specified,permanent,Engineering Jobs,22000,2013-09-20 12:00:00,2013-11-19 12:00:00,cv-library.co.uk
3180,70219038,Bms Support Engineer,Scotland,PDA,non-specified,permanent,Engineering Jobs,,2013-11-02 00:00:00,2014-01-31 00:00:00,cv-library.co.uk
4255,69012180,Scientific Director,London,ID,non-specified,permanent,Healthcare & Nursing Jobs,55000,2012-09-02 12:00:00,2012-12-01 12:00:00,cv-library.co.uk
4394,67903619,"Agile Java Developer, Online Gaming",Central London,Bishop,non-specified,permanent,IT Jobs,53000,2013-05-21 00:00:00,2013-07-20 00:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
49462,71375525,Qualified Accountant Immediate Start,Belfast,Bond Ltd,full_time,contract,Accounting & Finance Jobs,32500,2012-09-30 15:00:00,2012-11-29 15:00:00,nijobs.com
49498,67637107,Finance Business Analyst,Croydon,Greybridge Ltd,non-specified,permanent,Accounting & Finance Jobs,55000,2013-09-03 15:00:00,2013-10-03 15:00:00,onlineinsurancejobs.co.uk
49694,69133532,Senior Manager It Project & Program Risk Assu...,Central London,CGS,full_time,permanent,Accounting & Finance Jobs,90000,2013-05-16 15:00:00,2013-08-14 15:00:00,careersinaudit.com
50014,70574295,Hr Project Manager (6 Months Temporary With St...,North West London,PDA Ltd,full_time,contract,Engineering Jobs,35000.0,2013-06-17 00:00:00,2013-09-15 00:00:00,thecareerengineer.com


In [99]:
# Get companies containing & character with spaces
symbols = df['Company'].str.contains(r' & ')
df[symbols]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
7977,71774937,"Biztalk Specialist Global Corporation, Egham ...",Egham,Canary Wharf & City Ltd,non-specified,permanent,IT Jobs,,2013-12-09 15:00:00,2014-03-09 15:00:00,jobserve.com
8096,72372986,Qa Automated Tester,Central London,Hudson Banking & Finance,non-specified,permanent,IT Jobs,70000,2012-10-08 15:00:00,2012-11-07 15:00:00,jobserve.com
8324,71336370,"Test Analyst, System, Integration, Uat, Siebel...",London,Capita Assurance & Testing,non-specified,contract,IT Jobs,60000,2012-04-30 12:00:00,2012-05-30 12:00:00,jobserve.com
8666,70249406,Compliance Solution Specialist,London,J & C Associates Ltd,non-specified,permanent,IT Jobs,72500,2012-09-20 15:00:00,2012-12-19 15:00:00,jobserve.com
8971,71774803,"Data Centre Engineer Needed (Citrix, Xenapp, X...",London,McCabe & Barton,non-specified,permanent,IT Jobs,65000,2012-12-01 15:00:00,2013-01-30 15:00:00,jobserve.com
...,...,...,...,...,...,...,...,...,...,...,...
46975,71212592,Assistant Front Office Manager,Bristol,Independently Owned & Operated Holiday Inn,full_time,non-specified,Hospitality & Catering Jobs,15000.0,2012-01-02 12:00:00,2012-04-01 12:00:00,careerbuilder.com
47126,71700819,Full Time And Part Time Sales,Trowbridge,MR I MARR & MISS G HIVES,full_time,non-specified,Sales Jobs,20000.0,2013-06-25 12:00:00,2013-09-23 12:00:00,careerbuilder.com
48349,69133890,European Sales And Marketing Administrator – C...,Uk,Veenus Hotels & Experiences,non-specified,permanent,"PR, Advertising & Marketing Jobs",18750,2013-06-05 15:00:00,2013-09-03 15:00:00,Multilingualvacancies
49764,70578346,B**** Fire Safety Engineer,Uk,Health & Safety Laboratory,non-specified,non-specified,Teaching Jobs,34635,2012-01-23 12:00:00,2012-02-22 12:00:00,Jobs Ac


Let's check if these companies has similar names

In [100]:
indices = list(df[symbols].index)
for ind in indices:
    # Replace & with ' and ' and see if the company exists
    rpr_and = df['Company'][ind].replace(' & ',' and ')
    closest = find_closest_match(rpr_and,df['Company'].unique(),0.85)
    if len(closest) > 0 and closest[0] != df['Company'][ind]:
        print('Found match for ',df['Company'][ind]," as ",closest)

Found match for  Randstad Financial & Professional  as  ['Randstad Financial and Professional Ltd']
Found match for  Edwards & Pearce  as  ['Edwards and Pearce Ltd']
Found match for  Randstad Financial & Professional  as  ['Randstad Financial and Professional Ltd']
Found match for  Randstad Financial & Professional  as  ['Randstad Financial and Professional Ltd']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Randstad Financial & Professional  as  ['Randstad Financial and Professional Ltd']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Badenoch & Clark   as  ['Badenoch and Clark']
Found match for  Badenoch & Cl

As expected, most of them does. Thus, all those symbols would be replaced by 'and'

In [101]:
indices = list(df[symbols].index)
for ind in indices:
    company = df['Company'][ind].replace(' & ',' and ')
    closest = find_closest_match(company,df['Company'].unique(),0.85)
    if len(closest) > 0 and closest[0] != df['Company'][ind]:
        company = closest[0]
    # Reocrd to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',df['Company'][ind],company,'Contains & character','Replace & with and')
    # Replace the values with the closest match
    df.loc[ind,'Company'] = company
df[symbols].head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
7977,71774937,"Biztalk Specialist Global Corporation, Egham ...",Egham,Canary Wharf and City Ltd,non-specified,permanent,IT Jobs,,2013-12-09 15:00:00,2014-03-09 15:00:00,jobserve.com
8096,72372986,Qa Automated Tester,Central London,Hudson Banking and Finance,non-specified,permanent,IT Jobs,70000,2012-10-08 15:00:00,2012-11-07 15:00:00,jobserve.com
8324,71336370,"Test Analyst, System, Integration, Uat, Siebel...",London,Capita Assurance and Testing,non-specified,contract,IT Jobs,60000,2012-04-30 12:00:00,2012-05-30 12:00:00,jobserve.com
8666,70249406,Compliance Solution Specialist,London,J and C Associates Ltd,non-specified,permanent,IT Jobs,72500,2012-09-20 15:00:00,2012-12-19 15:00:00,jobserve.com
8971,71774803,"Data Centre Engineer Needed (Citrix, Xenapp, X...",London,McCabe and Barton,non-specified,permanent,IT Jobs,65000,2012-12-01 15:00:00,2013-01-30 15:00:00,jobserve.com


Let's take a last look at those with companies with '&'

In [102]:
# Get companies containing & but no spaces
amp = df['Company'].str.contains(r'&')
df[amp]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
10348,69209598,Field Service Engineer (Novellus Or Lam),Glasgow,P&C Recruitment Ltd,full_time,permanent,IT Jobs,33500,2012-12-09 12:00:00,2013-03-09 12:00:00,fish4.co.uk
11037,69593023,Assistant Manager For Premier Sports Shop,London,A&P Resources,non-specified,permanent,Sales Jobs,15000,2013-10-28 15:00:00,2013-12-27 15:00:00,totaljobs.com
12768,69800316,Retail Events Assistant Target Driven Individ...,Newcastle Upon Tyne,A&R Events,non-specified,permanent,Sales Jobs,15600,2012-02-04 12:00:00,2012-03-05 12:00:00,totaljobs.com
13205,68685159,B&Q Insurance And Business Continuity Advisor,Eastleigh,B&Q,non-specified,permanent,Accounting & Finance Jobs,52500,2012-08-04 12:00:00,2012-09-03 12:00:00,totaljobs.com
13990,71431181,Sales Negotiator Leading Estate Agency Golde...,London,A&P Resources,non-specified,permanent,Sales Jobs,14000,2012-08-04 12:00:00,2012-10-03 12:00:00,totaljobs.com
14371,68711497,Hardware Design Engineer Sussex ****K,Lewes,RE&M,non-specified,permanent,Engineering Jobs,37500,2013-12-21 15:00:00,2014-02-19 15:00:00,totaljobs.com
15578,68685326,B&Q Finance Business Partner People,Eastleigh,B&Q,non-specified,permanent,Accounting & Finance Jobs,42500,2012-10-20 15:00:00,2012-11-19 15:00:00,totaljobs.com
19228,72239048,Fpga Design Engineer Avon ****K,Shepton Mallet,RE&M,non-specified,permanent,Engineering Jobs,35000,2012-11-12 00:00:00,2013-01-11 00:00:00,totaljobs.com
20656,68360151,Aspnet Developer Mvc****,Surrey,H&R Talent,non-specified,contract,IT Jobs,81000,2013-05-22 15:00:00,2013-06-21 15:00:00,cwjobs.co.uk
22043,70781811,1St/2Nd Line It Support Analyst,Uk,H&R Talent,non-specified,contract,IT Jobs,18720,2013-02-24 15:00:00,2013-03-26 15:00:00,cwjobs.co.uk


From the output above, all remaining companies with '&' seems to be correct. Thus, they are kept as is

In [103]:
colon_company = df['Company'].str.contains(r':')
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+')
amp_company = df['Company'].str.contains(r'&')
df[~valid_company & ~non_specified & ~colon_company&~amp_company]['Company']

7904                               C.O.A.L IT Services Ltd
8025                               C.O.A.L IT Services Ltd
8044                            Digital Guru's Recruitment
8075                               C.O.A.L IT Services Ltd
8076                               C.O.A.L IT Services Ltd
                               ...                        
49798                           Queen's University Belfast
49804                                King's College London
50137    The Chartered Institution of Wastes Management...
50199            Guy's and St Thomas' NHS Foundation Trust
50460                           Sotheby's Institute of Art
Name: Company, Length: 380, dtype: object

Now, let's check the values with "'"s

In [104]:
apro_company = df['Company'].str.contains("\'")
df[apro_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8044,72161182,Senior Build Engineer Lead,Central London,Digital Guru's Recruitment,non-specified,permanent,IT Jobs,55000,2012-02-29 15:00:00,2012-03-30 15:00:00,jobserve.com
10169,70320456,Advanced Nurse Practitioner Primary & Physical...,Uk,St Andrew's Healthcare,non-specified,permanent,Healthcare & Nursing Jobs,35K,2013-12-28 15:00:00,2014-01-27 15:00:00,fish4.co.uk
12298,68061830,Mortgage & Protection Advisor,Oxford,O'Neil Consultancy,non-specified,permanent,Accounting & Finance Jobs,37000,2012-07-30 15:00:00,2012-08-29 15:00:00,totaljobs.com
12685,69168873,Client Insight Manager,Oxford,O'Neil Consultancy,non-specified,permanent,"PR, Advertising & Marketing Jobs",31500,2012-02-21 15:00:00,2012-05-21 15:00:00,totaljobs.com
12796,68061810,"Cemap Protection Consultant Rushden, Northamp...",Rushden,O'Neil Consultancy,non-specified,permanent,Accounting & Finance Jobs,37500,2012-07-04 15:00:00,2012-08-03 15:00:00,totaljobs.com
...,...,...,...,...,...,...,...,...,...,...,...
49727,71743454,Registry Office Assistant,London,Regent's College,non-specified,non-specified,Teaching Jobs,21000,2013-12-23 15:00:00,2014-01-06 15:00:00,Jobs Ac
49798,68587485,**** Funded Phd Studentships In Psychology,Northern Ireland,Queen's University Belfast,non-specified,non-specified,Teaching Jobs,13726,2013-01-30 12:00:00,2013-03-31 12:00:00,Jobs Ac
49804,69734998,Acting Deputy Director,London,King's College London,non-specified,non-specified,Teaching Jobs,51070,2012-06-24 00:00:00,2012-08-23 00:00:00,Jobs Ac
50199,69587192,Clinical Nurse Specialist Fetal Cardiology,London,Guy's and St Thomas' NHS Foundation Trust,non-specified,non-specified,Healthcare & Nursing Jobs,41463,2013-08-25 12:00:00,2013-09-24 12:00:00,rcnbulletinjobs.co.uk


From the output above, it seems like these companies are correct. Thus, they are all kept as is

In [105]:
colon_company = df['Company'].str.contains(r':')
valid_company = df['Company'].str.fullmatch(r'([a-zA-Z0-9]\s?)+')
amp_company = df['Company'].str.contains(r'&')
apro_company = df['Company'].str.contains("\'")
df[~valid_company & ~non_specified & ~colon_company&~amp_company&~apro_company]['Company']

7904                               C.O.A.L IT Services Ltd
8025                               C.O.A.L IT Services Ltd
8075                               C.O.A.L IT Services Ltd
8076                               C.O.A.L IT Services Ltd
8094                                      24/7 Recruitment
                               ...                        
49389                                           D.P. Group
49413                                           D.P. Group
49437                                           D.P. Group
49449                                           D.P. Group
50137    The Chartered Institution of Wastes Management...
Name: Company, Length: 307, dtype: object

The last few values contains "."s

In [106]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
7904,71774904,Data Administrator Knowledge/Exp Of Rims And ...,London,C.O.A.L IT Services Ltd,non-specified,contract,IT Jobs,35000,2012-04-13 15:00:00,2012-04-27 15:00:00,jobserve.com
8025,68627800,Murex Support Team Leader Leading Financial,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,90000,2012-01-04 00:00:00,2012-03-04 00:00:00,jobserve.com
8075,71336234,Java Developer/Spring/Hibernate/Github/Jsp/Vel...,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,55000,2013-07-18 00:00:00,2013-10-16 00:00:00,jobserve.com
8076,72161120,Java Developer Ecommerce Fx/Money Markets Dev...,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,65000,2012-01-06 15:00:00,2012-04-05 15:00:00,jobserve.com
8107,69788055,Security Analyst,Southampton International Airport,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,45000,2013-09-24 12:00:00,2013-11-23 12:00:00,jobserve.com
...,...,...,...,...,...,...,...,...,...,...,...
49302,68847270,**** X Service Maintenance Engineers,West London,Fasttrack Management Serv. Ltd,non-specified,permanent,Engineering Jobs,25000,2012-03-26 15:00:00,2012-05-25 15:00:00,londonjobs.co.uk
49389,68014633,Oracle Financials Developer,Belfast,D.P. Group,full_time,permanent,IT Jobs,40000,2013-03-28 12:00:00,2013-05-27 12:00:00,nijobs.com
49413,67385052,Java Developers Multiple Roles,Belfast,D.P. Group,full_time,permanent,IT Jobs,NaN,2013-01-19 15:00:00,2013-04-19 15:00:00,nijobs.com
49437,68014632,Oracle Technical Lead,Belfast,D.P. Group,full_time,permanent,IT Jobs,50000,2012-04-26 12:00:00,2012-07-25 12:00:00,nijobs.com


Let's check out some of the values

In [107]:
coal = df['Company'].str.contains(r'C\.?O\.?A\.?L')
df[coal]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
7904,71774904,Data Administrator Knowledge/Exp Of Rims And ...,London,C.O.A.L IT Services Ltd,non-specified,contract,IT Jobs,35000,2012-04-13 15:00:00,2012-04-27 15:00:00,jobserve.com
8025,68627800,Murex Support Team Leader Leading Financial,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,90000,2012-01-04 00:00:00,2012-03-04 00:00:00,jobserve.com
8075,71336234,Java Developer/Spring/Hibernate/Github/Jsp/Vel...,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,55000,2013-07-18 00:00:00,2013-10-16 00:00:00,jobserve.com
8076,72161120,Java Developer Ecommerce Fx/Money Markets Dev...,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,65000,2012-01-06 15:00:00,2012-04-05 15:00:00,jobserve.com
8107,69788055,Security Analyst,Southampton International Airport,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,45000,2013-09-24 12:00:00,2013-11-23 12:00:00,jobserve.com
8259,71335526,Java Programmer/Java Developer (Market Data An...,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,40000,2013-03-27 12:00:00,2013-06-25 12:00:00,jobserve.com
8265,72160560,Business Systems Analyst Uml,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,50000,2012-05-07 12:00:00,2012-07-06 12:00:00,jobserve.com
8280,70622221,Unix Technical Support Engineer Worldwide Sof...,Calcot,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,40000,2013-04-16 12:00:00,2013-07-15 12:00:00,jobserve.com
8378,68496467,Senior Ui Developer/Ruby/Html5/Css3/Javascript,London,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,57500,2012-09-15 15:00:00,2012-10-15 15:00:00,jobserve.com
8630,68628549,Linux/Windows Support Engineer Cambridge ***...,Little Chesterford,C.O.A.L IT Services Ltd,non-specified,permanent,IT Jobs,32500,2013-04-02 00:00:00,2013-06-01 00:00:00,jobserve.com


In [108]:
dp = df['Company'].str.contains(r'D\.?P\.')
df[dp]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
49389,68014633,Oracle Financials Developer,Belfast,D.P. Group,full_time,permanent,IT Jobs,40000,2013-03-28 12:00:00,2013-05-27 12:00:00,nijobs.com
49413,67385052,Java Developers Multiple Roles,Belfast,D.P. Group,full_time,permanent,IT Jobs,NaN,2013-01-19 15:00:00,2013-04-19 15:00:00,nijobs.com
49437,68014632,Oracle Technical Lead,Belfast,D.P. Group,full_time,permanent,IT Jobs,50000,2012-04-26 12:00:00,2012-07-25 12:00:00,nijobs.com
49449,68014930,Technology Consultant,Belfast,D.P. Group,full_time,permanent,IT Jobs,52500,2013-05-11 15:00:00,2013-07-10 15:00:00,nijobs.com


These companies seems to be legitimate. However, there is a 'Serv.' value there.

In [109]:
serv = df['Company'].str.contains(r'[sS]erv?\.')
df[serv]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
49246,68847157,Installation Service Engineer,North London,Fasttrack Management Serv. Ltd,non-specified,permanent,Engineering Jobs,28000,2013-03-18 12:00:00,2013-06-16 12:00:00,londonjobs.co.uk
49302,68847270,**** X Service Maintenance Engineers,West London,Fasttrack Management Serv. Ltd,non-specified,permanent,Engineering Jobs,25000,2012-03-26 15:00:00,2012-05-25 15:00:00,londonjobs.co.uk


Serv seems to be a short form for Services

In [110]:
services = df['Company'].str.contains(r'[sS]ervices')
df[services]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
78,68544566,Site Supervisor (M Amp;E),Kent,Calco Services Ltd,non-specified,permanent,Engineering Jobs,32000,2012-09-02 00:00:00,2012-11-01 00:00:00,rengineeringjobs.com
246,71196065,Intermediate Estimator (Rc Frame And Groundworks),North London,Calco Services Ltd,non-specified,permanent,Engineering Jobs,20000,2013-05-21 12:00:00,2013-08-19 12:00:00,rengineeringjobs.com
281,68805113,Estimator (Fit Out Interior),London,Calco Services Ltd,non-specified,permanent,Engineering Jobs,50000,2013-11-22 00:00:00,2014-01-21 00:00:00,rengineeringjobs.com
317,67286147,Project Manager (Pqs Background),London,Calco Services Ltd,non-specified,permanent,Engineering Jobs,60000,2012-10-13 15:00:00,2012-12-12 15:00:00,rengineeringjobs.com
344,71196096,Project Manager (Glass Partitioning Projects),Buckinghamshire,Calco Services Ltd,non-specified,permanent,Engineering Jobs,50000,2013-04-12 15:00:00,2013-05-12 15:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50602,70086304,Senior Financial Planner (Private Banking),London,Hanover Search Financial Services,non-specified,permanent,Accounting & Finance Jobs,70000,2012-03-13 12:00:00,2012-04-12 12:00:00,ifaonlinejobs.co.uk
50613,68511180,Senior Professional Practice Financial Planner,Berkshire,Hanover Search Financial Services,non-specified,permanent,Accounting & Finance Jobs,37500,2013-01-31 00:00:00,2013-04-01 00:00:00,ifaonlinejobs.co.uk
50615,68511172,Junior Financial Planner,Central London,Hanover Search Financial Services,non-specified,permanent,Accounting & Finance Jobs,50000,2013-11-05 12:00:00,2013-12-05 12:00:00,ifaonlinejobs.co.uk
50621,72169211,"E, C Amp; I Maintenance Technician C****K",Marton-In-Cleveland,Support Services Group,full_time,permanent,Accounting & Finance Jobs,27250,2013-08-22 00:00:00,2013-11-20 00:00:00,jobsfinancial.com


Thus, these 2 values would be converted to Services

In [111]:
indices = list(df[serv].index)
for ind in indices:
    company = df['Company'][ind].replace('Serv.','Services')
    # Add to error Dict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',df['Company'][ind],company,'Short form of a word exists','Convert to actual form')
    # Replace the values with Serv. changed to Services
    df.loc[ind,'Company'] = company
df[serv]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
49246,68847157,Installation Service Engineer,North London,Fasttrack Management Services Ltd,non-specified,permanent,Engineering Jobs,28000,2013-03-18 12:00:00,2013-06-16 12:00:00,londonjobs.co.uk
49302,68847270,**** X Service Maintenance Engineers,West London,Fasttrack Management Services Ltd,non-specified,permanent,Engineering Jobs,25000,2012-03-26 15:00:00,2012-05-25 15:00:00,londonjobs.co.uk


Continue our checking for "."

In [112]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
8271,71774481,"Database Technical Analyst (Ms Access, Sql, Or...",Solihull,Next Move I.T. Ltd,non-specified,permanent,IT Jobs,30000,2013-08-22 00:00:00,2013-11-20 00:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com
9160,68628028,Boxi/Informatica Team Lead London City To ****K,Central London,Deerfoot I.T. Resources Ltd,non-specified,permanent,IT Jobs,75000,2012-10-03 12:00:00,2013-01-01 12:00:00,jobserve.com
...,...,...,...,...,...,...,...,...,...,...,...
43315,72627084,Relationship / Sales Executive Multilingual,London,C.K.R. Recruitment Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",30000,2013-10-03 00:00:00,2014-01-01 00:00:00,cityjobs.com
43328,72394874,Risk Manager Consumer Credit,West Sussex,C.K.R. Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,57500,2013-08-24 12:00:00,2013-11-22 12:00:00,cityjobs.com
43346,68063261,Compliance Monitoring Manager Banking,The City,C.K.R. Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,75000,2012-08-29 00:00:00,2012-11-27 00:00:00,cityjobs.com
43386,71356473,Senior Operational Risk Analyst (Avp),London,C.K.R. Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,45000,2012-08-24 12:00:00,2012-10-23 12:00:00,cityjobs.com


In [113]:
ckr = df['Company'].str.contains(r'C\.?K\.?R\.?')
df[ckr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
43239,72194600,"Senior Manager, Operational Risk (612 Month Co...",The City,C.K.R. Recruitment Ltd,non-specified,contract,Accounting & Finance Jobs,56000,2012-03-17 15:00:00,2012-04-16 15:00:00,cityjobs.com
43315,72627084,Relationship / Sales Executive Multilingual,London,C.K.R. Recruitment Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",30000,2013-10-03 00:00:00,2014-01-01 00:00:00,cityjobs.com
43328,72394874,Risk Manager Consumer Credit,West Sussex,C.K.R. Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,57500,2013-08-24 12:00:00,2013-11-22 12:00:00,cityjobs.com
43346,68063261,Compliance Monitoring Manager Banking,The City,C.K.R. Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,75000,2012-08-29 00:00:00,2012-11-27 00:00:00,cityjobs.com
43386,71356473,Senior Operational Risk Analyst (Avp),London,C.K.R. Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,45000,2012-08-24 12:00:00,2012-10-23 12:00:00,cityjobs.com
43461,69122468,Client Reporting Analyst,The City,C.K.R. Recruitment Ltd,non-specified,permanent,Accounting & Finance Jobs,42500,2012-05-14 15:00:00,2012-07-13 15:00:00,cityjobs.com


The CKR company seems to be valid as well

In [114]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp&~ckr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
8271,71774481,"Database Technical Analyst (Ms Access, Sql, Or...",Solihull,Next Move I.T. Ltd,non-specified,permanent,IT Jobs,30000,2013-08-22 00:00:00,2013-11-20 00:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com
9160,68628028,Boxi/Informatica Team Lead London City To ****K,Central London,Deerfoot I.T. Resources Ltd,non-specified,permanent,IT Jobs,75000,2012-10-03 12:00:00,2013-01-01 12:00:00,jobserve.com
...,...,...,...,...,...,...,...,...,...,...,...
40872,68446046,Baby Room Practitioner Col****,Colchester,www.essex eyjobs,full_time,non-specified,Teaching Jobs,13440,2012-05-03 00:00:00,2012-08-01 00:00:00,Jobcentre Plus
40928,67447764,National Training & Development Manager,Milton Keynes,B.S.H. Home Applicances Ltd,full_time,non-specified,Teaching Jobs,30000,2012-12-12 00:00:00,2013-02-10 00:00:00,Jobcentre Plus
40929,66750270,Home Support Assistants,North Shields,At Home with...Helen McArdle Care,full_time,non-specified,Healthcare & Nursing Jobs,14976,2013-08-22 15:00:00,2013-09-21 15:00:00,Jobcentre Plus
40986,69150243,Gas Technical Engineer (Breakdown And Repair) ...,Birmingham,E.on UK Plc,full_time,non-specified,Engineering Jobs,34760,2013-07-16 00:00:00,2013-08-15 00:00:00,Jobcentre Plus


Let's check for title that contains the word 'I.T.'

In [115]:
# Get companies containing the word IT, with or without .s
it = df['Company'].str.contains(r'\b[Ii]\.?[Tt]\.?\b')
df[it]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
858,70607960,"Overhead Electrical Lineworkervictoria, Austra...",Uk,Ccity IT Recruitment,non-specified,permanent,Engineering Jobs,57500,2013-01-08 00:00:00,2013-03-09 00:00:00,cv-library.co.uk
895,68356718,Data Security Analyst,Cambridgeshire,Pytec IT Recruitment,non-specified,permanent,IT Jobs,35000,2013-11-25 00:00:00,2014-01-24 00:00:00,cv-library.co.uk
948,69541291,Junior Service Desk Analyst,Cambridge,Deerfoot IT Resources Ltd,non-specified,permanent,IT Jobs,23000,2013-09-10 12:00:00,2013-10-10 12:00:00,cv-library.co.uk
951,72546752,Php Developer / Lamp Developer Web And Cloud ...,Brighton,Arc IT Recruitment,non-specified,permanent,IT Jobs,30000,2013-09-03 15:00:00,2013-11-02 15:00:00,cv-library.co.uk
1421,69083921,Project Manager Automation / Dudley / **** **...,London,Concept IT,non-specified,contract,Engineering Jobs,52800,2013-03-30 12:00:00,2013-04-13 12:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50690,72579026,French Speaking Regional Sales Manager Uk Base,South East London,IT Executive Recruitment,full_time,permanent,IT Jobs,60011,2013-04-19 00:00:00,2013-05-19 00:00:00,TotallyExec
50717,68538223,Senior Configuration Manager ****,London,Rullion IT Plus,non-specified,permanent,Accounting & Finance Jobs,60000.0,2013-03-27 15:00:00,2013-06-25 15:00:00,frontofficebanking.com
50719,69181411,Senior Business Analystdebt Financereading****K,Uk,Rullion IT Plus,non-specified,permanent,Accounting & Finance Jobs,65000.0,2013-06-08 15:00:00,2013-07-08 15:00:00,frontofficebanking.com
50728,68538232,Perl Developer Portsmouth ****K,Uk,Rullion IT Plus,non-specified,permanent,Accounting & Finance Jobs,40000.0,2013-04-15 12:00:00,2013-06-14 12:00:00,frontofficebanking.com


It seems like there are multiple variations of IT. All of these would be reformatted to a universal form 'IT'

In [116]:
it_company = df['Company'].str.contains(r'\b[Ii]\.?[Tt]\.?\b')
indices = list(df[it_company].index)
for ind in indices:
    it_wrd = re.search(r'\s*([Ii]\.*[Tt]\.*)\s*',df['Company'][ind]).groups()[0]
    old = df['Company'][ind]
    # Replace word with IT
    df.loc[ind,'Company'] = df.loc[ind,'Company'].replace(it_wrd,'IT')
    # Add to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',old,df['Company'][ind],'Multiple variations of a word','Set to a uniform word')
df[it_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
858,70607960,"Overhead Electrical Lineworkervictoria, Austra...",Uk,CcITy IT RecruITment,non-specified,permanent,Engineering Jobs,57500,2013-01-08 00:00:00,2013-03-09 00:00:00,cv-library.co.uk
895,68356718,Data Security Analyst,Cambridgeshire,Pytec IT Recruitment,non-specified,permanent,IT Jobs,35000,2013-11-25 00:00:00,2014-01-24 00:00:00,cv-library.co.uk
948,69541291,Junior Service Desk Analyst,Cambridge,Deerfoot IT Resources Ltd,non-specified,permanent,IT Jobs,23000,2013-09-10 12:00:00,2013-10-10 12:00:00,cv-library.co.uk
951,72546752,Php Developer / Lamp Developer Web And Cloud ...,Brighton,Arc IT Recruitment,non-specified,permanent,IT Jobs,30000,2013-09-03 15:00:00,2013-11-02 15:00:00,cv-library.co.uk
1421,69083921,Project Manager Automation / Dudley / **** **...,London,Concept IT,non-specified,contract,Engineering Jobs,52800,2013-03-30 12:00:00,2013-04-13 12:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50690,72579026,French Speaking Regional Sales Manager Uk Base,South East London,IT Executive Recruitment,full_time,permanent,IT Jobs,60011,2013-04-19 00:00:00,2013-05-19 00:00:00,TotallyExec
50717,68538223,Senior Configuration Manager ****,London,Rullion IT Plus,non-specified,permanent,Accounting & Finance Jobs,60000.0,2013-03-27 15:00:00,2013-06-25 15:00:00,frontofficebanking.com
50719,69181411,Senior Business Analystdebt Financereading****K,Uk,Rullion IT Plus,non-specified,permanent,Accounting & Finance Jobs,65000.0,2013-06-08 15:00:00,2013-07-08 15:00:00,frontofficebanking.com
50728,68538232,Perl Developer Portsmouth ****K,Uk,Rullion IT Plus,non-specified,permanent,Accounting & Finance Jobs,40000.0,2013-04-15 12:00:00,2013-06-14 12:00:00,frontofficebanking.com


Now that's done, we continue with our searching

In [117]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp&~ckr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com
13231,68693588,Public Relations Officer,Uk,St.Marys International Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",26000,2013-10-06 00:00:00,2013-11-05 00:00:00,totaljobs.com
15956,72233005,Senior Mechanical Estimator Building Services,Northamptonshire,T.E.D. Ltd,non-specified,permanent,Engineering Jobs,47500,2013-07-02 00:00:00,2013-08-01 00:00:00,totaljobs.com
23403,68837702,Bar And Lounge Manager Br,Uk,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2012-08-07 00:00:00,2012-10-06 00:00:00,caterer.com
23612,69470872,Group Reservations Executive Up To ****K,South East London,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,17000,2013-03-27 12:00:00,2013-04-26 12:00:00,caterer.com
23812,71959934,Meeting & Events Sales Manager Reactive Exeter,Exeter,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2013-02-13 12:00:00,2013-02-27 12:00:00,caterer.com
23862,71276961,Sous Chef Plus Service Charge,South East London,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2012-04-25 15:00:00,2012-05-09 15:00:00,caterer.com
27602,66934491,Locum Physiotherapist Required Central London...,London,D.R.C. Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,48000,2013-04-21 12:00:00,2013-06-20 12:00:00,staffnurse.com


In [118]:
drc = df['Company'].str.contains(r'[dD]\.?[rR]\.?[cC]\.?')
df[drc]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
4424,69933586,Registered General Nurse (Rgn Bank),Uk,DRC,non-specified,contract,Healthcare & Nursing Jobs,,2013-02-19 00:00:00,2013-03-21 00:00:00,cv-library.co.uk
5904,69933552,Physiotherapist Band 6 Inpatients,Essex,DRC,non-specified,permanent,Healthcare & Nursing Jobs,28000,2013-03-16 12:00:00,2013-06-14 12:00:00,cv-library.co.uk
6327,69933547,Registered Nurse (Rmn/Mental Health),Oxford,DRC,non-specified,permanent,Healthcare & Nursing Jobs,26052,2013-03-31 12:00:00,2013-06-29 12:00:00,cv-library.co.uk
27273,66935105,Rgn (Bank Shifts) Preston,Preston,DRC Locums,non-specified,contract,Healthcare & Nursing Jobs,40320,2012-03-15 15:00:00,2012-05-14 15:00:00,staffnurse.com
27602,66934491,Locum Physiotherapist Required Central London...,London,D.R.C. Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,48000,2013-04-21 12:00:00,2013-06-20 12:00:00,staffnurse.com
27870,71607588,Band 6 Speech And Language Therapists London,London,D.R.C. Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,47040,2013-10-28 12:00:00,2013-11-27 12:00:00,staffnurse.com
28350,71614468,Surgery Nurse Norwich,Norwich,D.R.C. Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,0,2012-03-11 12:00:00,2012-06-09 12:00:00,staffnurse.com
29112,70016274,Mental Health Support Worker Barnsley,Barnsley,DRC Locums,full_time,non-specified,Healthcare & Nursing Jobs,13500,2012-09-03 12:00:00,2012-10-03 12:00:00,staffnurse.com
29135,72187316,Heath Care Assistant Immediate Start Liverpool,Liverpool,D.R.C. Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,14400,2012-09-28 00:00:00,2012-11-27 00:00:00,staffnurse.com
29285,71093048,Locum Occupational Therapist Required Gloucester,Gloucester,D.R.C. Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,47040,2013-05-25 12:00:00,2013-08-23 12:00:00,staffnurse.com


It seems like this 'DRC' company needs to have its values tweaked

In [119]:
indices = list(df[drc].index)
for ind in indices:
    # Add to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',df.loc[ind,'Company'],'DRC Locums Ltd','Multiple namings','Set to a global name')
    # Set to a global name
    df.loc[ind,'Company'] = 'DRC Locums Ltd'
df[drc]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
4424,69933586,Registered General Nurse (Rgn Bank),Uk,DRC Locums Ltd,non-specified,contract,Healthcare & Nursing Jobs,,2013-02-19 00:00:00,2013-03-21 00:00:00,cv-library.co.uk
5904,69933552,Physiotherapist Band 6 Inpatients,Essex,DRC Locums Ltd,non-specified,permanent,Healthcare & Nursing Jobs,28000,2013-03-16 12:00:00,2013-06-14 12:00:00,cv-library.co.uk
6327,69933547,Registered Nurse (Rmn/Mental Health),Oxford,DRC Locums Ltd,non-specified,permanent,Healthcare & Nursing Jobs,26052,2013-03-31 12:00:00,2013-06-29 12:00:00,cv-library.co.uk
27273,66935105,Rgn (Bank Shifts) Preston,Preston,DRC Locums Ltd,non-specified,contract,Healthcare & Nursing Jobs,40320,2012-03-15 15:00:00,2012-05-14 15:00:00,staffnurse.com
27602,66934491,Locum Physiotherapist Required Central London...,London,DRC Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,48000,2013-04-21 12:00:00,2013-06-20 12:00:00,staffnurse.com
27870,71607588,Band 6 Speech And Language Therapists London,London,DRC Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,47040,2013-10-28 12:00:00,2013-11-27 12:00:00,staffnurse.com
28350,71614468,Surgery Nurse Norwich,Norwich,DRC Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,0,2012-03-11 12:00:00,2012-06-09 12:00:00,staffnurse.com
29112,70016274,Mental Health Support Worker Barnsley,Barnsley,DRC Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,13500,2012-09-03 12:00:00,2012-10-03 12:00:00,staffnurse.com
29135,72187316,Heath Care Assistant Immediate Start Liverpool,Liverpool,DRC Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,14400,2012-09-28 00:00:00,2012-11-27 00:00:00,staffnurse.com
29285,71093048,Locum Occupational Therapist Required Gloucester,Gloucester,DRC Locums Ltd,full_time,non-specified,Healthcare & Nursing Jobs,47040,2013-05-25 12:00:00,2013-08-23 12:00:00,staffnurse.com


In [120]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp&~ckr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com
13231,68693588,Public Relations Officer,Uk,St.Marys International Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",26000,2013-10-06 00:00:00,2013-11-05 00:00:00,totaljobs.com
15956,72233005,Senior Mechanical Estimator Building Services,Northamptonshire,T.E.D. Ltd,non-specified,permanent,Engineering Jobs,47500,2013-07-02 00:00:00,2013-08-01 00:00:00,totaljobs.com
23403,68837702,Bar And Lounge Manager Br,Uk,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2012-08-07 00:00:00,2012-10-06 00:00:00,caterer.com
23612,69470872,Group Reservations Executive Up To ****K,South East London,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,17000,2013-03-27 12:00:00,2013-04-26 12:00:00,caterer.com
23812,71959934,Meeting & Events Sales Manager Reactive Exeter,Exeter,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2013-02-13 12:00:00,2013-02-27 12:00:00,caterer.com
23862,71276961,Sous Chef Plus Service Charge,South East London,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2012-04-25 15:00:00,2012-05-09 15:00:00,caterer.com
29919,69688264,Urgently Requiredtrainee Teachers,London,Schools Courses Career Development C.I.C,non-specified,permanent,Teaching Jobs,21000,2013-01-17 00:00:00,2013-03-18 00:00:00,jobsite.co.uk


In [121]:
# Get all companies with the short form with or without the .
short_form = df['Company'].str.contains(r'[nN]\.?[rR]\.?[lL]\.?|[cC]\.?[oO]\.?[sS]\.?|[kK]\.?[Bb]\.?[cC]\.?|[bB]\.?[sS]\.?[hH]\.?|[cC]\.?[iI]\.?[cC]\.?')
df[short_form]                                        

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
669,67208419,Mechanical Commissioning Engineer(S),Cumbria,NRL Ltd,non-specified,contract,Engineering Jobs,57600,2013-01-22 12:00:00,2013-02-05 12:00:00,cv-library.co.uk
2602,69541463,Applictaion Support Analyst,Leicestershire,NRL Ltd,non-specified,permanent,IT Jobs,30000,2012-02-15 15:00:00,2012-03-16 15:00:00,cv-library.co.uk
5033,70172134,Cluster Hr Manager,Gatwick,kbc Associates,non-specified,permanent,Hospitality & Catering Jobs,,2012-11-07 00:00:00,2013-02-05 00:00:00,cv-library.co.uk
10730,69803991,Brand Marketing Executivecosmo Restaurant Grou...,Uk,Cosmo Service Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",26500,2013-04-13 00:00:00,2013-07-12 00:00:00,totaljobs.com
14545,72447697,Strategic Marketing And Planning Manager,South East London,Decostyle Recruitment,non-specified,permanent,"PR, Advertising & Marketing Jobs",62500,2012-04-24 00:00:00,2012-05-24 00:00:00,totaljobs.com
15170,71432270,Senior Marketing Planner Campaign,Northampton,Avon Cosmetics,non-specified,permanent,"PR, Advertising & Marketing Jobs",34000,2012-05-19 12:00:00,2012-06-18 12:00:00,totaljobs.com
23139,71956446,Store Manager Chichester Costa,Chichester,Costa,non-specified,non-specified,Hospitality & Catering Jobs,19500,2013-07-15 15:00:00,2013-09-13 15:00:00,caterer.com
23267,71276615,Assistant Manager Exeter New Opening,Exeter,Cosy Club,non-specified,non-specified,Hospitality & Catering Jobs,21000,2012-10-30 00:00:00,2012-12-29 00:00:00,caterer.com
23403,68837702,Bar And Lounge Manager Br,Uk,K.B.C. Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2012-08-07 00:00:00,2012-10-06 00:00:00,caterer.com
23437,71276619,Sous Chef – Exeter New Opening,Exeter,Cosy Club,non-specified,non-specified,Hospitality & Catering Jobs,0,2013-11-06 12:00:00,2014-01-05 12:00:00,caterer.com


It does look like some of them has it without the dots. Thus, they will all be converted into ones without

In [122]:
pattern =r'([nN]\.[rR]\.[lL]\.?)|([cC]\.[oO]\.[sS]\.?)|([kK]\.[Bb]\.[cC]\.?)|([bB]\.[sS]\.[hH]\.?)|([cC]\.[iI]\.[cC]\.?)|([tT]\.[eE]\.[dD]\.?)'
dot_form = df['Company'].str.contains(pattern)
indices = list(df[dot_form].index)

for ind in indices:
    old = df['Company'][ind]
    # Remove all dots
    new = old.replace('.','')
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',old,new,'Word contains .','Remove .s')
    # Replace the values in the dataset
    df.loc[ind,'Company'] = new
df[dot_form]

C:\Users\User\AppData\Local\Temp\ipykernel_2328\2149376107.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dot_form = df['Company'].str.contains(pattern)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
15956,72233005,Senior Mechanical Estimator Building Services,Northamptonshire,TED Ltd,non-specified,permanent,Engineering Jobs,47500,2013-07-02 00:00:00,2013-08-01 00:00:00,totaljobs.com
23403,68837702,Bar And Lounge Manager Br,Uk,KBC Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2012-08-07 00:00:00,2012-10-06 00:00:00,caterer.com
23612,69470872,Group Reservations Executive Up To ****K,South East London,KBC Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,17000,2013-03-27 12:00:00,2013-04-26 12:00:00,caterer.com
23812,71959934,Meeting & Events Sales Manager Reactive Exeter,Exeter,KBC Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2013-02-13 12:00:00,2013-02-27 12:00:00,caterer.com
23862,71276961,Sous Chef Plus Service Charge,South East London,KBC Associates Ltd,non-specified,non-specified,Hospitality & Catering Jobs,22000,2012-04-25 15:00:00,2012-05-09 15:00:00,caterer.com
29919,69688264,Urgently Requiredtrainee Teachers,London,Schools Courses Career Development CIC,non-specified,permanent,Teaching Jobs,21000,2013-01-17 00:00:00,2013-03-18 00:00:00,jobsite.co.uk
29998,72479693,Safety Advisor,Cumbria,NRL Ltd,non-specified,contract,Engineering Jobs,57600,2013-09-07 00:00:00,2013-10-07 00:00:00,jobsite.co.uk
30421,69688196,Trainee Teaching Assistants Needed,London,Schools Courses Career Development CIC,non-specified,permanent,Teaching Jobs,18250,2012-04-10 15:00:00,2012-07-09 15:00:00,jobsite.co.uk
30682,69688041,Sccd Need Teachers And Teaching Assistants,London,Schools Courses Career Development CIC,non-specified,permanent,Teaching Jobs,19500,2012-08-23 15:00:00,2012-11-21 15:00:00,jobsite.co.uk
31860,71685846,Project Manager Events / Marketing Macclesfi...,Uk,COS Recruitment Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",30000,2013-09-29 15:00:00,2013-12-28 15:00:00,jobsite.co.uk


In [123]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp&~ckr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com
13231,68693588,Public Relations Officer,Uk,St.Marys International Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",26000,2013-10-06 00:00:00,2013-11-05 00:00:00,totaljobs.com
31159,67948117,Process Metallurgist,Uk,www.dexternicholas,non-specified,permanent,Engineering Jobs,NaN,2012-12-29 15:00:00,2013-01-28 15:00:00,jobsite.co.uk
34379,69675549,Hr Manager (Parttime) 3 Days A Week ( Flexi Wo...,London,M. ARIA RECRUITMENT Ltd,part_time,permanent,Teaching Jobs,13522,2013-09-06 12:00:00,2013-11-05 12:00:00,jobs.guardian.co.uk
34780,71885279,Solicitor/ Disciplinary Case Manager,London,M. ARIA RECRUITMENT Ltd,full_time,permanent,Teaching Jobs,50000,2013-02-18 00:00:00,2013-05-19 00:00:00,jobs.guardian.co.uk
35142,68838621,Software Developer Devon,Devon,Recruit.UK,full_time,permanent,IT Jobs,27500,2013-05-07 15:00:00,2013-07-06 15:00:00,planetrecruit.com
37242,69188513,"No Experience, No Problem New Roles In Bristol",Uk,HelpMeGo.To Ltd,non-specified,non-specified,Sales Jobs,15600,2012-05-09 00:00:00,2012-07-08 00:00:00,salestarget.co.uk
39249,66537257,Kitchen Staff,Uk,J.D.Wetherspoons,part_time,non-specified,Hospitality & Catering Jobs,12480,2013-09-15 15:00:00,2013-09-29 15:00:00,Jobcentre Plus


It seems like companies such as 'St.Marys..', 'HelpMeGo..', 'St. Paul's..' are correct.

In [124]:
w_dots_company = df['Company'].str.contains(r'\b([eE]\.?[oO]\.?[nN]|[mM]\.?\s?[aA]\.?[rR]\.?[iI]\.?[aA]\.?|[jJ]\.?[dD]\.?|[lL]\.?[A]\.?)\b')
df[w_dots_company] 

C:\Users\User\AppData\Local\Temp\ipykernel_2328\2207887085.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  w_dots_company = df['Company'].str.contains(r'\b([eE]\.?[oO]\.?[nN]|[mM]\.?\s?[aA]\.?[rR]\.?[iI]\.?[aA]\.?|[jJ]\.?[dD]\.?|[lL]\.?[A]\.?)\b')


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
9289,72578110,Senior Oracle Dba/Database Administratornorth ...,Yorkshire And Humberside,LA International Computer Consultants Ltd,non-specified,permanent,IT Jobs,42500,2013-01-02 15:00:00,2013-02-01 15:00:00,jobserve.com
21122,70783291,Service Delivery Manager Reigate,Reigate,LA International,non-specified,permanent,IT Jobs,46000,2013-12-30 15:00:00,2014-01-29 15:00:00,cwjobs.co.uk
21313,72128277,Evaluation Analyst,London,LA International,non-specified,contract,IT Jobs,86400,2012-11-16 12:00:00,2012-11-30 12:00:00,cwjobs.co.uk
21498,71565882,Network Architect Ccie The Netherlands,Uk,LA International,non-specified,contract,IT Jobs,91326,2013-10-25 15:00:00,2013-11-24 15:00:00,cwjobs.co.uk
21907,72128112,Senior Windows Server Engineernorth Yorkshire,North Yorkshire,LA International,non-specified,permanent,IT Jobs,31000,2012-08-19 12:00:00,2012-11-17 12:00:00,cwjobs.co.uk
22473,72126374,Project Managers / Senior Project Managers Lo...,London,LA International,non-specified,permanent,IT Jobs,57500,2012-07-29 15:00:00,2012-10-27 15:00:00,cwjobs.co.uk
30531,72688600,Performance Test Analyst,London,LA International Computer Consultants Ltd,non-specified,contract,IT Jobs,75000,2013-08-21 15:00:00,2013-10-20 15:00:00,jobsite.co.uk
31369,72248327,"Document Controller, Systems Controller, Share...",Newbury,LA International Computer Consultants Ltd,non-specified,contract,IT Jobs,54000,2013-03-17 15:00:00,2013-03-31 15:00:00,jobsite.co.uk
32821,68806012,"Solutions Architect Surrey **** , **** ****...",Uk,LA International Computer Consultants Ltd,non-specified,permanent,IT Jobs,55000,2013-10-04 00:00:00,2013-11-03 00:00:00,jobsite.co.uk
32837,71856595,Network / Infrastructure Security Analystnorth...,North Yorkshire,LA International Computer Consultants Ltd,non-specified,permanent,IT Jobs,38500,2012-02-07 12:00:00,2012-05-07 12:00:00,jobsite.co.uk


Looking at the values above, it seems like copanies with the above short forms are all unique. Thus, they are all kept as is

In [125]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp&~ckr&~w_dots_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com
13231,68693588,Public Relations Officer,Uk,St.Marys International Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",26000,2013-10-06 00:00:00,2013-11-05 00:00:00,totaljobs.com
31159,67948117,Process Metallurgist,Uk,www.dexternicholas,non-specified,permanent,Engineering Jobs,NaN,2012-12-29 15:00:00,2013-01-28 15:00:00,jobsite.co.uk
35142,68838621,Software Developer Devon,Devon,Recruit.UK,full_time,permanent,IT Jobs,27500,2013-05-07 15:00:00,2013-07-06 15:00:00,planetrecruit.com
37242,69188513,"No Experience, No Problem New Roles In Bristol",Uk,HelpMeGo.To Ltd,non-specified,non-specified,Sales Jobs,15600,2012-05-09 00:00:00,2012-07-08 00:00:00,salestarget.co.uk
39773,69769276,Sen Key Worker,Colchester,www.essex eyjobs,part_time,non-specified,Teaching Jobs,14016,2012-10-08 15:00:00,2012-10-22 15:00:00,Jobcentre Plus
39866,68239749,Nursery Practitioner L**** Ooc****,Sea,www.essex eyjobs,full_time,non-specified,Teaching Jobs,12480,2012-12-02 12:00:00,2013-03-02 12:00:00,Jobcentre Plus
39974,69554894,After School Club Worker,Chelmsford,www.essex eyjobs,part_time,non-specified,Teaching Jobs,12480,2012-07-06 00:00:00,2012-08-05 00:00:00,Jobcentre Plus


Continuing on our search, it seems like some of the companies start with www.. Since .com and .net is removed, these would be removed as well

In [126]:
www_company = df['Company'].str.contains(r'^[wW]{3}\.?')
df[www_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
31159,67948117,Process Metallurgist,Uk,www.dexternicholas,non-specified,permanent,Engineering Jobs,NaN,2012-12-29 15:00:00,2013-01-28 15:00:00,jobsite.co.uk
39773,69769276,Sen Key Worker,Colchester,www.essex eyjobs,part_time,non-specified,Teaching Jobs,14016,2012-10-08 15:00:00,2012-10-22 15:00:00,Jobcentre Plus
39866,68239749,Nursery Practitioner L**** Ooc****,Sea,www.essex eyjobs,full_time,non-specified,Teaching Jobs,12480,2012-12-02 12:00:00,2013-03-02 12:00:00,Jobcentre Plus
39974,69554894,After School Club Worker,Chelmsford,www.essex eyjobs,part_time,non-specified,Teaching Jobs,12480,2012-07-06 00:00:00,2012-08-05 00:00:00,Jobcentre Plus
40127,69769301,Preschool Practitioner,Colchester,www.essex eyjobs,part_time,non-specified,Teaching Jobs,14400,2013-08-28 00:00:00,2013-09-27 00:00:00,Jobcentre Plus
40872,68446046,Baby Room Practitioner Col****,Colchester,www.essex eyjobs,full_time,non-specified,Teaching Jobs,13440,2012-05-03 00:00:00,2012-08-01 00:00:00,Jobcentre Plus


In [127]:
indices = list(df[www_company].index)
for ind in indices:
    old = df['Company'][ind]
    # Get the www.
    www = re.search(r'^([wW]{3}\.?)',old).groups()[0]
    # Replace with none
    new = old.replace(www,'')
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',old,new,'Starts with www','Remove the www')
    # Replace old value with the new value
    df.loc[ind,'Company'] = new
df[www_company]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
31159,67948117,Process Metallurgist,Uk,dexternicholas,non-specified,permanent,Engineering Jobs,NaN,2012-12-29 15:00:00,2013-01-28 15:00:00,jobsite.co.uk
39773,69769276,Sen Key Worker,Colchester,essex eyjobs,part_time,non-specified,Teaching Jobs,14016,2012-10-08 15:00:00,2012-10-22 15:00:00,Jobcentre Plus
39866,68239749,Nursery Practitioner L**** Ooc****,Sea,essex eyjobs,full_time,non-specified,Teaching Jobs,12480,2012-12-02 12:00:00,2013-03-02 12:00:00,Jobcentre Plus
39974,69554894,After School Club Worker,Chelmsford,essex eyjobs,part_time,non-specified,Teaching Jobs,12480,2012-07-06 00:00:00,2012-08-05 00:00:00,Jobcentre Plus
40127,69769301,Preschool Practitioner,Colchester,essex eyjobs,part_time,non-specified,Teaching Jobs,14400,2013-08-28 00:00:00,2013-09-27 00:00:00,Jobcentre Plus
40872,68446046,Baby Room Practitioner Col****,Colchester,essex eyjobs,full_time,non-specified,Teaching Jobs,13440,2012-05-03 00:00:00,2012-08-01 00:00:00,Jobcentre Plus


In [128]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp&~ckr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com
13231,68693588,Public Relations Officer,Uk,St.Marys International Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",26000,2013-10-06 00:00:00,2013-11-05 00:00:00,totaljobs.com
34379,69675549,Hr Manager (Parttime) 3 Days A Week ( Flexi Wo...,London,M. ARIA RECRUITMENT Ltd,part_time,permanent,Teaching Jobs,13522,2013-09-06 12:00:00,2013-11-05 12:00:00,jobs.guardian.co.uk
34780,71885279,Solicitor/ Disciplinary Case Manager,London,M. ARIA RECRUITMENT Ltd,full_time,permanent,Teaching Jobs,50000,2013-02-18 00:00:00,2013-05-19 00:00:00,jobs.guardian.co.uk
35142,68838621,Software Developer Devon,Devon,Recruit.UK,full_time,permanent,IT Jobs,27500,2013-05-07 15:00:00,2013-07-06 15:00:00,planetrecruit.com
37242,69188513,"No Experience, No Problem New Roles In Bristol",Uk,HelpMeGo.To Ltd,non-specified,non-specified,Sales Jobs,15600,2012-05-09 00:00:00,2012-07-08 00:00:00,salestarget.co.uk
39249,66537257,Kitchen Staff,Uk,J.D.Wetherspoons,part_time,non-specified,Hospitality & Catering Jobs,12480,2013-09-15 15:00:00,2013-09-29 15:00:00,Jobcentre Plus
39404,69768265,Venues & Events Energy Consultant,Evesham,E.on UK Plc,full_time,non-specified,Sales Jobs,22812,2012-01-30 00:00:00,2012-02-29 00:00:00,Jobcentre Plus


It looks like there is this Sand Resource company with a lot of information at the back

In [129]:
sand = df['Company'].str.contains(r'[sS]and\s[rR]esource')
df[sand]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources. A division of InterQuest Group...,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources. A division of InterQuest Group...,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com


It seems like these 'Sand Resources..' could be written as 'Sand Resources'

In [130]:
indices = list(df[sand].index)
for ind in indices:
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company',df['Company'][ind],'Sand Resources','Extra information','Removed extra information')
    # Replace company name with only Sand Resources
    df.loc[ind,'Company'] = 'Sand Resources'
df[sand]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com


In [131]:
df[sand]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
8128,71593642,Oracle Hr Functional Consultant/Oracle Ebusine...,Wiltshire,Sand Resources,non-specified,permanent,IT Jobs,45000,2013-07-22 12:00:00,2013-10-20 12:00:00,jobserve.com
9050,68627684,Oracle Hr Functional Support Analyst,Wiltshire,Sand Resources,non-specified,permanent,IT Jobs,34000,2013-03-26 12:00:00,2013-05-25 12:00:00,jobserve.com
9085,69574473,"C Net Developer (Python, Scrum, Agile, Ibm Web...",Coscote,Sand Resources,non-specified,contract,IT Jobs,32500,2013-05-17 12:00:00,2013-07-16 12:00:00,jobserve.com


In [132]:
dot_company = df['Company'].str.contains(r'\.')
df[dot_company&~coal&~dp&~ckr]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
13231,68693588,Public Relations Officer,Uk,St.Marys International Ltd,non-specified,permanent,"PR, Advertising & Marketing Jobs",26000,2013-10-06 00:00:00,2013-11-05 00:00:00,totaljobs.com
34379,69675549,Hr Manager (Parttime) 3 Days A Week ( Flexi Wo...,London,M. ARIA RECRUITMENT Ltd,part_time,permanent,Teaching Jobs,13522,2013-09-06 12:00:00,2013-11-05 12:00:00,jobs.guardian.co.uk
34780,71885279,Solicitor/ Disciplinary Case Manager,London,M. ARIA RECRUITMENT Ltd,full_time,permanent,Teaching Jobs,50000,2013-02-18 00:00:00,2013-05-19 00:00:00,jobs.guardian.co.uk
35142,68838621,Software Developer Devon,Devon,Recruit.UK,full_time,permanent,IT Jobs,27500,2013-05-07 15:00:00,2013-07-06 15:00:00,planetrecruit.com
37242,69188513,"No Experience, No Problem New Roles In Bristol",Uk,HelpMeGo.To Ltd,non-specified,non-specified,Sales Jobs,15600,2012-05-09 00:00:00,2012-07-08 00:00:00,salestarget.co.uk
39249,66537257,Kitchen Staff,Uk,J.D.Wetherspoons,part_time,non-specified,Hospitality & Catering Jobs,12480,2013-09-15 15:00:00,2013-09-29 15:00:00,Jobcentre Plus
39404,69768265,Venues & Events Energy Consultant,Evesham,E.on UK Plc,full_time,non-specified,Sales Jobs,22812,2012-01-30 00:00:00,2012-02-29 00:00:00,Jobcentre Plus
39791,69554526,Membership Adviser,Southampton,L.A Fitness,full_time,non-specified,Sales Jobs,14000,2012-06-08 12:00:00,2012-08-07 12:00:00,Jobcentre Plus
39909,69766368,Software Development Project/Programme Manager,Nottingham,E.on UK Plc,full_time,non-specified,IT Jobs,58706,2013-11-02 00:00:00,2014-01-01 00:00:00,Jobcentre Plus
40420,68450122,Skilled Senior Care Assistant,London,St.marys International Education and Employmen...,full_time,non-specified,Healthcare & Nursing Jobs,14976,2013-08-15 15:00:00,2013-10-14 15:00:00,Jobcentre Plus


Seems like we are done with the Company Column!

Do some last formatting

In [133]:
# Record to errorDict for capitalizing each word except for the value 'non-specified'
errorDict = addToErrorDict(errorDict,'ALL','ALL','Company','word','Word','Messy capitalization and white spaces','Capitalize each word and trim white spaces')
df['Company'] = df['Company'].apply(lambda x: x.title().strip() if (x != 'non-specified') else 'non-specified')
df['Company']

0           Resourcing Solutions
1        Orleston Associates Ltd
2                       Randstad
3                       Randstad
4                       Randstad
                  ...           
50748                       Gchq
50749                    Unknown
50750                   Barclays
50751        Carrington West Ltd
50752       Pursuit Training Ltd
Name: Company, Length: 50750, dtype: object

There is still one company labelled as unknown

In [134]:
unknown = df['Company'] == 'Unknown'
indices = list(df[unknown].index)
for ind in indices:
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Company','Unknown','non-specified','Name labeled as unknown','Set to non-specified')
    df.loc[ind,'Company'] = 'non-specified'
df['Company']

0           Resourcing Solutions
1        Orleston Associates Ltd
2                       Randstad
3                       Randstad
4                       Randstad
                  ...           
50748                       Gchq
50749              non-specified
50750                   Barclays
50751        Carrington West Ltd
50752       Pursuit Training Ltd
Name: Company, Length: 50750, dtype: object

Here, we are done with cleaning and formatting Company column.

Now, it is time to check the data for the Salary column.

### Task 2.10 Examining and Cleaning Salary Column

In [135]:
df['Salary'].describe()

count     50113
unique     3466
top       35000
freq       1706
Name: Salary, dtype: object

From the output above, it seems that in the dataset, Salary column has nominal values! The column would be converted into numeric.

In [136]:
try:
    # Try converting to numeric to see if it's possible
    df['Salary'] = pd.to_numeric(df['Salary'])
except Exception as e:
    print(e)

Unable to parse string "36K" at position 12


It seems like there are some non-digit values within the column. Thus, some cleaning is done before evaluating the values

In [137]:
null_salary = df['Salary'].isna()
df[null_salary]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
5,67944712,Theatre Lead/Deputy Manager,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,NaN,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
7,68346203,Test Analyst Join A Leading Fund Manager Ess...,Essex,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-04-12 00:00:00,2013-06-11 00:00:00,rengineeringjobs.com
17,70519783,Water Quality Engineer Middlesex,Uk,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-12-14 00:00:00,2014-01-13 00:00:00,rengineeringjobs.com
69,67945892,Senior Structural Engineer London,London,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-04-04 15:00:00,2013-06-03 15:00:00,rengineeringjobs.com
131,66905220,Business Development Manager Fm,Leicestershire,Fusion People,non-specified,permanent,Engineering Jobs,NaN,2013-07-31 12:00:00,2013-08-30 12:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50705,70417262,"Coventure Graduate Scheme Asset Management, H...",Uk,Co Venture,non-specified,non-specified,Sales Jobs,NaN,2013-03-01 00:00:00,2013-03-15 00:00:00,wikijob.co.uk
50722,72179080,Business Intelligence Solutions Architect,London,Selecture Global,non-specified,permanent,Accounting & Finance Jobs,NaN,2013-09-19 00:00:00,2013-12-18 00:00:00,frontofficebanking.com
50726,68718953,Trainee Broker Ote Uncapped,London,One Two Trade,non-specified,permanent,Accounting & Finance Jobs,NaN,2013-09-22 12:00:00,2013-10-22 12:00:00,frontofficebanking.com
50727,71802817,Tax Reporting Manager,London,Integral Search,non-specified,permanent,Accounting & Finance Jobs,NaN,2013-02-22 12:00:00,2013-04-23 12:00:00,frontofficebanking.com


From the output above it can be seen that there are null values for Salary. These values would be kept for further evaluation

In [138]:
# Get salaries that does not contain digits
salary_w_digit = df[~null_salary]['Salary'].str.match('\d+',na=False)
df[~null_salary][~salary_w_digit]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
719,66921939,"Systems Support Engineer Derby, East Midlands",Derby,V Selective Recruitment,non-specified,permanent,IT Jobs,,2013-07-09 00:00:00,2013-08-08 00:00:00,cv-library.co.uk
741,69012173,"Staff Nurse, Elderly Nursing Home, Taunton",Taunton,White Recruitment Ltd,non-specified,permanent,Healthcare & Nursing Jobs,,2013-01-14 00:00:00,2013-04-14 00:00:00,cv-library.co.uk
772,71296009,Research Development Manager Hosiery Exp,West Midlands,Interaction Recruitment,non-specified,permanent,Healthcare & Nursing Jobs,,2013-07-27 15:00:00,2013-09-25 15:00:00,cv-library.co.uk
788,65880148,Internal Sales Director Elearning,South West England,Aarca Sales Recruitment,non-specified,permanent,Sales Jobs,,2012-06-29 00:00:00,2012-07-29 00:00:00,cv-library.co.uk
820,69079710,Civil / Structural Maritime Engineer,London,Energi Recruitment Services Ltd,non-specified,permanent,Engineering Jobs,,2013-12-21 15:00:00,2014-03-21 15:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
50452,71109346,**** Month Ap/Treasury Manager Contract,West London,Investigo Ltd,full_time,contract,Accounting & Finance Jobs,,2013-08-16 00:00:00,2013-11-14 00:00:00,myjobs.cimaglobal.com
50482,71073530,Engineering Quality Auditor Calibration,West Yorkshire,Informatiq Consulting,full_time,permanent,Engineering Jobs,,2013-11-23 15:00:00,2014-01-22 15:00:00,JobSearch
50491,72579991,Finance Analyst ****K Pa Central Bristol Pe...,Avon,Morgan Mckinley,full_time,permanent,Accounting & Finance Jobs,,2013-05-09 00:00:00,2013-08-07 00:00:00,JobSearch
50551,71041572,Metals Analyst,London,Jobg8,full_time,permanent,Accounting & Finance Jobs,-,2012-05-19 00:00:00,2012-06-18 00:00:00,JustLondonJobs


From above it can be seen that other than missing values, there are also some other values that only contains values like "-" and " ". All these values would be set to none as well

In [139]:
for ind in list(df[~null_salary][~salary_w_digit].index):
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Salary','Not digit values','None','Missing Value','Convert dummy values to None')

# Replace nan and non-digit salaries with None
df.loc[null_salary,'Salary'] = None
df.loc[~null_salary&~salary_w_digit,'Salary'] = None
df[df['Salary'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
5,67944712,Theatre Lead/Deputy Manager,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,None,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
7,68346203,Test Analyst Join A Leading Fund Manager Ess...,Essex,Randstad,non-specified,permanent,Engineering Jobs,None,2013-04-12 00:00:00,2013-06-11 00:00:00,rengineeringjobs.com
17,70519783,Water Quality Engineer Middlesex,Uk,Randstad,non-specified,permanent,Engineering Jobs,None,2013-12-14 00:00:00,2014-01-13 00:00:00,rengineeringjobs.com
69,67945892,Senior Structural Engineer London,London,Randstad,non-specified,permanent,Engineering Jobs,None,2013-04-04 15:00:00,2013-06-03 15:00:00,rengineeringjobs.com
131,66905220,Business Development Manager Fm,Leicestershire,Fusion People,non-specified,permanent,Engineering Jobs,None,2013-07-31 12:00:00,2013-08-30 12:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50710,68390860,Cto/It Director/Cio/ Technical Director,Reading,Eligo Recruitment,full_time,non-specified,IT Jobs,None,2012-05-01 00:00:00,2012-06-30 00:00:00,jobs.perl.org
50722,72179080,Business Intelligence Solutions Architect,London,Selecture Global,non-specified,permanent,Accounting & Finance Jobs,None,2013-09-19 00:00:00,2013-12-18 00:00:00,frontofficebanking.com
50726,68718953,Trainee Broker Ote Uncapped,London,One Two Trade,non-specified,permanent,Accounting & Finance Jobs,None,2013-09-22 12:00:00,2013-10-22 12:00:00,frontofficebanking.com
50727,71802817,Tax Reporting Manager,London,Integral Search,non-specified,permanent,Accounting & Finance Jobs,None,2013-02-22 12:00:00,2013-04-23 12:00:00,frontofficebanking.com


Now that dummy values are set as None, it is time to check for invalid values

In [140]:
# Get salary that is 0
salary_is_0 = df['Salary'].str.fullmatch('0',na=False)
df[salary_is_0]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
10453,68712324,Private Client Director Guildford,Guildford,Pro Recruitment Group,non-specified,permanent,Accounting & Finance Jobs,0,2013-04-20 00:00:00,2013-07-19 00:00:00,totaljobs.com
10461,68068322,Clinical Nurse Manager (Rn) Barchester,Ashford,Barchester Healthcare,non-specified,permanent,Healthcare & Nursing Jobs,0,2013-09-04 12:00:00,2013-12-03 12:00:00,totaljobs.com
10503,71680705,Ppc Client Manager (Entry Level),South East London,Propel,non-specified,permanent,"PR, Advertising & Marketing Jobs",0,2012-12-15 15:00:00,2013-03-15 15:00:00,totaljobs.com
10513,72445032,Contract Manager / Project Manager,Winchester,Red Kite Selection,non-specified,permanent,Engineering Jobs,0,2012-08-26 15:00:00,2012-10-25 15:00:00,totaljobs.com
10524,68064583,Exclusive Qualified Social Workers,Lincoln,Action First Recruitment,non-specified,permanent,Healthcare & Nursing Jobs,0,2013-08-15 12:00:00,2013-09-14 12:00:00,totaljobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50567,70086312,Assistant Wealth Manager,London,Roland Dowell,non-specified,permanent,Accounting & Finance Jobs,0,2012-03-13 12:00:00,2012-06-11 12:00:00,ifaonlinejobs.co.uk
50573,68511191,Financial Planner International Eb Consultancy,The City,Roland Dowell,non-specified,permanent,Accounting & Finance Jobs,0,2012-07-11 00:00:00,2012-08-10 00:00:00,ifaonlinejobs.co.uk
50580,68511163,"Office Based Mortgage Broker, Cambridge",Cambridge,Morris Jones,non-specified,permanent,Accounting & Finance Jobs,0,2012-12-16 15:00:00,2013-01-15 15:00:00,ifaonlinejobs.co.uk
50737,69693532,Locum Gp With Potential For Partnership,Uk,Mayfield Medical Centre,part_time,non-specified,Healthcare & Nursing Jobs,0,2013-01-06 12:00:00,2013-01-20 12:00:00,jobs.gponline.com


It can also be seen that some Salary is set as 0, since these are all jobs and not volunteering events, every salary should be more than 0! These data would be set to None as well

In [141]:
for ind in df[salary_is_0].index:
    # Record to errorDict
    errorDict = addToErrorDict(errorDict,ind,df['Id'][ind],'Salary','0','None','Value is 0','Set from 0 to None instread')

# Replace those values with none
df.loc[salary_is_0,'Salary'] = None
df[df['Salary'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
5,67944712,Theatre Lead/Deputy Manager,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,None,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
7,68346203,Test Analyst Join A Leading Fund Manager Ess...,Essex,Randstad,non-specified,permanent,Engineering Jobs,None,2013-04-12 00:00:00,2013-06-11 00:00:00,rengineeringjobs.com
17,70519783,Water Quality Engineer Middlesex,Uk,Randstad,non-specified,permanent,Engineering Jobs,None,2013-12-14 00:00:00,2014-01-13 00:00:00,rengineeringjobs.com
69,67945892,Senior Structural Engineer London,London,Randstad,non-specified,permanent,Engineering Jobs,None,2013-04-04 15:00:00,2013-06-03 15:00:00,rengineeringjobs.com
131,66905220,Business Development Manager Fm,Leicestershire,Fusion People,non-specified,permanent,Engineering Jobs,None,2013-07-31 12:00:00,2013-08-30 12:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50726,68718953,Trainee Broker Ote Uncapped,London,One Two Trade,non-specified,permanent,Accounting & Finance Jobs,None,2013-09-22 12:00:00,2013-10-22 12:00:00,frontofficebanking.com
50727,71802817,Tax Reporting Manager,London,Integral Search,non-specified,permanent,Accounting & Finance Jobs,None,2013-02-22 12:00:00,2013-04-23 12:00:00,frontofficebanking.com
50737,69693532,Locum Gp With Potential For Partnership,Uk,Mayfield Medical Centre,part_time,non-specified,Healthcare & Nursing Jobs,None,2013-01-06 12:00:00,2013-01-20 12:00:00,jobs.gponline.com
50744,71280413,Project Manager / Payments (Essential) / Watford,Uk,Rdf,non-specified,permanent,IT Jobs,None,2013-10-05 15:00:00,2013-11-04 15:00:00,cardandpaymentjobs.com


In [142]:
# Get null salary to exclude them
null_salary = df['Salary'].isna()
# Get salary with digits and maybe decimal places to exclude them
salary_w_digit = df[~null_salary]['Salary'].str.fullmatch('\d*\.?\d*',na=False)
# Get invalid salaries
df[~null_salary][~salary_w_digit]
# 36K, /Year, To , per Year invalid values found

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
12,67946056,Mobile Senior Mri Radiographer Central/ East,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,36K,2013-06-21 12:00:00,2013-07-05 12:00:00,rengineeringjobs.com
101,69669920,Software And Systems Test,Birmingham,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,33000/Year,2013-07-19 15:00:00,2013-08-02 15:00:00,rengineeringjobs.com
106,67946543,Junior Software Consultant Financial Services,Guildford,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,40000/Year,2013-08-01 12:00:00,2013-08-15 12:00:00,rengineeringjobs.com
320,72247697,"Site/Project Manager Dunstable, Bedfordshire",Dunstable,Randstad,non-specified,contract,Engineering Jobs,35480 To 37480,2013-11-22 00:00:00,2013-12-06 00:00:00,rengineeringjobs.com
517,69669916,Senior Scientist Biosimilars Mass Spectrometrist,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,30500 To 32500,2013-12-10 15:00:00,2014-01-09 15:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
49235,69734297,Contracts Engineer (Mechanical),London,Bluefire Consulting Ltd,non-specified,permanent,Engineering Jobs,30000/Year,2013-06-17 00:00:00,2013-09-15 00:00:00,londonjobs.co.uk
49436,69092365,Ifa Professional Practice,Belfast,Johnstongreer Ltd,full_time,permanent,Accounting & Finance Jobs,42500per Year,2013-09-29 12:00:00,2013-10-29 12:00:00,nijobs.com
49519,67784900,Economic Planner,Kent,Macdonald Co,full_time,permanent,Engineering Jobs,25000 To 27000,2012-08-02 12:00:00,2012-10-01 12:00:00,jobs.planningresource.co.uk
50253,69555936,Pensions Technician,Brighton,Hays,non-specified,permanent,Accounting & Finance Jobs,43000 To 45000,2013-04-17 12:00:00,2013-05-17 12:00:00,professionalpensionsjobs.com


In [143]:
# Exclude previously found values to get more different invalid values
invalid_salary1 = df[~null_salary]['Salary'].str.contains(r'Year|To|K')
df[~null_salary][~invalid_salary1 & ~salary_w_digit]
# ~, per Hour,- invalid values found

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
2124,67904794,Cisco Network Consultant,The City,Jam Recruitment Ltd,non-specified,permanent,IT Jobs,49000 ~ 51000,2013-04-05 00:00:00,2013-07-04 00:00:00,cv-library.co.uk
3776,69001894,Vending Operator,Uk,Fmcv,part_time,non-specified,Hospitality & Catering Jobs,9.99 p/h,2012-07-20 00:00:00,2012-09-18 00:00:00,cv-library.co.uk
3780,70764179,"Rgn Sexual Health, Family Planning",Kent,Support Services Group,part_time,non-specified,Healthcare & Nursing Jobs,14.43 per Hour,2012-05-30 00:00:00,2012-06-29 00:00:00,cv-library.co.uk
3795,69560030,Aluminium Fabricators Required Immediate Start,Great Yarmouth,Workshop Recruitment,non-specified,contract,Engineering Jobs,30680 ~ 32680,2013-11-12 12:00:00,2014-01-11 12:00:00,cv-library.co.uk
4021,68217483,Mechanical Maintenance Engineer / Fitter,London,Randstad Construction,non-specified,contract,Engineering Jobs,26840 ~ 28840,2013-12-01 15:00:00,2013-12-15 15:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
45103,69868543,Associate Transportation Planner,Uk,Resourcing Solutions Ltd,non-specified,permanent,Engineering Jobs,29000 ~ 31000,2012-10-02 15:00:00,2012-12-01 15:00:00,justengineers.net
45521,71911455,Ks**** Primary Teacher North Leicester,Loughborough,non-specified,non-specified,contract,Teaching Jobs,25400 ~ 27400,2013-03-02 12:00:00,2013-03-16 12:00:00,hays.co.uk
46432,69035071,Primary Supply Teachers Ks**** & Ks****,Brighton,Capita Education,part_time,contract,Teaching Jobs,12.36 per Hour,2012-12-28 00:00:00,2013-01-11 00:00:00,jobs.newstatesman.com
47289,67770221,Design Engineer Autocad Electrical Designs,Witham,non-specified,non-specified,non-specified,Engineering Jobs,26000 - 28000,2013-03-09 15:00:00,2013-04-08 15:00:00,britishjobsonthe.net


In [144]:
# Exclude previously found values to get more different invalid values
invalid_salary2 = df[~null_salary]['Salary'].str.contains(r'~|[Hh]our|\-')
df[~null_salary][~invalid_salary1 & ~salary_w_digit & ~invalid_salary2]
#p/h invalid values found, nothing else

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
3776,69001894,Vending Operator,Uk,Fmcv,part_time,non-specified,Hospitality & Catering Jobs,9.99 p/h,2012-07-20 00:00:00,2012-09-18 00:00:00,cv-library.co.uk
4892,70758515,Part Time Home Based Compliance Support Consul...,Birmingham,Compliance Resource Solutions Ltd,part_time,non-specified,Accounting & Finance Jobs,15.09 p/h,2012-12-09 15:00:00,2013-01-08 15:00:00,cv-library.co.uk
9524,69214043,Performing Arts Drama Music Art Qts Supply Tea...,Tamworth,Aspire People Ltd,part_time,permanent,Teaching Jobs,13.79 p/h,2013-09-09 00:00:00,2013-10-09 00:00:00,fish4.co.uk
26028,68363853,Care Worker Job Surrey,Guildford,non-specified,part_time,non-specified,Healthcare & Nursing Jobs,12.48 p/h,2013-09-23 15:00:00,2013-11-22 15:00:00,careworx.co.uk
28138,71091672,Health Visitor (Dn**** Area) Nottingham,Nottingham,The A24 Group,part_time,non-specified,Healthcare & Nursing Jobs,14.61 p/h,2013-07-07 00:00:00,2013-07-21 00:00:00,staffnurse.com
29766,68347305,Marketing Executive (Financial Services / Tech...,London,Easywebrecruitment,part_time,non-specified,"PR, Advertising & Marketing Jobs",15.33 p/h,2012-07-22 15:00:00,2012-08-05 15:00:00,jobsite.co.uk
40272,69150013,Esol **** Posts,Leicester,Cmt Uk Pedagogy Ltd,part_time,non-specified,Teaching Jobs,14.71 p/h,2012-01-31 00:00:00,2012-03-01 00:00:00,Jobcentre Plus
40346,71600933,"Senior Training Officer, Safe Guarding In School",London,Wandsworth Borough Council,part_time,non-specified,Teaching Jobs,9.38 p/h,2012-06-22 15:00:00,2012-07-22 15:00:00,Jobcentre Plus
40836,69960958,Part Time Tutor In New Media Development,Bangor,Bangor University,part_time,non-specified,Teaching Jobs,14.57 p/h,2012-04-18 00:00:00,2012-06-17 00:00:00,Jobcentre Plus
44220,69731349,Careers & Employment Worker,Uk,non-specified,part_time,non-specified,Teaching Jobs,11.0 p/h,2013-09-28 15:00:00,2013-10-12 15:00:00,jobstoday.co.uk


From the 3 cells above, it can be seen that 'Salary' comes in formats like
<ul>
<li>XX p/h</li>
<li>XX - XX</li>
<li>XX To XX</li>
<li>XX ~ XX</li>
<li>XXk</li>
<li>XX /Year</li>
<li>XX per Year</li>
<li>XX per Hour</li>
</ul>

All these values would be reformatted to its correct value

In [145]:
# Function to reformat 'Salary'
def col_salary(data):
    range_values = r'(\d*\.?\d*)\s*(?:[tT]o|~|\-)\s*(\d*\.?\d*)'
    per_year = r'^(\d+)\s?(?:per|/)\s?[yY]ear$'
    k_salary = r'(\d*\.?\d*)[kK]'
    per_hour = r'(\d*\.?\d*)\s*p(?:/|er)\s*[hH](?:our)?'
    if data == 'None':
        return None
    else:
        # Yearly
        if re.fullmatch(per_year,data):
            floats = [float(x) for x in re.match(per_year,data).groups()]
            return str(floats[0])
        # Range values
        elif re.fullmatch(range_values,data):
            floats = [float(x) for x in re.match(range_values,data).groups()]
            return str(sum(floats)/2)
        # Values with K
        elif re.fullmatch(k_salary,data):
            floats = [float(x) for x in re.match(k_salary,data).groups()]
            return str(floats[0]*1000)
        # Salary per hour
        elif re.fullmatch(per_hour,data):
            floats = [float(x) for x in re.match(per_hour,data).groups()]
            return str(floats[0]*37.5*52)
        else: 
            # Print and return he data if it was not correctly processed
            if not re.fullmatch(r'\d*\.?\d*',data):
                # Data not filtered by regex
                print(data)
            else:
                return data

In [146]:
# Record to error dict
for row in df.index:
    val = df['Salary'][row]
    if val != None:
        new_val = col_salary(df['Salary'][row])
        if val != new_val:
            fix = 'Change '+str(val)+' To '+str(new_val)
            errorDict = addToErrorDict(errorDict,row,df['Id'][row],'Salary',val,new_val,'Incorrect Format',fix)

In [147]:
# Format salary using the function col_salary
df['Salary'] = df['Salary'].apply(lambda x: col_salary(str(x)))

In [148]:
valid_salary = df['Salary'].str.fullmatch('\d*\.?\d*',na=False)
df[~df['Salary'].isna()][~valid_salary]

C:\Users\User\AppData\Local\Temp\ipykernel_2328\1645166485.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[~df['Salary'].isna()][~valid_salary]


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


From the output above, it can be seen that all values of Salary are in the correct format, with none values as none and others as numeric values. Since all are in the correct format now, we will convert "Salary" into numeric values with 2 decimal places

In [149]:
df['Salary'] = pd.to_numeric(df['Salary'])
df['Salary'] = df['Salary'].round(2)
errorDict = addToErrorDict(errorDict,'ALL','ALL','Salary','String','Numeric','Incorrect format','Switch to numeric format')

From here, we are done converting Salary column into numeric values

### Task 2.11 Checking for duplicates

Lastly, let's check for and drop duplicates

In [150]:
df[df.duplicated(['Company','ContractType','ContractTime','Salary','OpenDate','Location'],keep=False)]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
50677,60905059,Pricing Analyst,Gloucestershire,Jobg8,full_time,permanent,Accounting & Finance Jobs,35000.0,2012-02-05 15:00:00,2012-05-05 15:00:00,jobsfinancial.com
50679,71295301,Pricing Analyst,Gloucestershire,Jobg8,full_time,permanent,Accounting & Finance Jobs,35000.0,2012-02-05 15:00:00,2012-05-05 15:00:00,jobsfinancial.com


In [151]:
duplicates = df.duplicated(['Company','ContractType','ContractTime','Salary','OpenDate','Location'])
for ind in df.index[duplicates]:
    errorDict = addToErrorDict(errorDict,'ALL',df['Id'][ind],'ALL','ALL','ALL','Duplicated values','Removed duplicated values')
df.drop_duplicates(['Company','ContractType','ContractTime','Salary','OpenDate','Location'],inplace=True)

Now, let's move on to Salary imputation

### Task 2.12 Salary Imputation

In [152]:
df['Salary'].describe().round(2)

count       48252.00
mean        34884.87
std         66100.50
min          5000.00
25%         23000.00
50%         31200.00
75%         42500.00
max      10000000.00
Name: Salary, dtype: float64

In [153]:
# Get box plot stats of salary
salaries = df['Salary'][~np.isnan(df['Salary'])]
stats = boxplot_stats(salaries)
stats

[{'mean': 34884.87406739617,
  'iqr': 19500.0,
  'cilo': 31060.627655994584,
  'cihi': 31339.372344005416,
  'whishi': 71500.0,
  'whislo': 5000.0,
  'fliers': array([87500., 80000., 77500., ..., 93600., 90000., 90000.]),
  'q1': 23000.0,
  'med': 31200.0,
  'q3': 42500.0}]

It seems like outliers are approximately 70k to 90k. Let's filter these values out

In [154]:
# Lower bound of valid salary
more = df['Salary'] > stats[0]['whislo']
# Upper bound of valid salary
fil = df['Salary'] < 80000

In [155]:
# Get data with no salaries
df[df['Salary'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
5,67944712,Theatre Lead/Deputy Manager,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,NaN,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
7,68346203,Test Analyst Join A Leading Fund Manager Ess...,Essex,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-04-12 00:00:00,2013-06-11 00:00:00,rengineeringjobs.com
17,70519783,Water Quality Engineer Middlesex,Uk,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-12-14 00:00:00,2014-01-13 00:00:00,rengineeringjobs.com
69,67945892,Senior Structural Engineer London,London,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-04-04 15:00:00,2013-06-03 15:00:00,rengineeringjobs.com
131,66905220,Business Development Manager Fm,Leicestershire,Fusion People,non-specified,permanent,Engineering Jobs,NaN,2013-07-31 12:00:00,2013-08-30 12:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50726,68718953,Trainee Broker Ote Uncapped,London,One Two Trade,non-specified,permanent,Accounting & Finance Jobs,NaN,2013-09-22 12:00:00,2013-10-22 12:00:00,frontofficebanking.com
50727,71802817,Tax Reporting Manager,London,Integral Search,non-specified,permanent,Accounting & Finance Jobs,NaN,2013-02-22 12:00:00,2013-04-23 12:00:00,frontofficebanking.com
50737,69693532,Locum Gp With Potential For Partnership,Uk,Mayfield Medical Centre,part_time,non-specified,Healthcare & Nursing Jobs,NaN,2013-01-06 12:00:00,2013-01-20 12:00:00,jobs.gponline.com
50744,71280413,Project Manager / Payments (Essential) / Watford,Uk,Rdf,non-specified,permanent,IT Jobs,NaN,2013-10-05 15:00:00,2013-11-04 15:00:00,cardandpaymentjobs.com


These null values as well as values out of range would be used for imputation

In [156]:
# Set values out of range as None
df.loc[~fil,'Salary'] = None
df.loc[~more,'Salary'] = None
df[df['Salary'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
5,67944712,Theatre Lead/Deputy Manager,Uk,Orleston Associates Ltd,non-specified,non-specified,Engineering Jobs,NaN,2012-04-24 00:00:00,2012-05-24 00:00:00,rengineeringjobs.com
7,68346203,Test Analyst Join A Leading Fund Manager Ess...,Essex,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-04-12 00:00:00,2013-06-11 00:00:00,rengineeringjobs.com
17,70519783,Water Quality Engineer Middlesex,Uk,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-12-14 00:00:00,2014-01-13 00:00:00,rengineeringjobs.com
69,67945892,Senior Structural Engineer London,London,Randstad,non-specified,permanent,Engineering Jobs,NaN,2013-04-04 15:00:00,2013-06-03 15:00:00,rengineeringjobs.com
131,66905220,Business Development Manager Fm,Leicestershire,Fusion People,non-specified,permanent,Engineering Jobs,NaN,2013-07-31 12:00:00,2013-08-30 12:00:00,rengineeringjobs.com
...,...,...,...,...,...,...,...,...,...,...,...
50727,71802817,Tax Reporting Manager,London,Integral Search,non-specified,permanent,Accounting & Finance Jobs,NaN,2013-02-22 12:00:00,2013-04-23 12:00:00,frontofficebanking.com
50730,69815649,Futures Dma Desk Support,London,Agora,non-specified,permanent,Accounting & Finance Jobs,NaN,2013-06-13 15:00:00,2013-08-12 15:00:00,frontofficebanking.com
50737,69693532,Locum Gp With Potential For Partnership,Uk,Mayfield Medical Centre,part_time,non-specified,Healthcare & Nursing Jobs,NaN,2013-01-06 12:00:00,2013-01-20 12:00:00,jobs.gponline.com
50744,71280413,Project Manager / Payments (Essential) / Watford,Uk,Rdf,non-specified,permanent,IT Jobs,NaN,2013-10-05 15:00:00,2013-11-04 15:00:00,cardandpaymentjobs.com


Now its time to build a model and impute on the empty values!

In [157]:
# Use attributes such as 'ContractTime','ContractType' and 'Category' for training the model
dataset = df[fil&more][['Salary','ContractTime','ContractType','Category']]

# Use pd.get_dummies to turn categorical values into numerical ones
dataset = pd.get_dummies(dataset)
# encoder = LabelEncoder()

# Get columns of dataset that is not Salary
cols_xsalary = [col for col in list(dataset.columns) if col !='Salary']

# Split dataset into training and testing
X_train_xna, X_test, y_train_xna, y_test = train_test_split(dataset.loc[:,cols_xsalary],dataset.loc[:,['Salary']],random_state=123)

# Use Decision Tree Regressor for this dataset
dtr_xna = DecisionTreeRegressor()
dtr_xna.fit(X_train_xna,y_train_xna)

print('r-squared for Salary fitting = ',dtr_xna.score(dataset[cols_xsalary],dataset['Salary']).round(2))

r-squared for Salary fitting =  0.23


In [158]:
# Set out of range values as none
df.loc[~fil,'Salary'] = None
df.loc[~more,'Salary'] = None

In [159]:
# Get data with null salary
imputed_salaries_na = df[df['Salary'].isna()][['Salary','ContractTime','ContractType','Category']].copy() 
imputed_salaries_na = pd.get_dummies(imputed_salaries_na)

# Get prediction from the trained model
imputed_salaries_na.loc[:,'Salary']  = dtr_xna.predict(imputed_salaries_na.drop(['Salary'],axis=1))
imputed_salaries_na.loc[:,'Salary']

# Combine the predicted values with the existing dataset to form a full training set
X_train = pd.concat([X_train_xna,imputed_salaries_na[cols_xsalary]])
X_train

y_train = pd.concat([y_train_xna,imputed_salaries_na.loc[:,['Salary']]])
y_train.shape

# Train model on newly created model
dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)

print('r-squared for Decision Tree Regressor = ',dtr.score(X_test,y_test))

r-squared for Decision Tree Regressor =  0.23333770409825338


In [160]:
df.loc[df['Salary'].isna(),'Salary'] = imputed_salaries_na['Salary']
df[df['Salary'].isna()]

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName


After imputation, it can be seen that every row of data has a salary value!

### Task 2.13 Final Format Checking

Before saving the file, let's convert everything to its correct format

In [161]:
# Convert object columns into strings
errorDict = addToErrorDict(errorDict,'ALL','ALL','Title,Location,Company,ContractType,ContractTime,Category,SourceName','Object','String','In Object format','Convert to String format')
df[['Title','Location','Company','ContractType','ContractTime','Category','SourceName']] = df[['Title','Location','Company','ContractType','ContractTime','Category','SourceName']].astype('string')

In [162]:
# Check namings and data types of each column
df.dtypes

Id                       int64
Title                   string
Location                string
Company                 string
ContractType            string
ContractTime            string
Category                string
Salary                 float64
OpenDate        datetime64[ns]
CloseDate       datetime64[ns]
SourceName              string
dtype: object

In [163]:
df['ContractType'].value_counts()

non-specified    37883
full_time        11283
part_time         1583
Name: ContractType, dtype: Int64

In [164]:
df['ContractTime'].value_counts()

permanent        31052
non-specified    14001
contract          5696
Name: ContractTime, dtype: Int64

In [165]:
df['Category'].value_counts()

IT Jobs                             13170
Healthcare & Nursing Jobs            8066
Engineering Jobs                     7517
Accounting & Finance Jobs            6770
Sales Jobs                           4869
Hospitality & Catering Jobs          4376
Teaching Jobs                        3455
PR, Advertising & Marketing Jobs     2526
Name: Category, dtype: Int64

In [166]:
# Round string into 2 decimal places
errorDict = addToErrorDict(errorDict,'ALL','ALL','Salary','Unknown decimal places','2 decimal places','Has unknown decimal places','Convert to 2 decimal places')
df['Salary'].round(decimals = 2)

0        14400.00
1        24500.00
2        17250.00
3        46500.00
4        45000.00
           ...   
50748    25400.00
50749    39340.76
50750    36000.00
50751    20000.00
50752    18000.00
Name: Salary, Length: 50749, dtype: float64

Now that all data is supposedly in its right format, its time to save them into a file!

## Saving data
Save the parsed and cleaned data as well as the error list, named as:
- \<student\_id\>_dataset1_solution.csv
- \<student\_id\>_errorlist.csv

In [167]:
# Save cleaned dataset into the csv file, and exclude the index column
df.to_csv('s3936897_dataset1_solution.csv',index=False)

In [168]:
# errorDict was a dictionary, after all values are inserted, it is converted into a dataframe
# The columns and values are checked before saving into a csv file
errorlist = pd.DataFrame(errorDict)
errorlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76022 entries, 0 to 76021
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   indexOfdf   76022 non-null  object
 1   Id          76022 non-null  object
 2   ColumnName  76022 non-null  object
 3   Original    76022 non-null  object
 4   Modified    76022 non-null  object
 5   ErrorType   76022 non-null  object
 6   Fixing      76022 non-null  object
dtypes: object(7)
memory usage: 4.1+ MB


In [169]:
errorlist.to_csv('s3936897_errorlist.csv')

## Summary

It is unfortunate that the model did very badly, possible ways of improving this could include position extraction from the title ('Senior','Junior','Graduate').

There are still plenty errors within the dataset that could have been missed, given more time and computing resources, more detailed information extraction is definitely possible